# Similarity search example

## Imports

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import pandas as pd
import shelve
import os

## Define the query vector (In this example we use GSM1001648 vector, to find TOP300 most similar samples to our query vector)

### You can find vectors of interest by accessing the Qdrant UI at http://0.0.0.0:6333/dashboard, choosing the collection and copying vector to the clipboard (bottom-left corner of each data point, next to 'Default vector')

In [14]:
query_vector = [5.0385623,2.4159524,2.2303903,7.193359,4.629608,2.2201774,2.2127523,2.592439,2.2127523,2.2878864,5.7986493,8.137515,2.2127523,3.0547462,2.2127523,2.2127523,5.590464,7.9178715,4.610931,4.195082,5.9650645,8.492918,2.2127523,7.6974607,9.845057,6.1340985,5.514815,6.573873,6.998206,4.5794196,8.356517,7.8751287,7.573997,3.5006654,5.5229325,6.132975,6.322322,2.334043,2.2127523,2.3762584,2.635231,8.941507,3.3553398,4.744823,4.4405546,7.408116,3.514888,3.022614,3.3973286,9.278018,2.2127523,2.2127523,2.2127523,3.9361718,7.148775,2.841094,3.3996832,7.1055083,7.8108187,2.2127523,2.2127523,2.2539182,2.2127523,9.623834,7.006707,4.412109,6.4897366,3.1087327,2.2127523,2.2243404,3.9193952,2.2127523,2.3168426,9.492627,4.708,9.321335,8.304122,5.107362,5.7033353,6.831286,4.2586274,2.2127523,2.2127523,2.2127523,2.2307823,7.490826,4.141687,7.9410925,2.2127523,6.776544,7.389911,7.643886,7.500602,6.3361893,5.365998,2.2127523,9.907129,5.7288213,8.678709,6.9940934,9.702447,5.224319,5.974196,3.889758,4.983589,7.65852,6.138072,6.398993,4.0853376,8.094985,6.218647,6.5330415,2.6335802,4.7612195,3.4591236,7.445961,2.2127523,10.072798,6.675453,3.5461814,2.771943,2.2127523,2.2350926,2.2127523,3.6656222,2.2341743,2.2127523,2.2127523,2.2193525,2.2127523,2.2127523,2.2198024,8.6982,7.4819126,8.691389,2.2728293,2.2171788,2.2479267,2.3902285,2.212873,2.6662195,2.3494322,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.8729844,2.2907236,2.2127523,2.2136261,2.2127523,2.6555173,2.2127523,2.2455506,2.2127523,2.7388573,2.5958817,2.591402,2.2127523,2.2127523,2.2127523,2.4223845,2.2127523,2.2127523,2.443757,4.61886,2.213301,10.116313,2.2127523,2.2128427,2.2127523,2.2127523,2.2127523,2.2127523,2.3452795,2.2127523,4.6072445,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.330719,2.2710824,2.6474566,5.6654224,4.5175138,3.938997,2.2127523,2.3324764,5.6253567,2.3637064,2.66037,2.2127523,2.2127523,3.2207136,2.2127523,2.5278487,2.2216887,2.2869666,2.2127523,2.2326446,2.2127523,2.2127523,2.2127523,2.2127523,2.9553864,2.2127523,3.7968369,2.9244478,2.2127523,2.2127523,2.2127523,7.452842,2.2127523,2.2127523,2.2127523,2.9250615,2.2127523,2.2127523,2.2127523,2.4451408,3.0655212,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,11.574657,2.2174742,2.2127523,2.2127523,2.2127523,2.2127523,4.9139743,8.594446,6.190254,6.312739,3.620522,3.0796192,6.328872,8.117781,7.387771,3.3897777,10.17362,5.445501,7.345658,10.407979,3.4512146,4.830577,6.3217897,4.6029296,4.754099,7.1304126,11.032126,4.2195497,7.8222713,7.342092,4.9900093,8.027024,4.587527,2.4521954,2.3413365,5.6984167,2.2657385,7.388549,2.6873395,2.2127523,3.764897,2.2785487,8.694112,2.2127523,7.142416,5.582633,5.8988056,7.2936525,2.39657,2.2127523,8.660744,5.1929193,2.2127523,6.214246,5.2921925,9.316971,6.5983996,7.0311637,5.9717407,3.659274,8.084507,8.664909,11.4449215,6.4079447,2.911191,2.8732162,2.2127523,2.9238749,8.11246,2.2172248,2.223927,2.2205868,5.8236027,3.5356803,9.703501,9.394382,5.9834223,10.261379,2.3369472,2.2127523,2.2127523,2.2127523,6.3400426,2.4036317,3.254576,7.154713,3.9482555,2.2127523,5.4565735,2.2127523,14.64412,8.969976,2.9487503,11.220686,4.44154,2.6488404,9.663403,2.2127523,2.2165368,2.2127523,2.2127523,2.2127523,6.9779806,2.2413979,2.2127523,9.252299,11.4632225,7.75691,8.267137,8.92105,9.658677,3.9527082,2.2419152,2.7924511,2.2233407,5.211913,5.899789,5.6115417,2.2127523,3.0941815,9.741957,3.9703057,3.078254,6.511474,5.100598,2.7829223,4.675758,2.2127523,2.3417714,6.5550494,7.2839384,6.0047336,2.2128391,3.9944694,2.7889597,8.921494,2.2127523,3.0507493,6.00417,8.073876,2.2127523,4.388509,6.7297955,2.2127523,2.2127523,2.2142117,2.2205193,2.2131982,4.1788697,2.2127523,2.2127523,2.2127523,4.923349,2.2193005,2.3491952,2.2127523,2.2127523,2.2127523,5.050789,6.9447927,6.6511936,3.8658967,3.9956925,4.2293916,3.3201191,2.7089827,2.3877118,2.2129314,2.2165108,2.2127523,2.2127523,4.136814,2.2127523,2.757205,2.2127523,2.9073327,2.2398393,2.2127523,3.449311,3.5140235,2.8891888,2.2419925,2.7984123,2.3693478,3.950544,2.2127523,5.748821,8.039041,12.610396,3.3954356,2.2127523,2.2127523,7.6041665,3.326376,4.0476108,5.263447,6.0120435,5.4074197,3.3429587,2.8650997,2.2621207,2.2127523,3.6775055,2.2405143,7.1737947,4.345706,2.680194,7.376509,5.6123724,2.2127523,3.653524,2.2298791,2.2127523,7.38211,2.2827418,10.381105,2.3508348,2.252599,2.326997,6.3737054,2.9347653,3.489709,9.744935,2.2127523,2.2127523,3.7936158,9.215566,2.5448697,2.2127523,9.960115,7.4127436,9.297952,2.2865026,9.20849,7.856472,2.398955,2.586471,9.555603,8.112727,8.566151,2.4530478,6.799298,2.2127523,6.763284,5.6189547,7.8371463,4.2884135,2.7353504,7.9731,6.570547,2.2570384,2.2127523,2.2127523,5.5677204,2.284602,4.6949987,3.4860692,5.5946894,2.2127523,4.4719844,6.0890927,7.932727,9.09724,6.134048,4.8599544,4.750332,8.132635,5.9549675,6.5540466,9.698913,2.2350953,5.7965226,2.2127523,3.5910184,2.2708085,2.2127523,2.2271352,3.7176468,2.2184963,2.261554,2.2127523,2.2230847,2.2127523,3.7958176,2.4228415,3.5567393,3.3282845,2.2127523,2.2127523,2.2339914,2.2127523,7.7772913,5.172399,4.3153434,6.758201,3.2311635,3.2189987,5.7513614,2.2127523,2.451708,6.036031,5.2856207,6.4554873,2.2127523,3.5553796,2.2127523,9.046539,8.777141,5.442075,7.686708,4.0500507,2.3040016,2.349884,4.2696033,7.3717513,5.4238763,4.9828005,2.2127523,2.2245107,2.9308026,2.2127523,4.277632,5.3590536,6.782964,2.5304682,7.2978244,5.465542,9.645942,2.732374,4.8205647,6.687367,5.866515,4.6609774,6.5715923,9.91342,6.879697,2.805664,2.9714298,5.6176233,3.8690655,5.0179224,7.79329,6.8556805,8.730944,4.814711,2.2127523,5.366346,6.339143,3.3675287,4.045783,5.769656,2.2139955,2.2127523,6.882045,9.444157,8.385583,7.4826293,3.19795,4.310047,6.0554905,5.8189917,10.274905,2.8150558,8.333353,7.4310384,2.2178729,4.66133,2.2127523,11.204221,8.331568,2.7265575,7.202816,6.2181306,2.2127523,4.855718,8.590728,3.1862087,10.299179,8.849882,8.181083,8.009669,2.2127523,2.451608,4.6295867,2.5967755,2.2127523,2.2127523,6.5495124,5.05636,5.9665594,5.6182003,2.254099,3.3813791,9.551136,2.2127523,3.9220684,2.36309,3.4810889,2.2127523,2.2127523,5.042277,7.100284,2.2127523,9.784163,2.6365125,2.2127523,2.222229,2.2127523,2.2127523,5.6415377,4.903551,2.2130334,2.3630452,2.2127523,2.2127523,9.812953,7.503622,2.3717613,6.6709266,2.3109803,2.3734043,2.2127523,2.2127523,2.2127523,3.5957155,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.784836,3.646997,4.007686,2.2127523,2.2127523,2.6013336,2.2127523,2.62088,10.285665,2.4170003,2.2127523,6.5337524,2.2127523,2.689319,2.2127523,10.177575,2.6599758,10.966867,2.2127523,6.1482954,2.2127523,9.641854,2.9061399,3.2227092,4.002562,7.833168,5.8525305,8.145548,3.9816966,7.326608,2.2127523,6.64347,7.289993,2.2127523,2.2127523,3.9353123,2.2127523,5.299495,7.2265897,4.3391027,6.49674,8.241334,7.980726,6.327252,4.9314876,2.2127523,10.644967,9.76177,2.2127523,3.3384433,4.129413,6.9747305,2.2127523,2.2127523,2.2127523,2.5828953,8.241424,5.5267835,7.4366174,5.4704747,3.8116786,4.581026,8.511862,3.0481582,2.2127523,2.4061143,3.2071974,2.2127523,2.2127523,3.296817,2.2127523,2.213693,2.2127523,6.9577174,4.1831913,2.2127523,6.3980675,7.141008,10.414455,2.2127523,2.2127523,9.313342,5.5872564,9.571975,9.7431555,2.8932862,4.5104003,5.0951333,3.1539178,2.2310677,3.4099622,11.60101,3.5645816,9.655046,6.4688272,2.3444974,2.5527108,6.1371045,5.91873,6.4093766,2.313302,10.430955,12.213013,2.2127523,2.3025763,5.0336695,5.5959554,4.8795485,8.360959,6.3589363,7.110283,5.0878897,6.540392,7.9669447,4.9473124,6.661321,8.2704735,6.1360593,2.2796414,4.821514,5.5504613,7.3201666,2.2127523,5.1878767,6.4105697,7.2959924,5.626987,5.539949,2.2127523,7.5996814,3.2535865,2.2127523,2.2127523,2.2127523,2.2720692,2.2438846,7.9615355,7.274839,11.541625,2.2127523,2.2127523,5.9504766,2.2557468,6.192547,3.5653205,2.8391602,2.6045153,4.8709245,5.237257,2.5390978,2.3748815,2.2127523,2.2127523,2.2127523,9.401628,4.2388625,2.2382743,2.2127523,4.883731,8.909359,2.2616534,2.7874744,2.2127523,2.2127523,2.2332907,2.2127523,2.2127523,5.3582916,3.3357987,2.2127523,8.632566,3.8934999,13.027376,5.249641,5.381125,9.73273,3.182954,4.997349,2.2127523,3.83361,8.620752,3.0883086,6.2611184,4.5397425,3.2736635,6.4260855,2.981238,11.326412,7.5696383,6.6151547,6.8837767,7.305129,6.328216,6.9703536,10.365592,6.4884214,9.319785,8.520276,8.092901,6.2915864,4.695133,6.160933,3.178317,3.2814672,2.28533,2.8720753,4.9783955,2.2127523,5.546569,2.5716271,2.2127523,2.4794166,2.2245772,2.9610834,6.4996963,2.2237175,3.5703259,5.0329194,5.659591,5.1100287,4.563957,8.781542,7.0476446,2.2152824,5.5525293,2.9267607,9.779264,8.810554,2.2127523,6.2062845,7.9905634,6.958051,7.9938884,8.524564,4.1928487,5.9343357,5.5581894,4.1291924,9.384964,3.1215138,2.2127523,2.2127523,2.497381,4.3514543,6.48407,2.900354,2.2127523,7.557213,2.7042522,2.2297366,3.295571,2.2338305,8.770777,7.809974,8.669015,2.2127523,2.2127523,2.2291036,2.2345665,2.2171607,2.2127523,4.9888635,2.2445543,2.2127523,2.2127523,2.2127523,5.305683,5.336503,10.424841,6.6639533,3.7819726,5.451722,6.2592955,5.9948483,5.8807025,2.2127523,2.5945642,8.094027,8.581643,6.299962,3.9410756,2.7786608,4.4362216,4.4006143,5.5236454,2.2127523,6.984715,2.2127523,5.004911,9.46562,2.2127523,3.3627267,2.2127523,3.9236774,2.2127523,6.545859,6.252911,7.645757,5.1065836,2.2133276,2.2679036,2.218864,2.825361,6.0195017,2.2127523,2.2860265,6.1006,9.273515,3.2156072,12.389064,2.2127523,2.2127523,11.228585,3.380739,4.168422,5.874694,10.23552,2.2127523,9.482911,2.2127523,11.079695,4.7573733,12.211735,4.8598394,5.598128,8.083335,10.755536,11.931332,5.2944903,10.7715435,3.509156,3.0213838,5.8668647,2.2127523,2.236953,8.190705,4.0604315,3.4397132,2.222664,2.3561313,2.2127523,2.2127523,2.213576,3.9653854,2.7454383,2.2127523,7.766936,2.215545,2.5396566,2.2127523,2.3977356,2.2127523,2.4239984,2.2603135,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.7503953,2.2130556,3.8423803,7.1750684,8.360302,5.1198688,6.2675633,6.1385446,8.175753,8.657995,3.8008351,4.1437926,6.669144,8.447699,10.291038,10.340858,8.011685,2.2127523,6.8604894,6.529975,5.051777,11.285411,7.3255186,4.0711427,3.290126,6.3896728,5.402917,3.5608764,6.033691,6.242957,6.77376,4.1450725,2.3482976,5.109329,5.798218,2.43133,5.8573966,4.3886137,3.822713,5.4772797,2.9293153,5.6870813,2.2127523,2.2127523,2.2127523,5.809923,2.489204,11.549011,4.854915,7.23079,5.088825,7.4900002,5.3696976,6.8197746,2.2713823,2.872942,4.449519,3.6450343,10.341145,9.098588,2.5974114,10.725357,2.2127523,4.3207483,2.2127523,2.317586,2.2127523,5.0908375,6.0305114,7.261671,9.11271,2.242958,4.6807723,6.2924376,2.2127523,3.040942,11.573267,3.4029057,2.2127523,5.1246133,9.86255,2.2127523,4.995585,8.091773,6.2165565,7.2018375,4.423452,2.2127523,8.459872,8.2611,2.9489245,7.170355,6.1736083,8.196466,10.669048,7.671546,8.10175,7.135365,8.7192955,6.783167,5.8168716,13.405265,2.2127523,2.2974255,2.6362574,2.455642,11.147922,10.0315485,2.533196,2.5312295,2.5638459,3.6924384,2.2127523,6.332943,2.2127523,6.094979,2.7788439,5.7704067,5.7192287,4.4190745,3.9401276,3.0500896,11.202233,7.483724,3.5887341,8.507926,2.2127523,8.975569,12.580435,7.7291107,10.671888,5.953727,5.271971,10.443994,8.060107,5.9021754,8.34584,8.760814,6.4468904,2.2139618,3.476256,8.0451565,5.30239,5.8364444,2.2859154,2.232697,9.632583,5.085606,8.706097,9.075637,5.2646666,6.088876,6.7681866,2.2127523,2.2127523,3.8833992,3.2583864,5.28389,4.2075057,2.217792,3.4448905,2.7051673,4.2403493,6.7928424,5.508022,3.3270006,2.5438077,5.4998198,2.2127523,3.8051648,6.9039445,4.388841,4.1693473,6.838864,4.4139967,3.1445637,5.5812454,7.213676,7.1762137,8.698643,2.3479006,6.642397,5.061193,5.9829574,3.9851947,3.64288,2.6734197,4.6965275,4.8923187,8.921514,4.8733444,8.297718,5.9367075,5.0111246,4.4896398,3.5360863,9.476123,2.2127523,2.2127523,4.177734,2.3111205,2.2216108,2.2752666,4.2015615,5.204865,9.209322,4.2830443,2.2127523,2.2127523,5.653954,7.1571183,4.4268866,7.2817063,4.0876503,5.2438984,7.9547853,7.4244556,6.783672,3.6292753,6.85697,7.0116825,2.5210705,9.922569,3.1261475,4.4772286,6.9936895,2.2127523,7.574849,5.9325914,8.400987,4.4991016,7.20314,8.057974,4.5045147,9.448521,6.9374146,4.5537806,9.0369835,7.203639,2.2886274,3.4913151,11.813307,8.831292,12.801608,6.3423862,7.4618807,10.50907,2.852386,10.359614,8.7360735,2.2137215,2.307037,11.500205,2.4456172,2.2241209,6.078991,5.597684,6.1812205,5.533584,5.2168207,8.535569,7.390476,10.156706,3.6946375,6.3984995,6.8602047,5.7897797,5.5201144,7.678328,6.910348,3.2035823,2.234667,10.342492,11.703147,12.423801,13.167088,6.489696,8.570232,10.054512,5.8080325,9.789807,2.2127523,2.2127523,7.1800256,6.935731,5.106914,7.717997,11.854726,2.2127523,9.30346,3.0542338,3.55323,6.5528827,3.392209,2.2127523,3.597299,2.2384374,4.030091,3.5153747,2.2530408,2.2127523,2.2127523,2.508912,2.299438,5.841652,3.9437666,3.490488,6.349553,9.848068,4.703142,7.7395544,2.2199335,4.150441,5.677813,4.0846577,4.326303,2.2127523,2.2127523,2.2171113,8.184397,2.2140985,2.2127523,2.4022727,3.405201,2.2127523,3.6631446,5.737354,2.2248416,2.2127523,5.446304,5.6836505,6.619398,6.380376,7.217396,7.9397454,7.294902,2.3331401,3.326182,2.2127523,2.5882368,8.48816,2.3904247,2.2228873,2.6272738,8.397202,8.179945,8.65658,2.2468805,2.2127523,2.2127523,2.2127523,2.2127523,8.691333,2.3527567,7.6318936,2.4053247,8.405856,5.5049706,9.470919,2.3479705,2.8535082,2.2127523,6.871375,4.083112,4.6224484,4.183633,7.833522,2.9427764,5.9744244,6.0167775,11.400008,6.048853,2.2127523,3.5414622,9.423338,7.054381,6.2791057,2.2813058,2.2127523,7.39923,7.044386,4.743184,5.155546,4.4170322,4.5851083,2.4319494,3.619239,2.3255913,4.936595,8.706571,6.3181515,4.3144636,12.003447,5.3342676,9.005987,2.4354331,4.8554883,8.352822,3.9136622,4.0575514,8.684382,8.332489,7.2774835,8.245731,6.1269536,5.4376307,7.0737553,3.8049471,7.685914,7.6063023,7.347334,7.389993,5.482151,7.3950634,5.251979,5.894759,2.2127523,5.9492054,10.013881,7.2138524,6.7931457,6.6463003,5.974635,6.5121202,3.8706846,2.2127523,2.2127523,4.811848,8.309718,4.311887,3.5659518,8.662221,7.2404904,2.2776322,5.2034397,5.143811,2.2127523,8.452308,4.736793,7.9618406,5.2290354,6.3096614,2.2127523,10.774728,2.9676547,2.513264,2.2127523,2.2127523,13.5315,3.2932096,5.2238765,2.2127523,2.2127523,7.628803,4.113343,7.995079,2.2127523,2.2127523,8.091765,7.6552105,4.0574408,2.2127523,2.213018,7.9783053,11.212137,10.820259,7.962001,8.062563,8.784493,8.250397,7.424845,4.578046,10.171457,8.89398,11.7681875,12.179636,2.2263625,6.6122074,12.189709,8.172202,6.6446986,6.732746,12.291605,9.428475,3.1029465,8.760053,5.7377415,3.8480484,2.2266955,9.854763,9.622147,10.754367,4.456799,8.556386,7.226163,2.2127523,10.099935,2.3307617,10.068021,7.8496003,2.3699756,6.497338,10.394835,2.9617546,11.132585,8.47996,2.2227871,8.72141,2.2127523,4.5360613,6.611973,2.4533436,7.053588,2.6961997,10.632776,6.8321214,2.2127523,3.2728188,2.7879868,7.4067063,3.4094565,7.902303,4.9075036,8.835442,6.2845473,11.497557,5.6788726,6.7152,2.2127523,8.392893,6.4389925,8.2396145,8.019216,8.40044,4.0125647,4.861871,2.2127523,8.142175,2.8508472,2.2127523,7.037079,7.7321663,2.2127523,8.225986,2.2127523,10.478938,4.610413,6.3565307,9.375453,2.5993733,2.2127523,6.4818044,2.2127523,2.4597979,2.4398053,2.2127523,7.5618453,5.159902,4.8571434,2.2127523,6.813529,2.3739197,2.3713665,2.2127523,2.220022,4.4325404,2.2127523,2.2297304,2.2127523,2.2127523,2.2127523,5.648225,9.543125,2.2127523,2.2330112,2.2127523,2.2127523,2.2127523,3.7433295,2.2127523,2.4843392,2.2127523,2.2127523,2.2127523,2.216697,2.2630835,2.2127523,4.1736107,2.215137,2.2127523,3.7432003,3.7039504,4.453673,2.6045766,4.2422466,2.2127523,2.2127523,2.327369,6.489352,2.2127523,2.391352,5.887268,2.915901,2.5740528,4.7557526,3.4962754,5.9722366,2.2127523,6.8361707,4.61193,2.2127523,6.615744,10.4605255,4.6020927,2.2127523,12.184296,5.1775417,5.4557633,2.2127523,2.5562167,4.5103784,2.2218893,5.0003567,4.211236,2.2127523,2.7313414,5.0047126,4.6988277,6.5488963,3.7185612,2.728632,5.6316586,2.517026,5.8580713,5.205404,6.017696,3.320769,2.2127523,2.2420387,4.5312433,2.2439847,4.8562217,4.543997,5.645119,7.2957425,10.115269,2.8285778,5.322322,3.64178,3.771687,2.7607038,2.2139378,2.3316698,8.7160225,5.438208,5.2558966,3.505594,4.1438475,5.0314054,8.72901,4.1815305,10.667329,5.686122,5.840988,8.048924,2.219988,2.262649,3.0297534,7.092846,2.9211195,2.2127523,2.2456326,5.991104,6.69253,8.762183,2.2141883,3.5913649,5.7429147,9.475114,2.2422106,10.0665455,2.2127523,7.089321,7.268993,5.8183117,5.854023,2.2127523,2.2536683,2.2127523,2.281047,5.538588,7.050948,6.6350994,5.6898994,7.8538804,10.9620075,4.452119,4.730597,9.32928,5.2066226,5.8689337,2.4304144,4.5041175,7.334016,5.226596,7.213488,7.29167,3.42451,5.1291614,4.135016,7.8148065,3.670271,3.0987582,2.5172756,2.2127523,5.0842166,2.2127523,2.2127523,2.21429,2.2127523,8.406366,2.2127523,2.2127523,2.3211,5.23593,2.7000246,7.4909697,2.2127523,7.330057,2.2127523,2.5427797,2.2127523,2.2127523,2.2127523,2.2127523,5.817327,2.2127523,2.2127523,2.2127523,2.2127523,2.5609035,2.2169847,2.2127523,2.2380788,2.2127523,2.2127523,5.340556,5.988292,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.214959,2.2127523,2.2127523,2.3371928,2.2127523,2.2127523,2.2127523,5.585336,2.2127523,2.7994204,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2179348,2.2127523,2.2127523,2.2127523,2.2127523,2.284177,2.2167847,2.7015882,2.3716686,2.2127523,2.7869763,2.2219129,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.561116,2.2127523,7.0113163,2.2318234,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.213139,2.2127523,2.2127523,2.2127523,2.3210368,2.2127523,2.2127523,3.671345,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.3907757,2.2127523,5.4199376,2.2127523,2.2160287,2.2127523,2.2753701,2.2301672,2.2127523,2.2127523,2.2127523,2.2127523,2.225039,2.2139044,2.2127523,2.2127523,7.2322974,2.2166228,2.2127523,2.2127523,2.3509982,2.3243527,2.2469628,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.0068152,5.121541,2.2127523,8.6253805,2.2127523,4.213521,4.1191564,2.350947,2.2168453,2.2127523,2.2127523,2.2127523,6.6519294,2.2127523,9.6282015,2.2127523,2.2127523,2.2127523,3.1295757,2.2127523,2.2127523,2.2127523,5.3657746,2.2543123,6.371162,10.396172,4.1307077,6.545576,2.2127523,2.7371385,9.650423,4.7627378,3.5951943,8.864818,5.107223,7.5982127,4.9663916,2.3810492,2.4567838,8.453488,3.3146045,6.913584,5.308525,3.6505184,5.6154323,4.024014,9.878948,5.7862372,2.2127523,5.720163,5.4507523,5.6503673,2.237818,3.1042383,7.8571115,7.1968017,5.478858,9.309997,3.258154,4.308672,7.7490172,8.254223,4.809537,6.445054,7.6594396,2.2441478,2.2127523,6.2428536,3.6508167,2.2127523,6.5311775,2.2243094,6.938889,4.598249,4.0007324,5.5326877,2.2134197,3.6685739,2.6303797,2.2127523,6.3227515,6.7842393,7.021211,2.3380773,2.2127523,4.5970335,2.2127523,2.7803545,2.3344967,2.886159,2.666601,2.2127523,2.2127523,2.4354389,10.1138315,5.7726192,3.0720425,6.4179325,7.964309,7.282155,10.128799,3.4562829,2.2299368,2.2127523,4.344641,2.4180162,8.134295,3.9126084,4.4221,2.2127523,10.422641,4.475639,2.2127523,3.0964983,4.143501,4.5236764,6.412693,6.52745,5.1024594,5.8000855,7.3572397,4.4914165,2.5156655,10.743049,8.245876,3.395985,9.69934,5.551673,4.0058904,4.6745896,8.452469,2.2127523,2.213062,3.4625971,5.5435925,9.133206,8.796503,3.2711494,10.087512,4.3230567,8.199148,7.4525137,12.174089,9.948807,7.3761306,10.157048,6.6844163,5.660274,11.14184,3.4262002,2.2127523,2.2127523,8.847999,5.885683,2.7107365,7.0915647,4.4030232,2.6923404,2.3653855,2.2241368,2.9378886,2.2127523,5.7564526,4.3697557,7.478683,8.166348,2.2127523,6.919863,7.172245,6.5339193,2.276977,2.2127523,3.824511,5.1862,9.43353,9.603294,11.46155,3.6204689,4.397632,9.378018,6.1709123,2.2206867,7.184707,6.414759,6.5780396,9.761468,2.2127523,2.2127523,5.829442,6.2356696,2.2127523,8.336322,7.2476354,2.2474442,3.3472695,2.2127523,2.2127523,2.2127523,2.2677116,2.3766704,2.2127523,2.2127523,2.2127523,2.2127523,5.312864,7.9521914,2.2205641,5.9432893,6.6551948,4.135309,4.5341744,3.8147511,7.7986207,6.7064753,9.101387,6.6670094,5.968785,6.1272454,2.2141526,5.286341,8.0039625,2.2127523,7.171853,2.4370148,2.7742982,3.031226,5.9079943,7.2696137,8.896234,2.2365184,3.504142,2.2127523,2.2127523,4.5706253,7.844119,8.310137,7.5643864,7.2894897,6.8738728,7.2916164,4.0198774,2.2127523,2.9270775,2.2127523,5.207052,2.586257,3.2803864,8.191951,6.8943486,10.913517,2.2127523,2.2157164,2.8313956,8.396624,4.297926,9.664461,9.415245,10.968096,7.6559,2.242198,2.2127523,2.2127523,2.2127523,2.9186227,6.8572097,8.937564,8.219423,5.8497114,2.2127523,3.5627093,4.1843657,4.359196,10.558299,6.6347766,2.2127523,5.722644,2.3897815,8.549679,8.570461,7.6424046,2.2127523,7.0005274,4.3963614,2.2127523,6.6846943,4.6781526,2.2130852,7.89282,10.564055,9.519916,2.2127523,3.2625494,2.2566385,3.5251346,3.4757037,2.8012872,6.1507473,7.2541714,7.3491917,6.0758185,2.236065,2.2127523,2.2127523,2.2127523,2.2127523,6.431959,6.7242823,4.3104634,12.424378,10.262676,7.2571607,2.2158859,6.043458,2.2127523,3.9749246,2.2127523,5.1130133,3.003094,3.0447474,9.737826,5.491549,2.2127523,2.213341,4.5075617,2.2127523,2.2516031,2.2254455,2.2216072,2.2127523,5.6177015,2.2127523,2.2127523,5.5632663,2.21818,2.2127523,2.2713382,2.2127523,5.9426823,3.2269742,2.6377964,6.588791,4.802143,8.628591,2.2127523,2.2127523,2.2127523,4.0369444,9.779292,4.7541733,6.386538,2.2127523,7.681543,4.9667897,7.3961473,3.892292,2.219891,7.3249946,2.4264104,6.437713,2.2238333,9.25099,6.982288,7.8049984,7.698802,4.520324,2.2127523,2.2197616,2.2127523,3.1526945,6.165672,2.26102,3.665582,11.574753,2.2127523,6.959431,6.7215276,7.775992,2.2127523,2.2142544,7.1726985,7.3111835,6.506673,7.796865,2.2127523,2.2251854,2.218023,2.2127523,10.680339,2.646686,2.2127523,5.8692236,5.905317,6.5844145,2.2127523,7.2168403,5.796559,2.2127523,2.2127523,2.2329385,5.353948,2.2151957,10.657186,4.020611,8.988721,5.726008,12.35999,6.1425304,2.2127523,2.2372985,2.3309019,2.2127523,9.795167,2.2127523,6.971501,3.0557241,2.2127523,2.6156824,3.4520445,4.3187904,4.476518,2.2127523,2.7942407,2.2127523,3.9695542,4.534881,5.3760667,3.4998355,2.2127523,2.2127523,12.242275,2.2127523,5.9979534,2.2127523,2.2127523,4.6789947,4.7037797,8.8308735,2.212904,5.66121,4.7450986,2.2127523,4.4487596,3.878377,7.4890323,8.415781,6.4258804,2.2127523,4.782219,4.700405,3.2056582,8.390501,2.749813,2.2127523,9.121606,2.2127523,4.159792,4.549594,9.952303,6.478416,2.238324,3.734466,3.8974988,3.9530387,3.1840222,2.2167962,2.2127523,10.638802,2.2127523,2.7866802,4.036695,9.69115,3.3238974,10.128697,2.2165885,3.3849008,4.3797174,2.2127523,7.341138,2.2127523,2.2127523,5.979782,2.2211435,5.065124,7.211534,3.1411178,4.1306376,2.2127523,2.2127523,2.2127523,2.213263,5.2864604,3.8853629,13.5319605,2.2127523,2.2290087,2.8068085,2.2127523,3.9979217,10.317407,6.1009045,4.842725,7.3277497,4.5371084,2.3699062,10.894958,5.278364,8.425531,2.2246823,2.2185476,2.256346,6.9637856,5.5840425,10.441379,6.5180182,2.2127523,6.74706,7.3996596,5.8431683,9.39395,2.2127523,2.2127523,2.2127523,2.8120797,2.2135518,2.3057017,10.73046,10.069882,5.7356987,11.961108,11.79055,11.051772,11.99329,2.2127523,3.950478,2.2127523,2.8397255,4.903791,2.2254484,4.5086474,2.4306188,3.3289897,2.2127523,2.619083,9.698655,7.5242476,3.291057,9.924875,2.2127523,2.2415872,2.2127523,9.282994,7.826222,2.2127523,2.2127523,4.864427,3.9194071,11.688694,8.417194,8.598144,2.2127523,9.030277,2.2127523,2.2186146,2.7647207,8.736911,2.2127523,3.46407,7.5339704,2.2127523,2.2127523,5.2205815,5.2206244,2.5008035,10.282566,2.693463,2.2234526,4.506208,6.5240283,2.2267532,5.9364057,2.2127523,6.3951855,2.2141147,5.248855,2.2127523,8.351965,7.808103,11.051538,6.7637563,6.9472804,5.654205,8.56688,5.524728,2.4874618,2.2139666,2.2127523,2.946608,8.259877,2.903014,2.2234707,2.2742317,3.6695514,5.898665,2.3311546,2.2127523,2.67466,2.377212,2.2127523,2.2127523,2.2127523,7.644281,2.2127523,4.0482593,2.2127523,3.667237,2.2870157,11.015642,2.2127523,5.424848,2.2127523,2.2490826,2.213472,2.2127523,2.219965,5.771024,2.2127523,2.2235446,3.5808802,8.013142,2.2127523,2.220363,2.4030824,5.810747,2.2167537,2.215079,2.2133877,2.2191212,6.395264,2.4292543,2.4122393,2.2127523,2.2127523,5.264881,2.2761686,3.8441947,7.3712215,2.2132268,3.912249,7.4896297,4.2992277,4.767237,3.928651,2.3477006,2.2127523,10.291973,2.6093032,2.2645602,3.9815824,3.285982,5.219734,5.962166,4.6353555,8.59187,6.535348,2.2178924,4.804323,2.2174134,2.2127523,7.9476466,8.951901,2.2127523,2.2127523,2.3802319,2.2127523,7.9617834,6.406435,2.2127523,4.5884113,2.233929,2.2127523,2.5888858,11.215054,9.814682,2.2127523,5.2404437,5.8377514,5.925758,2.2127523,2.2127523,2.2127523,2.2127523,3.347673,2.2127523,2.2127523,7.0271845,2.2127523,2.2427754,3.915654,2.554577,2.2127523,4.174884,4.6930847,2.2127523,2.226575,2.848039,2.2127523,4.2499638,7.974088,10.563906,3.2301378,2.2131097,3.9544106,2.2127523,2.2127523,5.3441305,7.436535,11.167195,4.916589,3.538891,2.3631628,2.2127523,8.281817,4.3127856,8.090105,2.2127523,5.885537,8.178456,2.2127523,7.38871,2.2127523,2.2528546,6.5320306,6.866472,2.2160332,10.308295,8.076261,4.495095,5.304873,5.904737,2.2127523,2.2127523,3.3217196,5.925728,4.752185,6.112586,2.2127523,4.4954605,2.2127523,3.0758333,2.2127523,2.2497456,6.9265137,2.2127523,6.6459365,2.366932,8.072403,2.21911,3.433698,5.5586543,2.2647588,2.2577083,2.2521133,2.2714295,2.2127523,3.2612228,2.2127523,2.2127523,7.8370676,10.086035,4.0606475,4.132696,2.2127523,5.401558,11.36496,2.367288,7.061675,2.2127523,8.545275,5.8603716,2.2127523,6.709668,3.6540987,6.663285,5.2160163,5.7480636,2.2188702,2.3532574,4.9976687,5.1412215,4.2553506,8.014724,8.905081,4.6490107,6.7507434,2.2471828,2.2127523,4.702336,2.2127523,2.2127523,6.1641073,2.2586231,2.2152646,2.2127523,2.2127523,2.2127523,2.2127523,2.246466,7.388227,2.4362464,6.3006177,10.652003,5.0891485,9.214461,2.2459574,2.7600644,2.2127523,3.2922888,8.606205,3.3913584,2.2155974,2.2127523,8.29868,7.178465,4.664882,2.2127523,4.7801266,4.179126,2.485963,2.2127523,2.5305095,2.2127523,7.389833,6.711458,5.155832,6.8300104,2.2127523,2.2534218,2.2127523,2.6479146,8.760168,2.834011,3.7798116,4.8259773,5.688533,2.2127523,2.2904804,2.2127523,2.2127523,2.2127523,2.2127523,10.771344,2.3914433,7.1645937,5.2244925,8.14337,3.8008368,4.8565116,4.958066,4.0600014,5.0952315,2.5328557,2.2136567,7.435965,2.8559442,4.803599,2.5733566,7.802435,2.7343414,3.1450083,2.2127523,3.2371988,5.8720036,2.3214545,2.216236,2.4158785,2.3137975,8.534022,8.926956,3.8525906,7.0965605,4.6014614,4.637413,2.216819,2.2127523,2.40401,2.2127523,2.4038246,2.2127523,2.5979106,5.9369555,4.2007074,2.5987484,2.2127523,3.1842518,2.678842,2.2127523,2.2127523,2.2274616,2.2127523,2.2132154,2.3649118,2.9177363,3.533679,7.2095222,2.807004,3.4352381,2.97423,3.0169683,7.3366055,2.390935,2.6682699,2.2127523,2.2127523,3.5618925,2.62881,2.4635553,2.2127523,2.5822897,4.0706635,4.564395,9.176966,4.4115257,6.248855,2.2136512,2.2127523,2.6343944,2.2127523,2.6056976,2.2127523,8.565611,2.2127523,2.3914428,2.2127523,3.5153534,2.2274241,2.2127523,7.268254,6.2432733,2.2148397,4.3992395,5.5180616,2.2169816,6.002843,2.2127523,9.145656,2.2158813,4.029295,2.2127523,2.2135372,2.2127523,8.853206,2.2127523,8.510965,2.2127523,6.093186,7.1248064,2.2234461,2.2127523,2.2127523,7.501706,5.1014814,4.3317432,2.3268247,3.397527,2.2127523,5.338523,3.3174849,2.2127523,10.031229,3.555707,4.4540515,8.250278,2.8343058,6.532502,6.6942844,7.135155,2.2127523,4.413434,8.864837,8.50709,11.815407,4.4312162,11.174849,7.631984,3.4313865,2.4080224,2.2127523,7.1420965,3.1072338,2.2127523,4.060921,9.320433,6.8769855,4.090013,2.2127523,8.445294,9.894157,3.1565244,11.1352005,2.2127523,7.4877405,5.803397,4.722479,3.4885364,3.2933884,7.266253,8.639321,2.2127523,8.743646,4.4255185,5.2836,2.2343423,7.4807386,2.2127523,9.306664,5.831154,4.476362,4.5079365,4.859856,8.292867,6.638309,2.8556404,6.033667,2.2127523,3.438705,2.2127523,2.2127523,2.2127523,2.2127523,3.0812368,2.3573678,2.226256,2.2127523,2.2127523,2.2296193,2.2127523,8.543925,6.005218,2.9517636,2.2127523,2.6489482,4.491586,5.5924926,2.2146456,5.9566755,9.170798,3.434785,2.46994,2.2127523,2.2127523,3.9283695,8.086826,8.857068,3.168626,9.140342,9.107207,5.822791,4.8111873,4.8486753,2.2127523,2.3243763,2.455311,2.6507263,8.115969,4.046864,7.5476127,3.320688,2.2127523,2.2408707,2.966444,2.2356586,2.2127523,2.2127523,2.2127523,2.2127523,9.529805,7.746517,2.2509506,5.2072916,2.6841507,8.67216,5.549482,5.8687096,2.2127523,7.7345414,2.2127523,2.2127523,2.3062115,2.395185,7.8323607,4.0462165,2.2202551,8.254673,3.2308648,10.271784,11.22991,2.6096525,10.154514,2.2127523,9.111883,7.463354,7.5111084,8.611003,3.7542117,5.561072,2.2127523,3.9235961,4.903876,3.306194,2.3127682,8.381232,4.1701384,4.6034684,2.6714578,8.541236,6.536421,3.4042487,6.198155,2.2127523,8.396783,2.2127523,5.3154206,7.386353,2.2127523,9.548847,3.0828776,2.2127523,3.902993,6.946325,3.0374243,8.5277815,2.2127523,5.858359,9.603876,2.2515476,3.053167,2.2127523,4.634913,5.263324,6.587186,7.0676565,2.2127523,2.560674,2.2167263,6.442557,4.443953,2.481102,2.2127523,2.3915272,7.3505363,2.7607753,4.756143,2.22132,2.2711806,6.5847187,2.2233338,2.2127523,4.3597684,2.2565567,2.414691,2.2127523,3.270173,4.1481,2.2406294,5.2118173,6.6701818,3.0406759,2.2327583,2.2127523,2.6847003,2.2127523,4.5985384,2.213285,4.1928325,7.2216516,2.2127523,2.3020191,2.2929835,2.2276669,2.5542104,2.4078076,2.2127523,2.5264537,6.096864,3.696489,2.2127523,2.2363367,2.3963928,3.2759182,2.4456306,2.2131948,2.2127523,2.6630538,7.7759986,2.4873703,6.4092364,8.357681,2.4446552,6.9711533,2.2127523,2.2127523,9.101765,2.475791,4.03947,2.3530052,2.2335222,4.55921,2.2127523,2.3725624,2.2127523,2.3421865,2.2225611,2.413115,2.2162511,3.6383994,8.159887,10.832227,6.7028337,7.528062,8.862963,2.2127523,3.510333,4.18398,8.804888,10.152277,3.0609093,2.5646112,2.5553067,2.2127523,4.1822577,4.072828,2.2490873,5.5401607,2.2127523,3.7816544,7.013663,2.4979124,2.2127523,4.780541,5.021575,2.8016326,3.438398,4.0801806,2.2127523,4.582635,5.611631,2.2347488,5.0031724,2.2127523,9.514167,5.3675,3.8283143,4.775579,6.3306336,2.2127523,5.2354817,2.7113683,4.7582145,2.2127523,3.5938134,5.9945827,8.576331,8.029864,5.884466,6.504487,2.2266002,4.830819,2.2127523,5.4209557,2.2127523,2.2228177,2.2153518,2.2127523,2.2127523,3.3296092,5.951918,4.740852,2.3732276,2.2127523,4.881857,12.722121,8.141479,9.209224,8.201654,7.3056674,4.8877,4.087539,2.4240773,2.2127523,2.2205806,2.252812,8.449132,8.781623,9.040634,10.090529,3.1016588,9.163106,8.612293,2.6790426,2.2127523,5.2114296,5.6892877,9.705682,4.0066724,2.2127523,6.680687,10.503513,4.551619,7.5163755,10.013791,3.3475556,4.6737475,2.8052323,11.41322,7.5168576,9.25066,10.499103,4.95652,4.962056,5.51806,7.179808,8.868451,4.755196,7.1145,6.9429336,2.2723458,6.895571,8.322579,2.2127523,4.7461295,8.204962,7.5673285,3.0249841,5.539029,2.2127523,2.2127523,9.861674,4.868553,5.5567503,2.238253,2.2127523,5.680104,2.6748955,7.4707756,3.8350174,2.2127523,5.6794686,11.120429,11.679048,8.508183,8.641581,10.311067,4.6880326,9.03883,10.383927,7.1533165,2.2127523,8.700073,7.684057,3.0599515,4.6698923,10.140928,9.225557,3.2113736,7.5245805,4.544806,10.802551,2.2795622,5.6229563,2.3747478,5.0071907,2.245212,6.4622793,4.7618027,3.880668,10.194186,4.7533193,3.5130792,3.2988675,3.458443,2.7814863,2.4243727,8.893331,2.8921585,7.286004,4.842378,8.94805,7.1391854,4.355197,3.1405678,7.235167,7.24022,3.9529412,3.8768034,2.2345505,7.725102,2.5179033,9.534649,6.147014,4.6946015,4.881134,3.8356235,5.1082234,7.4435706,9.714786,5.603798,5.43383,6.7397056,5.381098,6.9813623,2.2244422,7.203172,11.895746,9.0241585,6.5484643,4.3325396,11.219829,6.8552704,13.015485,6.9659734,10.393404,2.2127523,3.8015687,14.101505,6.106242,8.550195,4.5734596,3.3431048,5.440177,8.861925,6.689555,3.2420552,4.462791,11.780544,2.2192683,6.4638243,9.203051,5.431866,6.868979,8.571606,3.1685543,6.0074587,7.418301,3.9978468,9.811554,10.362458,4.149461,3.0573494,2.9138784,4.476312,4.1795015,9.351567,3.1754148,4.423214,2.2127523,9.7199335,4.4307313,3.277132,6.895055,2.236666,9.444905,2.2361503,9.879962,7.9196405,5.3627687,9.120904,6.658364,2.2127523,7.6602592,7.390002,5.3940873,7.768055,4.7402844,2.2127523,7.683857,4.0346007,11.28994,6.520764,4.9470363,7.752779,9.454787,2.2127523,6.770616,2.5407925,5.403301,8.604454,3.3195398,4.844497,6.704639,5.02593,6.7220507,8.602375,3.8469605,4.722263,10.953818,2.2127523,8.46175,2.2127523,4.221327,2.7469726,2.2127523,2.2127523,2.2127523,2.516256,4.092523,2.2127523,3.2910843,2.284995,3.051268,2.6359572,9.024645,2.2127523,4.5693636,2.5272396,2.2127523,2.2587724,2.2127523,2.2127523,2.372863,3.0241854,2.2158341,6.001844,11.349278,2.2127523,5.68073,7.063973,7.4844418,5.892636,6.2955055,5.0779467,2.3375933,5.5516133,7.6128097,5.0526342,4.5386143,4.745092,5.6753364,11.5809355,8.638747,4.7718983,9.900075,6.1726584,2.9475174,2.2127523,8.007754,5.9738545,10.956879,4.414098,9.2382145,6.7214046,4.4396186,3.6836107,3.454821,6.656837,4.5762606,2.2127523,9.609262,11.85,6.7520766,4.0583553,7.542066,5.0854025,5.1329923,2.2127523,6.6450458,2.7590125,6.2061105,2.2426224,2.9655569,2.777493,3.4137845,2.2127523,8.855486,5.774857,2.2143774,2.21349,2.2127523,6.0101824,6.5724845,6.2549586,4.083842,3.3321886,9.328439,4.149922,2.2127523,2.2127523,2.219908,7.758887,3.699701,5.6584315,2.310676,2.436241,2.4312754,2.5989747,8.138789,14.204035,2.3012304,2.6228247,2.2127523,7.1783366,6.5288186,12.576412,9.476996,3.6578624,7.2858343,9.684102,5.9769015,8.899272,3.6075666,2.2127523,7.6699924,2.2127523,2.2127523,2.2127523,2.2203357,2.5225177,2.2127523,2.218828,2.2127523,2.2127523,2.2127523,6.9772434,6.768768,2.2127523,2.2127523,2.2130299,2.2127523,2.2629306,2.2127523,3.7938416,3.7157996,3.3617268,2.2324343,4.9299583,2.2764642,2.2874537,3.2980149,6.721641,4.457354,6.588858,6.7933245,2.4905894,4.19632,4.416744,2.5154138,4.2178407,7.0211215,6.5972986,3.2754798,4.1951203,2.371708,2.2127523,5.255779,2.4041204,5.5008597,5.319018,2.2127523,6.6411514,5.4139943,4.596027,6.665404,2.7146173,3.053821,5.6856365,3.4264112,3.8628166,5.361793,7.704754,6.997121,4.4564557,7.204607,2.61452,6.740864,9.313135,3.3650367,6.117203,6.2050595,8.226878,2.9837444,5.3965793,6.337217,6.391936,4.3340483,6.86586,5.9918633,4.4960346,2.2127523,5.3210397,5.05217,2.8738315,7.0195513,4.180362,6.183487,2.2825625,3.8431957,7.860787,2.2127523,10.4475155,2.2127523,5.070618,5.063698,7.3195415,8.769578,2.2127523,7.8906264,2.829257,2.2188299,2.2127523,2.2127523,9.753628,11.131423,2.2127523,6.511715,2.2127523,2.2127523,9.437286,5.5376053,8.211414,4.492528,2.2127523,2.2127523,2.2127523,10.033544,7.5321426,9.3477335,8.562682,2.2362552,5.3984075,2.7692144,2.2127523,3.499384,2.599787,2.4948535,8.988351,6.473227,5.1051617,2.21707,7.8821507,6.36953,2.6474395,6.629384,2.2127523,2.8381677,3.9967997,4.270684,7.579426,2.2127523,9.951221,9.150321,12.134779,6.0434823,7.651663,3.788567,5.8031483,9.795469,9.706441,7.6684685,6.7658587,6.2655263,7.638524,2.4311764,4.260197,5.8542895,8.117448,7.4567523,2.2127523,4.225842,4.1189966,4.7887588,4.7935705,6.3287168,2.2136445,2.2147284,8.284013,4.1379523,7.742436,2.2127523,3.6472547,8.771292,6.9642415,8.297438,5.1449895,9.086206,2.9595995,4.7373905,2.3372753,2.2460647,2.2289972,4.844519,4.851835,7.893531,8.613063,11.26797,10.123809,10.574385,10.539135,10.421767,9.393585,12.095728,6.766216,7.7349286,7.7075806,4.855139,2.2569268,6.148241,9.67668,2.2127523,6.2623463,7.322485,6.367776,5.9004273,2.9919438,6.9073396,2.2127523,2.3095732,2.2750435,2.2127523,2.5985413,2.2713718,2.2127523,2.2127523,2.2127523,2.2127523,2.9768164,2.2186038,2.2127523,2.2127523,2.4603112,2.2127523,6.0488067,5.101116,2.2127523,2.2127523,2.291276,2.2127523,2.8685267,3.04851,6.2753263,6.5780373,6.732138,2.5947685,4.8083415,6.9509406,5.480276,4.0838804,2.2752826,2.2477052,2.2127523,3.4631827,2.3414335,2.2127523,9.78264,5.2101645,4.471242,8.79243,8.762782,8.166081,8.436765,5.469615,8.187021,6.0431986,2.4133236,11.147263,3.0449007,2.2127523,2.2127523,7.261402,2.2134686,5.7509365,2.575103,2.2134533,3.823567,4.0449247,2.247974,3.7744126,7.1014733,6.7904935,9.16339,2.4274306,6.819117,7.745474,6.4532337,6.003502,7.514633,7.3516912,2.2127523,7.508166,4.9202776,5.9534464,8.020918,2.8046591,6.5999403,7.9738135,8.195883,10.070211,2.2127523,2.6180882,2.5057528,6.9520903,9.738927,9.972577,7.628866,5.1870522,8.924024,2.281604,2.2127523,2.2353666,2.2127523,2.2127523,8.124548,4.8471003,2.2127523,3.2100773,7.7712135,2.690673,2.853708,3.555261,6.3459353,2.6051214,2.2423143,7.4097834,4.8584557,5.4009585,2.8566644,5.809083,2.2262328,4.339191,2.2127523,2.2127523,4.921786,3.4525864,3.5214508,2.2127523,6.1194444,4.2022495,9.87433,5.9933515,6.604426,2.2127523,10.4003935,2.2127523,2.3876212,6.7244396,2.2127523,6.327994,3.884022,3.3044326,2.2127523,4.953513,3.9696212,2.2127523,3.2439349,2.2127523,7.96812,6.3017645,2.2127523,2.2127523,7.04563,7.7330637,2.2127523,2.216948,4.060762,2.2127523,2.2132053,2.2127523,2.2127523,6.8620143,6.0392737,3.7334018,2.2127523,3.9134777,12.45609,2.273596,9.628967,9.015055,9.38416,5.305197,9.248864,9.072403,6.123289,2.2127523,3.0741932,3.0271912,7.4858956,9.861405,9.234847,4.826028,6.3717217,2.2127523,3.2200716,4.923583,6.3507223,7.928423,3.0371847,5.5860276,2.4228184,8.536237,7.425091,5.167546,3.5937867,8.40677,2.2603545,7.574446,8.730986,8.715815,2.213075,2.2127523,2.2127523,2.2127523,8.687599,2.2564893,2.2127523,4.479364,8.053801,5.744724,6.029613,7.6078367,2.7267346,6.4800215,4.703946,7.0117435,4.8533964,2.2465124,2.2127523,2.2127523,3.9823997,2.2127523,2.2127523,2.2127523,5.2485695,6.8240204,7.328399,9.986951,8.964725,5.269685,4.8674197,4.1191287,8.290714,10.075793,6.645092,2.2127523,3.3480258,6.3022165,6.8923426,2.3432095,6.922899,8.0765915,6.401477,8.090837,9.1966305,2.8736455,2.2127523,2.2159426,10.478474,2.2127523,5.9043436,8.958917,2.2127523,2.6259232,2.2127523,2.2159913,2.2853813,2.2127523,10.662773,2.2895157,2.4809864,7.6729264,6.933713,2.2127523,3.279464,5.77583,9.369399,2.371264,10.043435,2.2127523,2.2374835,3.1164546,6.1830106,5.40806,7.939548,7.3528824,9.0827465,6.13886,3.8172047,4.9758277,8.808248,5.414163,8.380114,10.279726,6.0099277,7.392387,2.2132866,8.428338,6.5365925,3.51571,2.494992,2.2127523,6.746177,5.4216666,2.2127523,2.5846808,2.2127523,4.2468095,2.2127523,3.2005146,3.5507822,2.2127523,4.5237308,2.2950892,2.2127523,2.4731116,3.7071178,2.246533,2.3041687,2.2127523,2.2127523,2.2127523,6.413133,3.140763,4.1258197,6.7090416,9.856959,8.357393,10.204485,5.584968,8.550752,4.3348913,3.9070385,10.592037,4.5048094,6.4572377,6.5566416,7.216984,7.7063904,5.759138,7.022758,6.729672,9.071827,8.746238,6.214259,3.9753833,4.7419653,5.6950383,3.182849,6.4044847,6.26784,2.2127523,7.273386,2.2127523,2.2127523,9.840605,11.846445,2.2185996,7.720469,2.2176776,2.2127523,2.4376945,2.2130587,3.0285711,5.278644,3.329515,2.2127523,11.533033,7.7216296,7.041657,5.239551,8.710461,5.2967124,3.920713,2.213187,7.867309,10.380629,2.680774,4.516015,7.2880483,11.793493,2.8681133,3.5196216,3.6711113,8.01318,5.253491,2.2127523,3.3873363,2.961647,8.069626,7.293309,2.2127523,4.4611435,9.873321,6.092786,2.914267,2.2127523,9.408405,5.983575,8.218056,9.7510805,6.902842,5.9883432,7.5660605,5.773132,9.451668,7.958555,3.5660655,7.0402403,5.7922454,7.4397507,8.926221,6.934799,5.760752,8.504179,4.272773,3.110323,8.994636,8.492754,8.952719,6.095758,10.086643,9.012838,7.309169,4.3072896,6.088287,10.522431,4.4594107,6.126945,5.421579,4.777214,5.783562,7.009235,9.232132,4.0929837,5.89565,6.311115,2.4526436,8.999496,7.2586465,10.34626,8.374481,2.4523811,2.5496526,5.1294484,3.3152192,7.0977106,16.22318,5.706978,3.015334,9.470628,9.754859,6.1589546,7.08343,6.313686,5.9787726,4.3859115,15.537024,9.353448,9.765573,8.024893,5.1631126,7.738129,9.840552,7.537513,2.2127523,11.808163,2.3254354,2.2127523,13.174055,3.0255373,2.2127523,6.5383782,13.224747,10.877055,9.976916,8.429537,2.2127523,9.6676,11.825333,11.692972,2.2127523,6.800883,2.2127523,2.2127523,9.743862,2.2127523,2.6299105,2.2127523,3.1451664,2.2127523,3.8219907,6.681598,11.163637,2.3399317,5.086723,2.2127523,3.9058504,7.982415,5.5046864,4.14805,2.374567,2.236492,7.961182,2.2127523,2.2127523,7.9584427,8.763422,10.130143,8.03914,5.494336,2.2127523,3.0090325,6.084086,2.2206511,2.2127523,8.650286,7.0626063,7.600906,2.9523253,2.2127523,5.676483,7.900079,8.832944,6.2080946,8.468181,7.954568,5.167469,3.6583078,2.2826767,7.4769382,6.406743,8.759637,2.2127523,2.3233037,3.1013143,3.136304,3.449754,2.2127523,2.5721686,3.0445619,2.2127523,10.010493,5.9382854,4.826999,7.430803,2.2127523,2.2127523,8.437414,9.367504,7.2501187,2.2127523,8.720959,8.676788,3.6020975,5.923941,2.2127523,6.4902124,3.974573,5.2299714,9.8446665,7.713157,6.8391285,11.624084,2.2127523,5.2984347,9.476681,5.4012976,2.2191675,2.2127523,2.2369735,2.9826972,2.2127523,8.950495,10.830829,8.205136,2.3161461,7.775718,8.057438,2.2127523,2.968383,2.2127523,3.5264382,4.7546215,9.58954,7.349057,4.5102873,2.2127523,9.70645,6.796494,4.257977,10.379179,7.31949,2.2127523,3.2788475,4.634392,2.469735,3.7427237,2.2127523,4.043373,7.3328376,7.9757147,3.6543028,6.1304684,7.2316923,2.2127523,9.115606,5.628071,2.2127523,5.2271967,2.2127523,2.2127523,2.2127523,2.219966,3.3438435,3.0043697,2.2127523,7.3718667,2.2127523,2.2127523,2.2127523,2.7125554,2.2127523,2.2127523,2.373264,6.8845587,4.0949845,9.460612,7.268226,10.291353,4.301052,2.2127523,4.656392,11.354742,7.4411845,3.639049,8.444705,3.0841281,5.25869,9.441864,2.2456667,4.680822,6.211836,8.564034,2.2127523,2.2127523,8.389811,2.215771,2.2132146,2.4416058,2.2127523,2.2375932,2.2127523,2.2127523,2.2127523,8.493428,2.2127523,7.30098,6.07305,5.6487184,7.5680013,2.2127523,6.684725,8.078352,7.981259,6.418975,3.0580893,2.2127523,2.2127523,7.058685,5.8533916,6.98074,6.476929,2.5454125,10.234538,6.924528,5.7262645,2.2127523,3.5379348,2.2127523,2.2127523,2.4619963,6.909791,3.8613257,2.2196364,7.3008947,7.5161543,2.2127523,2.2127523,2.2127523,9.952806,8.389319,6.6926775,5.411553,7.249569,7.151242,2.2127523,2.2127523,2.2127523,2.2162313,2.2127523,2.2127523,2.2127523,2.2127523,3.6947243,2.2127523,2.2185235,2.2128577,4.5185366,5.8402333,2.2127523,5.139874,3.5008502,4.057483,2.2136676,2.3927178,2.2127523,2.2127523,8.058028,6.9514413,4.0569906,2.2127523,2.2127523,2.291522,8.114163,2.2127523,5.7249193,2.2127523,2.2127523,6.0820923,4.0847983,7.8076396,2.2127523,3.0395079,7.5513763,2.2127523,2.2127523,8.321097,2.2127523,4.3589973,9.810148,8.563818,2.2127523,5.5000777,2.2566032,2.2127523,9.894927,5.824647,2.2127523,6.582391,2.2127523,8.781669,6.4067965,3.5462315,2.2127523,2.2172806,5.935939,2.2127523,2.255915,2.2127523,2.6460595,3.8680923,9.480846,2.2127523,2.3031611,2.9469407,2.272233,2.2127523,2.2127523,8.740429,2.2127523,2.2127523,2.3488064,2.2127523,6.632964,2.2127523,2.427202,2.2127523,3.9281673,2.2127523,2.2127523,2.5774064,2.2269528,2.2127523,2.2127523,2.5696137,8.062378,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2409546,4.2738805,2.2165961,2.2127523,2.2127523,2.2127523,2.2127523,5.725408,2.2134264,2.2127523,2.2127523,2.2127523,2.2127523,2.2460806,2.2127523,2.2127523,4.2035985,2.342114,2.2127523,4.661604,2.2127523,2.2127523,2.2127523,2.2244947,3.2162075,2.215591,2.2618146,4.489466,2.2127523,2.2127523,2.2127523,7.9455514,2.2192276,2.2127523,2.2127523,3.7135649,8.277544,5.689215,7.5835485,8.475587,5.192705,6.8636155,3.7412202,9.881456,4.7777925,11.507104,4.7155647,4.078868,9.831818,2.2127523,2.2127523,8.4137745,2.2127523,9.171792,7.1340384,7.122772,9.63519,5.8106446,4.922137,5.096783,7.060547,9.997606,2.2164357,3.9328504,2.239547,2.4015424,10.921444,12.373248,10.943825,9.633154,10.793923,5.1070595,4.0254908,11.883608,9.705855,10.917251,5.9218216,9.307368,10.966033,5.382145,4.9473443,5.5097184,5.965005,4.674316,7.303311,3.4746804,2.2127523,2.435346,2.2127523,2.225578,5.932222,8.332228,6.9921174,7.56656,7.711095,6.853643,8.053192,6.6750154,4.997416,4.509239,10.62245,6.7993317,8.221873,2.2127523,2.6269825,10.394405,9.191669,5.50724,8.0879345,4.8087173,4.4538417,2.2379665,8.34604,4.790076,8.081283,4.9164834,10.646486,7.1022573,6.9607563,9.889432,11.0133705,9.041432,11.514955,6.560009,7.5254498,6.7449245,2.608304,7.2924557,11.405827,3.1104085,5.4165053,3.6636217,9.7254505,2.2353487,6.2734847,7.5762477,2.2127523,3.2000515,9.163841,7.7199397,2.2127523,5.9425964,3.8904402,2.2127523,2.2127523,2.7034655,8.244685,7.002776,8.40465,2.2127523,4.520739,3.3017135,2.2127523,2.252127,2.2127523,6.2806187,7.509705,7.882379,4.954254,5.125482,9.906534,6.7867837,6.416072,6.1617274,5.9589176,3.7465189,9.953511,5.0259914,2.3017383,7.420967,6.008892,9.698511,9.620942,7.8945255,2.2149067,2.532123,2.2127523,11.376836,7.160981,3.7983043,4.1128993,2.3621082,2.2127523,5.5955677,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.5292013,7.6594014,2.2127523,7.7084646,2.2127523,3.4044642,2.2127523,3.252434,8.686878,5.3070946,2.457311,2.2127523,2.9851127,2.218019,2.2258723,4.7626524,4.9733443,2.9667132,2.2127523,2.2127523,2.2148387,2.735523,2.542236,2.2127523,3.2786512,5.0309877,2.920774,3.6437018,3.2994285,5.80046,2.561162,2.518319,3.2059426,2.2674177,2.2140367,2.2127523,2.2160335,5.409426,2.2224545,3.3664777,2.2348747,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.8078704,2.3307605,2.2137854,2.2127523,9.291509,2.2127523,2.2127523,2.36609,2.2127523,10.59087,2.8423615,5.7780266,2.261787,2.2127523,11.197862,7.3030357,5.539316,4.420376,4.4599457,9.366832,3.532854,5.0424585,2.2127523,2.2127523,5.5248256,5.9482527,6.5171533,4.1892686,2.2127523,7.350832,4.7038083,2.8987033,2.2127523,3.6033344,3.135779,3.9997938,2.2127523,11.318812,6.994069,2.2127523,5.460745,4.231019,6.53428,8.052876,2.2127523,2.2127523,2.226496,2.2127523,10.341979,9.579018,8.209616,8.101436,5.461396,5.242625,2.2696593,6.461511,9.276909,5.940442,2.7536085,7.491036,2.2127523,9.6082325,7.958892,2.2127523,2.5397527,8.034792,2.4686344,2.2127523,3.509272,11.481935,2.2127523,2.2127523,6.600089,2.4523149,8.736509,6.7985063,4.7478147,8.036838,6.3169556,8.288409,7.437432,10.414485,2.2127523,2.2127523,5.6463923,3.6077588,5.1841264,4.654304,3.9541178,2.2127523,2.2127523,5.646973,6.8997188,7.8106794,4.980594,2.2127523,3.3599303,6.7839975,6.597127,2.261835,2.2127523,2.2127523,2.3337839,3.3905542,3.868593,2.2127523,9.63168,3.9365401,2.39198,2.2127523,6.1661925,4.9286995,5.9437256,7.460987,7.8062196,7.944163,6.152202,7.310059,2.2127523,2.2127523,3.4853437,2.2144144,8.87704,6.9442253,2.2127523,4.72948,9.762897,7.7301855,5.7739444,9.862434,7.6872425,6.553737,4.842757,6.157103,6.969035,5.836964,2.216196,8.407715,4.2314053,4.915844,6.84729,9.896346,7.689337,2.6637113,2.2127523,5.304597,7.723656,2.2127523,6.21489,5.5548563,6.812162,11.944469,7.441929,2.2280605,4.098988,10.684368,9.509038,2.858588,6.2989097,8.871887,7.9042544,10.300478,4.28623,5.642866,2.2127523,3.7861001,8.481112,9.16534,5.8331733,4.109886,6.0947485,10.183784,8.532117,8.674538,2.2143555,5.5761805,8.354115,3.6043956,4.6138673,9.839432,7.5904465,2.472037,2.2127523,8.770538,7.8725653,6.7089677,8.835452,5.551611,11.647906,5.9858155,3.3374941,5.4227366,2.2127523,5.618095,6.9744186,6.408605,8.609762,8.34975,8.023428,10.560964,5.583849,4.189136,2.2127523,10.053771,7.267169,6.5425096,8.976825,6.73783,7.814884,2.5938182,2.2127523,2.2127523,2.2127523,3.0733442,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2241006,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.5015419,2.2127523,10.358108,3.6304395,11.890117,3.848429,7.0521126,4.620815,2.8742268,2.9393983,7.6826825,4.6322637,7.6211047,8.799979,7.6452584,9.5066,4.1614776,9.714179,4.5277,7.7265635,2.7720754,2.2127523,3.1810565,2.2177184,4.669003,2.473891,5.2148056,10.416291,8.594553,8.012733,5.2817054,3.8762362,8.275637,5.739354,4.046033,6.9212136,6.8733263,5.9833436,6.586428,3.6541667,2.916878,6.9439764,2.5656412,2.225664,4.856354,2.2127523,3.8481953,6.497742,2.2654152,8.780184,2.2127523,2.2127523,5.4499955,2.2258084,6.2717957,2.2127523,8.722951,2.901327,2.6392617,5.077471,2.2127523,2.2164576,4.9521093,5.080485,5.8744073,9.602683,6.1473665,5.782689,3.1307454,7.8471212,6.911191,2.2127523,2.6207767,8.028407,8.790645,5.128871,3.890606,6.4327517,2.5977297,8.678995,8.879809,5.4497404,5.082151,7.101633,2.2127523,5.5812263,4.631958,6.1408887,11.330793,7.9327235,9.392514,5.2437677,8.460131,7.267593,3.394889,3.5350435,9.105919,4.0994997,5.47247,10.574573,4.3163104,4.7637973,4.8321295,8.928329,9.735637,6.1863837,5.5821905,2.2127523,2.7580988,2.7124734,5.8153157,2.545262,6.370668,7.5887127,6.690619,2.2127523,4.9174366,2.2127523,2.4673624,3.1201172,4.4380045,4.867451,7.243144,2.2242785,2.2127523,9.785409,2.2127523,5.755847,2.2127523,6.8192196,8.727114,5.035168,2.5222754,5.5191383,2.2181687,4.5193686,8.214409,2.2197936,2.2127523,2.2127523,2.2127523,6.518488,2.5806253,2.2127523,10.5789995,2.2127523,2.21722,2.2127523,2.3886406,2.3304548,2.2127523,2.2127523,2.2127523,2.337094,2.2162564,3.9160905,3.600169,5.138543,2.2127523,2.2127523,2.2127523,8.803515,7.085891,8.986988,2.3596208,6.8838086,6.080929,2.34158,2.2127523,2.44263,6.529503,2.2127523,2.2981477,6.419331,2.2232504,2.2127523,5.4258037,2.6798975,2.3349383,5.3315387,2.2999895,2.2127523,2.2127523,2.2127523,2.2143166,5.9188037,5.6232033,2.4297464,4.054459,2.3577044,2.2127523,5.6249146,8.092599,2.2131321,2.6859577,2.6638837,2.632518,2.2127523,2.2127523,2.2127523,6.350789,9.711203,2.2127523,2.2599933,3.3456407,2.246955,7.735046,2.2127523,3.2333114,2.2127523,2.2766457,3.4086618,2.2127523,4.1531587,2.2127523,3.9711223,2.2127523,9.94301,3.8964336,3.8977547,7.8986554,6.785401,3.674691,3.683233,9.08136,2.2719834,2.465323,2.7195096,2.9389281,2.3775568,4.4363093,2.2215106,2.2127523,4.0144258,3.9355333,5.851866,2.2802188,2.5885046,2.409809,2.2221353,2.2127523,2.4795582,12.137073,2.2127523,9.33972,8.143919,4.9708843,8.890818,9.185328,6.6049433,3.2618368,9.976271,6.624155,7.2718554,2.4694061,9.633036,2.2426343,2.2127523,2.2127523,9.351019,9.411006,7.043502,6.5533752,3.1892092,4.914442,6.750422,5.5309696,5.227515,6.27078,3.5117347,7.9945,5.78555,5.5692253,3.6455028,5.564929,7.952748,4.307144,2.6290395,2.936509,9.22323,3.3318582,4.2442017,4.7374454,6.443616,4.7934155,2.2127523,3.588906,6.5479937,7.789139,6.817006,2.2127523,3.862303,4.995567,5.5544043,2.7841387,7.87497,2.2127523,3.0433369,8.582968,2.9727037,4.8345413,2.530497,2.2587316,6.6809144,3.3365927,6.635604,2.2127523,2.2127523,2.2227125,3.3039656,2.8113828,4.101132,4.55123,10.086586,4.380397,4.1238422,2.2127523,5.9080033,6.240241,2.2253017,10.587115,9.627968,2.2127523,2.312185,3.7607353,5.5018315,7.3033133,5.243872,2.216203,5.473658,2.2127523,4.8150606,3.8141725,5.500475,6.8291054,5.682015,2.4081354,4.124266,6.522114,7.8670955,4.6242723,5.7858925,3.1930776,3.100974,3.3646853,7.303708,7.5818877,8.774474,4.3279,3.6508625,3.6567369,7.2395573,7.5996795,2.4706285,2.2173831,4.594347,2.2264547,2.902641,8.519018,2.2165914,5.9612145,6.6233974,8.592722,2.2218065,6.959119,4.265898,2.2127523,7.641831,11.779833,8.5735445,9.1809635,3.5019116,5.2737,2.2127523,7.789909,10.50862,2.213,6.0490737,8.895148,4.1187,2.2127523,2.2138128,2.2127523,2.2127523,5.6948085,5.804426,2.2127523,2.3939521,2.6432657,2.2127523,2.2127523,8.187888,7.148705,9.184806,5.167434,2.397868,12.421916,5.614646,2.2407424,2.2127523,5.7490187,5.459469,2.2127523,2.2127523,2.2476258,2.2127523,2.2127523,2.2127523,3.172373,2.2127523,2.2127523,2.2127523,2.2442856,7.1497426,2.2127523,8.400277,2.2394083,2.2127523,7.062359,10.292902,8.48856,8.28759,2.2651608,2.4035628,2.2127523,2.75496,2.2127523,2.2127523,2.2150662,9.705196,6.5216928,2.9633255,9.870243,2.2127523,2.2127523,6.132873,2.2127523,2.2127523,2.2127523,2.2502046,3.9775603,2.2127523,6.7807016,2.2127523,2.2127523,3.433359,4.5584373,4.017379,5.198716,2.2127523,10.40213,2.2127523,2.2127523,6.698102,12.495221,5.038579,4.820826,10.075544,2.2127523,5.963587,5.021834,5.402532,7.3151426,2.219741,7.282988,2.5734427,2.2572083,5.6115894,4.5829797,6.2732553,3.3750231,5.178585,6.0144725,3.6265943,11.622026,8.871273,4.428315,4.042894,2.8514054,6.017992,2.412549,8.87284,5.5804234,4.1246347,6.682592,7.862093,7.4170766,8.969506,8.997223,2.3202412,5.3638487,2.3595412,6.127083,4.566298,2.8651528,3.5871022,2.2127523,8.6961,8.301535,3.8155298,2.2127523,5.845803,7.2833242,6.4538374,6.7539477,3.4455872,11.834923,5.130136,5.0670037,2.2127523,9.22516,2.6601992,2.2637155,4.5228777,8.146745,5.8324847,5.233228,5.976341,2.2127523,2.2127523,5.476615,4.322967,2.9147973,7.9220443,8.607245,8.396777,3.3281949,7.0583997,11.541176,3.72666,7.968249,5.9493294,2.2127523,11.679843,9.809964,3.9670663,2.358697,7.736311,3.844122,6.9184275,6.978475,5.0834036,2.4264607,3.8910112,4.8468266,6.1745224,2.2127523,3.491581,2.467812,6.7219577,3.9764616,7.0746818,7.354879,2.5030768,4.053743,6.2049017,8.242266,3.9579175,10.75518,4.6091533,9.345713,3.3518052,2.2127523,2.2127523,4.909632,3.238467,4.7175364,10.646588,8.650964,6.032408,10.280229,8.061412,7.104479,2.6459563,2.2127523,9.199302,9.516823,8.986851,7.3351183,10.951219,9.305688,8.100894,10.40161,4.287156,2.2127523,4.7857194,6.487011,4.999964,2.2855272,2.2127523,3.5671222,2.2127523,5.495624,6.237498,2.2127523,2.2127523,10.2697315,7.614851,8.928467,11.793544,4.930032,5.243356,2.251233,2.2175884,7.1546383,5.5568027,2.226529,4.334341,5.767305,6.4983454,12.4888525,5.7173395,4.2130604,14.273341,9.117805,4.176157,5.978409,14.180339,13.929286,8.082081,6.3508735,2.5974114,2.279659,4.156321,2.2127523,2.9423552,6.1101646,6.150892,2.2127523,4.97131,3.610331,2.2127523,2.412606,6.7888875,2.425876,7.054775,6.650375,2.2559357,5.1740785,2.3421192,4.620836,7.256786,10.532603,2.2581544,4.857311,6.1925654,3.8715541,3.8828015,5.406572,2.2703772,10.568173,3.3241692,7.8506975,7.045536,5.8619113,2.5028398,2.2127523,6.2631626,6.2655077,3.8785782,2.5987191,2.2127523,2.2127523,5.823002,6.5013022,4.6359506,4.903574,4.036338,10.768585,8.5807495,7.3440766,2.5958319,7.3698707,5.1364775,6.4668036,6.8023815,3.9858797,4.7660065,4.4404216,6.760265,5.713069,2.2127523,4.2932844,8.860455,10.627012,7.5743246,4.6460533,4.4762077,10.379353,6.339631,9.847991,2.4130979,6.6241693,2.5640771,11.236117,8.506751,7.59488,6.094025,6.1055384,9.801081,6.839236,6.5236993,8.002735,5.1753254,9.2276945,8.766203,8.177738,11.6296625,9.912752,7.6631184,9.764146,11.449716,10.251868,7.3411717,11.772716,10.535933,10.61206,9.344737,6.241112,9.469721,13.647116,3.6459522,7.038513,11.580637,8.284849,10.067531,10.102278,6.659564,7.035053,6.571231,6.9349084,6.3315883,4.9790626,9.233533,6.184414,5.0943923,11.461351,7.7590075,9.9656315,2.8221104,7.4118032,8.953814,4.8943796,4.95799,2.2127523,5.9789248,2.2127523,2.809173,2.2738078,9.261677,7.947296,9.410004,7.320911,3.232336,3.513492,2.2786374,5.140116,4.1626115,3.2155032,6.784304,5.2971115,2.79373,7.2507215,2.2127523,6.456609,4.946568,5.762342,7.519169,2.2147465,5.016231,4.1354685,4.113114,4.1573386,8.327483,9.186575,2.3369942,2.2127523,2.2127523,2.3539703,5.7675815,3.583784,6.7515316,5.9624376,7.667329,7.355397,7.2702746,4.8274703,2.2127523,5.269343,8.815924,4.6369863,5.525099,10.486145,6.286723,4.3771634,10.833237,8.946877,6.3401527,6.58506,5.803713,4.1192617,6.6834564,5.4024277,4.204302,8.655628,3.6046891,4.105775,4.6848063,5.52724,3.8315375,4.0429225,5.0880346,8.138265,2.289747,3.2903306,5.288021,8.690936,9.158561,3.9326005,3.2856772,2.2127523,2.2127523,2.2127523,2.215693,2.2127523,2.2127523,6.9083743,2.2127523,7.041775,7.424199,6.771563,2.2127523,3.9428115,6.504897,7.908137,2.3406,5.7506876,2.2127523,8.3717575,2.2173421,7.081516,2.5830379,10.085556,8.384714,3.8296344,4.4767513,3.3418725,2.4539926,8.99895,2.6458397,8.939242,8.03259,2.2127523,2.89679,8.6782875,2.2127523,3.6843097,7.1415806,2.949378,2.2768724,4.823639,2.2129676,6.4325657,3.8137953,5.336186,4.943454,7.1368484,8.916067,7.8074174,8.76148,6.612468,6.0425186,2.9679937,8.265577,4.1499915,3.5964577,5.8426113,7.2624526,3.598252,7.498036,2.7283616,3.9220765,6.952197,2.2127523,6.1340604,8.474637,13.896693,9.846986,4.0955477,2.9221964,2.2838619,2.4969938,6.3640366,2.4115796,5.647976,2.2614157,2.2127523,2.2127523,2.2127523,2.2167401,2.2451804,3.6431475,5.3071365,6.7251363,4.610028,11.195818,3.0561337,6.394581,2.6034813,3.1933002,8.2860565,3.9714282,4.7606726,2.2127523,3.0423963,2.4242022,5.226618,2.2127523,5.784166,9.924874,9.866065,3.3921018,2.2127523,12.764027,5.153645,5.197058,2.2127523,8.711787,5.0171113,2.2127523,4.044468,2.2127523,6.09196,6.5472407,6.747646,5.832591,8.394034,6.2135253,2.2141473,4.231003,2.379645,2.2127523,7.9148316,4.0041676,6.4508867,4.965129,9.467265,4.236622,3.1021948,4.939005,3.8082106,5.2547255,4.247872,3.03857,7.435312,7.477599,4.880177,11.949895,5.493837,4.37873,5.770152,3.8109555,2.2127523,2.3448796,2.2127523,2.2177746,2.5483937,2.2127523,7.758611,8.052407,5.715493,2.6863434,6.1704316,2.2127523,7.875204,6.0306997,5.857189,9.719096,4.679894,6.1893406,11.255693,7.426219,12.020706,3.0859716,5.9163876,2.2127523,2.2127523,2.2143042,2.2127523,2.2127523,2.2127523,3.4926286,2.2127523,3.8625343,5.62257,2.2127523,2.4123611,2.2127523,2.2701154,2.2165294,5.978012,8.206022,2.2898388,7.2341814,8.388122,5.796761,2.671084,3.1877048,2.2127523,2.9970853,2.2817702,2.2127523,9.730334,9.66738,6.481246,2.2155838,2.783047,2.2127523,8.574127,9.57905,5.1691246,7.85337,9.617898,10.895092,9.305208,7.1683097,6.7909718,7.1903424,2.2127523,2.2127523,6.810547,5.995254,5.799975,2.3832161,4.8773775,3.6425679,7.7440147,6.918727,4.6359844,2.2150283,5.7384715,5.874487,5.788801,4.3715177,2.7291522,9.58065,9.879144,4.533587,2.885065,7.256374,8.038012,4.166036,3.5089839,7.362135,2.2127523,5.396617,2.9888525,2.9395905,4.307921,9.799507,5.906696,6.926785,2.2505593,5.565339,2.2226944,5.9360185,7.618891,5.546535,3.6701853,6.3001447,9.717025,2.9861627,5.0638866,5.717857,5.0261755,4.912997,7.0809326,6.409004,6.655356,7.9811068,9.804352,7.8387656,6.7405243,6.1982956,6.332178,2.346394,5.747307,2.5333595,2.566351,2.2488916,2.2239838,3.6263072,2.5654762,2.2127523,4.694478,7.6789064,8.344486,2.7672555,2.5735302,2.2127523,10.392559,2.2393498,10.03353,2.2127523,2.2127523,5.067986,7.033402,2.3265724,2.2127523,8.349125,3.764328,3.4208453,6.2877836,8.787162,2.2127523,3.8292925,5.4408784,5.487941,2.2127523,2.2127523,4.7666306,2.2127523,6.1035914,11.948134,2.5070858,2.2127523,8.473742,7.5976534,2.4406433,7.563164,4.8366656,9.162713,4.8300815,7.7569013,6.097816,4.189856,8.245779,2.2232168,6.9356637,2.4588852,7.60706,5.4402876,7.6049585,9.400055,9.328752,4.936532,4.91581,2.639275,4.5116134,9.456075,6.4311223,6.595711,4.96543,3.294194,7.281674,2.2180831,5.625818,2.2351658,8.088471,7.430502,6.994422,2.227877,9.017932,4.405122,4.0883784,5.7252383,5.9732137,6.8523273,5.521064,3.9411883,5.7814207,8.073002,2.4970293,2.7341828,3.2915802,5.1757402,7.6569467,9.387298,7.50813,8.785609,7.750146,2.2334535,3.8747456,2.2127523,2.2127523,2.7487867,4.5375376,2.3246508,7.5208755,8.521498,3.9007804,8.507152,4.455692,2.2127523,9.20643,2.2127523,7.02515,4.8772492,9.1066265,2.5711892,5.6280084,5.208259,2.3143208,2.2127523,2.5806348,2.2127523,2.3438323,2.5018928,2.2127523,2.3578417,8.39458,2.2127523,2.555985,6.446126,5.6017485,3.4828255,4.0463877,6.1342015,7.5190506,5.2163024,2.2127523,2.2127523,2.2127523,3.4934537,5.120073,7.723911,6.1946425,3.6401412,2.2130423,2.2127523,2.2127523,2.2127523,4.8290606,2.2127523,2.9608898,6.3825755,7.7114296,7.01026,9.918303,6.113227,7.0413647,6.6346984,12.168751,5.5716505,4.3680034,4.1275043,8.836435,2.2127523,2.2127523,2.2127523,2.215666,2.4286523,2.2127523,5.3126984,6.2569246,5.007338,2.2127523,2.2127523,2.2127523,6.1646266,6.6404843,2.2135174,6.7228684,5.0939894,7.882713,7.626598,5.357476,7.171231,7.7256746,2.2127523,2.2127523,6.4538198,6.401946,2.2127523,2.4185092,2.2127523,3.2756548,4.714484,6.5603485,2.394251,2.2127523,5.7962084,2.361087,2.2127523,2.2127523,9.069654,4.685127,6.8141665,6.324464,2.2127523,2.2127523,9.642625,8.523255,3.615126,4.970332,8.565928,4.3434434,3.1342452,2.2991009,8.979262,7.5727925,7.9783754,5.240744,2.2127523,4.5674987,3.207888,2.2127523,7.8731437,2.267667,5.321053,6.4582148,9.321375,9.781134,2.2127523,10.097038,7.423975,2.2127523,2.242596,2.2127523,7.3846364,2.6075077,3.6981866,2.4208553,4.056693,5.3176565,2.3783488,2.4600983,6.767582,2.2191494,4.5345225,2.2127523,2.2127523,2.2127523,2.2127523,4.599642,2.2127523,3.1750925,5.9579144,2.2697034,2.5343616,9.156695,10.6720085,5.204351,8.874154,6.882162,3.0988233,2.2199836,3.6804066,2.223359,6.4710674,6.354748,2.403563,8.461419,2.2127523,2.2127523,2.2127523,2.2127523,4.0750494,6.3062415,6.5634317,8.493632,6.49103,3.0989723,3.995396,2.2477057,8.140164,5.756884,2.2478685,7.2965326,8.878757,8.570593,7.5484743,4.235608,4.2871876,5.128149,2.787052,2.836567,2.2142556,2.2127523,5.5719876,3.7089126,2.2127523,3.439811,2.2127523,2.2127523,2.2127523,2.2782304,2.2531683,2.2147875,2.2127523,7.0807586,7.0396814,6.4892635,4.068286,6.0398784,6.870107,2.2130492,2.2127523,3.9034538,5.789907,3.8215628,2.2127523,2.2127523,5.5139866,4.6688876,2.2171414,4.4676185,5.349123,4.2528563,7.476377,6.0110555,5.31923,2.2362287,3.5279722,2.2127523,6.341895,8.501494,10.619924,2.2127523,6.1234746,5.724334,2.294976,2.5569673,11.545336,10.309549,6.7547956,6.250796,6.9746704,2.2503154,2.2127523,2.2127523,2.2127523,5.433339,3.5852406,2.2127523,2.8548756,3.5406797,2.2969878,5.7966814,6.6304297,6.251059,4.4194856,10.038149,3.6831057,3.2056644,7.1571617,8.684096,2.2209768,4.1329885,2.3899086,6.2386937,4.2717166,5.124538,6.871636,5.4329114,8.870874,2.2127523,3.194857,4.5849147,7.3437366,9.123662,5.5737786,10.155773,7.0773416,9.157596,2.2127523,2.537766,6.1966953,3.6504076,13.343421,4.2584157,9.640853,3.7567024,11.000583,4.6382227,2.2212093,4.681948,4.4472694,5.6175256,3.6273813,6.4762635,2.3289814,3.2767022,11.129012,2.2156215,4.5844617,4.0793056,5.300387,2.2127523,4.515068,6.633776,3.1098733,4.1976233,3.1814358,3.7095702,2.2127523,6.198974,6.006349,2.2127523,2.2127523,6.351715,5.9971957,9.90524,3.0100596,4.496948,2.7191973,2.7146466,7.7731314,3.7036023,5.2488327,3.4053562,5.7066655,4.162339,6.8962226,6.2017426,2.2603314,7.2532296,2.2308397,8.360085,4.5562606,5.3385525,4.5294943,4.025528,7.552027,9.956417,2.297956,8.793534,2.2127523,4.51372,2.2127523,3.150408,5.2495327,2.2127523,3.6855752,5.8725476,6.4794345,5.787128,8.521055,8.582044,8.465727,6.30149,7.906841,6.1754994,5.4586334,9.346525,3.4681451,7.1965766,6.119955,4.8353453,4.6484733,2.2127523,2.2127523,4.4510055,10.246974,2.2190433,6.054204,8.0046215,5.1235476,8.396691,5.1265726,8.570715,6.973393,10.504846,8.068417,9.444331,2.6689703,7.7721453,2.728129,5.532218,5.012402,2.2127523,6.6523237,2.216678,2.4550378,3.8208954,2.5561383,4.645383,3.0965867,3.3464377,5.5681934,8.975528,10.694382,3.7985065,2.3126037,7.8047833,7.0921726,2.6413038,5.814366,5.4375124,4.801224,6.6519017,7.996431,6.807791,3.1637769,6.1132016,2.751786,2.2127523,2.2127523,2.2127523,5.097435,8.16552,7.397636,5.477062,2.2127523,2.2127523,3.7124317,7.1506953,2.2155159,2.3780787,2.2690167,2.2127523,2.344424,2.3103518,2.2127523,2.822206,3.7955966,5.517837,6.610995,4.7778163,6.556298,6.9091344,2.9312415,2.2127523,2.2127523,2.2139845,2.2127523,5.427616,2.2127523,2.277322,3.4500198,2.2127523,2.2127523,2.2127523,2.9137592,2.2127523,3.0680146,2.2127523,2.2127523,2.2254663,2.2127523,2.2127523,2.563399,2.2127523,3.7933269,3.4722133,2.2127523,3.4781275,6.054773,2.2404907,4.0348587,5.3175554,4.8713565,6.6621585,3.6648839,3.6885884,2.6552508,3.701019,7.361639,3.0766876,3.3612504,9.952286,7.0777116,8.716019,2.2560725,6.557598,2.895238,6.7006474,9.977521,3.2031868,2.4155273,7.4888144,2.2127523,2.3351178,3.5854144,8.602686,6.1884184,7.9146385,3.6614802,2.2127523,3.1009023,6.058071,8.184859,7.899314,6.56058,2.2127523,10.355175,4.3793845,4.961578,3.843704,3.7823293,5.087021,9.147833,6.6243205,5.914974,8.872203,2.7930017,4.803876,9.967092,7.1780105,5.0076427,9.190507,2.2127523,5.029013,4.100807,8.986372,3.4799564,4.1753893,2.2127523,8.997924,6.9883537,7.7874737,4.784433,2.2127523,2.213384,2.2127523,5.54528,7.6575966,4.502403,2.332608,2.5201051,3.765705,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6879048,2.2127523,6.540996,2.2127523,2.2127523,3.468293,2.2127523,2.2127523,2.2127523,2.2127523,7.05244,2.2127523,2.2127523,3.0282009,2.2127523,2.5123248,2.2127523,2.2127523,2.2127523,2.3761063,2.2127523,10.256946,2.2127523,2.2127523,2.2687242,2.2127523,2.2127523,4.7070913,2.2127523,2.2345674,2.2200384,2.478055,2.2127523,2.2127523,2.2127523,3.5918355,2.2127523,4.1812696,2.2127523,2.381695,2.8778806,2.2127523,3.7172554,2.2127523,2.2242312,2.2127523,2.2127523,5.903952,2.3011305,2.2127523,2.2127523,2.2127523,3.9964101,2.2432773,4.4769864,2.2127523,2.2127523,3.3905308,3.3697245,2.2127523,2.2131994,4.311617,2.2127523,10.023018,8.689372,4.8103356,7.2004437,6.465228,2.2127523,4.0736113,9.113564,4.874139,2.2260032,3.337202,3.2741997,8.635582,8.704352,5.433189,4.1064115,4.869221,2.836222,2.4432828,5.3357286,6.1983533,3.6009936,4.7110925,7.6704965,5.078437,4.8076205,8.031839,2.2127523,2.2127523,6.6395917,10.187029,2.2145386,10.016045,2.2127523,8.202394,7.326344,10.077908,5.7332473,4.982127,5.5954914,8.17797,2.9814029,3.0208867,2.2236779,2.2128663,2.2127523,4.774544,10.3284855,8.8450985,5.542625,3.0932205,2.3047264,11.389269,5.4402146,2.9587426,8.711671,9.833206,5.589515,2.6726558,6.399991,7.3549523,3.7668214,2.2127523,2.2127523,3.641459,2.4143853,2.2380862,2.2192674,2.2579696,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.695966,6.8682966,2.399996,2.3567781,2.2127523,5.0677257,4.8272643,5.865907,4.030253,4.1177173,2.4391582,2.2127523,2.3000588,2.2127523,5.844949,5.287774,2.7954557,2.2127523,2.2874641,5.364175,6.309822,5.291026,4.8689275,7.0524435,7.6200957,3.1226993,2.2253208,6.1688623,12.668931,2.2127523,2.2127523,6.4669776,2.8242984,2.2947927,7.2945967,7.4990945,2.2127523,5.129722,2.395311,8.023515,4.9423666,2.441031,5.3418217,6.99184,2.3386378,3.2623336,2.2127523,8.184596,3.4993763,4.8233957,2.2127523,4.8322597,4.646861,4.276235,2.667826,7.761387,2.4285402,2.2127523,4.1042595,2.263693,2.2127523,2.2127523,2.2182376,2.2127523,2.2127523,2.2127523,4.9487066,2.9443343,4.6048503,2.2127523,5.548811,4.1112094,2.615483,2.2127523,6.8092065,2.2127523,2.2127523,2.4736907,3.901708,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.8433874,4.59729,5.817528,2.2131364,2.2127523,2.3775327,2.88606,13.304337,14.351912,2.2127523,14.818373,8.622486,7.9764476,9.023972,8.468647,8.429944,7.5738406,8.72992,6.7118654,8.337156,7.0860696,9.815671,7.0037956,4.5333858,7.087084,7.6063333,3.9180229,2.616907,7.67334,4.805337,4.0855513,5.1441693,3.5531673,3.1566582,2.513535,6.20424,4.483524,2.2127523,5.562704,2.2127523,2.3147268,4.5985966,7.854875,6.899082,2.2127523,2.9124625,5.877328,2.2127523,8.642319,5.30516,2.2127523,6.928116,5.78512,7.819882,9.499442,5.240027,2.4321506,2.3050795,5.8526993,4.72376,3.4444683,5.911343,8.049582,6.376626,4.8419194,2.26158,4.7124925,9.095048,6.4669533,7.057431,8.803755,2.2213006,2.2127523,6.916322,5.058515,7.4642286,4.3716063,8.068872,3.9765534,2.2127523,12.706571,7.8868647,10.995865,2.2453907,5.337115,4.00254,6.8667083,6.5869,6.7469845,6.02167,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2134907,3.6189811,2.2960992,2.2161143,4.8842626,2.2136538,2.2127523,2.2127523,2.2481713,2.2127523,2.2127523,2.285305,2.2127523,2.2265499,2.2127523,11.244656,6.854184,6.7252283,3.7499006,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,8.391337,2.2127523,2.9994721,2.5266771,2.2127523,6.8859463,6.512751,5.034661,3.4886549,6.494378,7.817443,4.8724766,9.560241,7.6600604,11.0832205,7.7768598,2.916137,4.862852,2.7933812,2.3861103,3.3449557,7.3498406,6.3368783,6.530637,3.9700422,8.481971,9.798417,2.2901826,2.225213,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.6990566,5.194648,3.4975212,10.769464,5.5496984,2.2469504,14.523133,7.7310505,2.2127523,3.6343408,7.5876126,10.430867,5.6828227,2.4990194,2.8015602,11.06001,6.0409164,4.914963,5.8037047,5.771432,2.2127523,7.0279083,10.414193,2.2235565,6.853609,5.3028045,5.4074507,5.3267856,2.2267585,2.2127523,4.7194667,4.202308,2.2127523,2.2127523,2.2127523,4.446483,4.7485976,7.7804255,5.344388,6.96734,9.751312,3.3187783,2.672244,5.6285563,4.1746807,6.3024383,6.591995,2.2269306,5.094183,10.952672,9.084975,5.986126,6.2722974,9.413755,2.2127523,9.765412,10.427921,7.1661983,8.40577,2.214,2.2127523,2.2127523,2.2127523,9.473768,5.8478045,4.088729,6.0437036,5.255759,6.7625055,2.2127523,2.220701,9.680898,8.502842,2.2127523,2.2127523,6.0519133,7.8466606,2.2127523,2.2127523,8.545195,4.4920406,2.21303,5.366502,4.44122,2.2127523,3.2652392,6.709809,2.2282205,2.3176045,2.2127523,9.284866,2.2127523,3.945159,2.2127523,5.175077,10.519265,2.2127523,2.8242722,4.6216035,2.6127164,2.2143052,5.733793,2.2127523,8.2965145,11.820723,2.2127523,4.3634768,2.6040463,3.54484,4.06597,2.972652,8.401885,6.423504,5.8330603,7.2250986,6.0347486,4.8349814,3.9268458,3.9847333,4.0131207,2.2127523,4.9296346,4.3360753,2.2127523,6.38396,7.153273,3.9605033,4.234258,8.803095,5.388112,2.3897624,2.2713938,2.8156319,2.2127523,6.6951356,5.994633,4.935882,3.2987928,12.558465,8.137548,7.573784,2.225274,6.8891163,8.977689,7.3224163,3.9262466,3.5652053,4.152857,9.075271,8.245922,2.2128606,2.2127523,2.2393491,5.1645784,2.221177,10.966653,3.3149786,2.2127523,2.2127523,2.3304439,2.4686017,2.2127523,4.5099134,7.6222334,2.2127523,4.3756604,6.2415147,6.5321608,5.9029756,9.495542,8.856493,8.702173,9.583115,5.2785764,7.0543184,8.06557,5.821082,4.3091536,3.8681684,2.496442,2.2316375,8.490768,2.617803,2.7168102,3.064451,6.439842,5.284429,10.412191,2.2127523,2.2127523,3.1761665,6.3714023,2.2127523,5.522604,7.0746164,5.9527845,9.333972,3.3912709,2.2127523,2.2127523,6.7238836,3.3304403,2.749832,2.337916,3.3216076,2.2127523,6.026088,2.2747672,2.2127523,5.5341773,4.3152456,5.3401465,2.2127523,3.4488528,8.491378,9.03781,4.5156054,4.6263514,5.8263197,8.318272,5.7226186,2.885731,4.79462,5.8860064,8.422443,2.404266,3.037264,5.315209,2.2127523,6.4695916,8.185012,2.2127523,7.8371387,2.2653525,7.625108,4.8938193,2.2127523,6.680209,12.772054,9.366953,2.6863415,2.8136108,2.2127523,2.2127523,2.2127523,2.2127523,4.500856,11.026049,8.260802,5.242497,9.008729,7.0046544,2.8273304,4.055823,9.593626,6.723528,8.040335,2.2127523,6.7124534,8.13242,10.022072,9.014976,4.26276,10.808591,3.087057,2.2127523,2.2127523,2.2127523,2.537852,2.2127523,7.0088906,6.9926496,8.272663,4.477958,4.8928785,5.3014674,6.4173303,9.8992605,9.359685,4.5727634,2.2127523,2.2127523,8.293367,6.985956,3.4156473,9.2710705,6.4042478,7.4664345,7.526465,8.940249,7.378391,2.2127523,8.211627,5.8223753,8.780787,8.839316,3.2378237,3.261485,9.312837,9.102163,2.2811713,2.2127523,2.2127523,2.2127523,2.8239071,10.992598,3.170686,9.742711,10.899352,4.1244383,7.5798254,4.0702586,7.619701,12.290445,8.229123,7.521261,2.2127523,4.1386533,2.2127523,2.2127523,12.151325,4.8417773,2.2404466,2.2127523,2.694609,3.8282025,9.383432,7.0351253,4.6790323,9.445191,2.2548277,2.2127523,10.192956,9.218713,6.4565663,9.108144,5.762539,8.174884,2.7401433,2.2127523,2.2127523,2.2127523,9.677186,7.3541503,8.776502,2.2127523,5.9180164,4.8514137,6.047843,10.231455,9.407293,2.2405267,2.225458,2.2127523,2.2127523,10.590821,2.2476153,9.491157,3.4406703,7.5143175,6.4839816,2.3149428,11.471595,8.615688,5.89263,6.3808737,12.196094,10.1060915,4.978172,9.702206,7.1866274,6.325313,9.123478,4.5751467,10.273182,7.6872106,6.7811627,8.894955,2.2171106,7.8219995,2.2127523,2.6800482,3.966532,2.2166076,2.2127523,2.2301812,2.2127523,2.2127523,8.19044,6.844207,9.91872,7.424061,3.1647124,6.8745956,5.371287,4.9265385,4.7891965,2.2756877,5.173253,7.958921,2.31027,9.898925,8.6345215,3.9756432,2.3664176,3.2605972,7.875192,2.2647326,5.5253086,9.20993,2.7594662,9.560855,5.6547318,2.5526052,2.2127523,2.2127523,5.2102933,10.53765,2.596118,7.3870244,2.7949004,2.5417612,3.8686764,8.695199,6.8222795,9.375494,11.320392,3.891085,2.2143836,2.2127523,4.263866,8.642892,7.897528,2.2127523,2.2127523,2.2684872,3.331479,2.2127523,11.664918,2.2127523,2.2127523,4.742582,4.5229487,5.3203926,2.2127523,2.5403306,8.110931,2.2127523,3.1695247,9.428502,2.2127523,2.3477736,2.226119,4.332224,2.2127523,2.2127523,3.9182456,6.289926,2.9148989,4.4966793,2.2260654,8.616871,3.9462154,2.2127523,2.2127523,6.758527,2.2127523,2.2127523,3.8659084,4.698628,3.9344983,2.2359068,9.970697,2.2956462,3.6627517,2.9554422,2.2133873,2.2127523,2.220619,4.727652,2.2127523,2.2161374,2.2127523,8.226366,2.422988,2.7098439,2.267565,5.864269,3.677883,2.2127523,2.2127523,2.2127523,2.216583,6.128119,2.2127523,2.2127523,2.2127523,2.2127523,3.0308993,8.454566,4.0925627,2.2127523,2.2127523,3.0049067,2.4352498,3.9418497,2.6923802,2.2138977,2.2127523,7.92424,2.216568,2.3642397,4.9426227,9.066296,2.6344888,5.632025,2.2127523,2.2127523,2.2127523,4.0187845,5.0367036,8.211754,8.865738,5.1355004,5.448144,4.8172355,2.821632,3.6187606,12.5421295,2.3077319,10.015689,13.388649,2.2127523,5.100198,8.202761,6.755676,4.998209,6.98541,5.449522,6.7145386,5.3893237,2.3253274,2.5720065,2.2223554,4.779702,2.2127523,6.7891464,3.3758101,8.155291,8.086554,2.3226047,2.2357945,8.654293,2.2814322,6.7816305,7.5963545,2.2127523,9.344202,2.4216857,2.2127523,2.2213705,2.2127523,2.9229414,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2349725,3.0901277,2.2127523,3.2135472,2.704074,2.2127523,2.2127523,2.8488526,2.213036,2.2127523,2.2127523,7.8006883,2.3404336,2.2127523,5.4769087,2.2127523,2.2869856,3.9684157,6.1779456,2.2127523,2.215599,2.253821,2.2127523,2.968265,2.3944433,2.213783,2.2127523,2.2127523,10.811946,4.918107,7.906545,5.0695014,2.2127523,6.9022236,4.663295,2.2127523,7.025042,10.248718,2.6383119,2.2127523,2.2496722,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,10.42725,2.2127523,2.2127523,5.683857,2.6135175,7.4024296,7.354706,2.2127523,2.234378,2.3369057,6.048381,5.5368743,9.396982,5.348333,2.2127523,8.573489,7.625176,7.158008,8.393042,4.1996474,2.71187,9.108023,3.9955773,10.101807,4.8650026,6.3085604,4.1157165,4.5024886,4.093894,11.090201,4.439174,11.297881,7.4342117,3.3720772,2.2127523,6.0166626,2.2127523,2.2127523,3.5726976,5.851863,2.2127523,6.374309,8.73066,8.253828,7.885275,7.6420245,7.417503,6.0709124,9.473001,8.697068,9.765813,6.070005,4.431761,10.748023,8.167472,6.227923,3.6616235,10.203277,6.7453246,8.106864,2.2127523,5.1452866,6.5818,6.364638,11.419386,3.8957527,3.5973516,4.476282,5.563321,7.8047447,6.4721966,5.7532544,3.356053,2.2127523,2.2138963,2.2792675,2.2827964,2.651222,2.2127523,2.241675,2.2127523,8.599206,4.221207,7.491406,5.0585685,5.8673353,2.2127523,2.2162929,2.2127523,6.6875563,6.610032,5.3666215,7.381325,3.1643817,5.212963,2.2310529,9.539892,5.153773,2.2127523,5.4097433,3.349844,6.6557913,6.7572703,9.630487,2.5328948,5.8297014,2.2127523,2.2127523,8.119567,2.2412312,4.970779,2.2127523,5.570811,9.298874,9.031059,6.6942186,9.071327,4.8642597,2.9358308,9.60477,2.3185487,2.2127523,8.991347,2.2127523,2.253935,7.613095,9.553221,4.102776,8.945379,6.9127393,12.321012,2.2127523,11.770488,2.443285,12.868275,4.6742496,3.5513425,4.037348,2.253619,3.3170075,5.842025,7.496505,8.257026,9.75629,2.4434474,10.963947,8.096316,4.9470406,6.333065,2.2159622,2.2590466,4.7796493,2.2229335,2.2127523,2.2127523,2.2127523,2.2127523,2.5750468,2.3401935,2.2127523,5.3224125,2.2127523,2.7381883,2.8257294,8.762151,7.3811865,2.316324,3.895333,5.4842,10.701545,9.069673,7.5639863,8.541668,9.012273,5.1363764,7.046491,4.044952,2.7693079,2.2127523,2.3886168,4.739879,7.347054,11.6472435,9.181979,2.2127523,3.5667756,2.2127523,4.9443197,3.9443035,2.2127523,5.801684,2.2127523,5.109918,2.2127523,2.2127523,6.396333,2.2127523,7.852579,4.235916,7.3116336,6.839405,5.1900954,5.2611036,2.2127523,6.322088,2.9682348,2.4628277,2.2127523,2.6400876,2.2127523,8.809703,9.400879,2.2127523,2.2497857,2.4091096,2.2127523,8.447693,2.2127523,4.6687493,2.2127523,2.2127523,2.2127523,7.5441,10.35722,2.945641,4.103777,6.1868567,8.033078,3.787354,4.3941355,4.007343,4.2809505,5.4080825,3.739527,3.022534,6.2786627,6.784717,10.466315,4.9797835,6.45851,5.435811,8.38146,9.652637,6.7872486,6.196439,2.6379738,6.1919503,6.6617446,6.905539,2.2127523,9.534522,6.346859,7.967592,2.2127523,9.620102,6.279203,5.5732803,6.545005,3.525943,5.8652053,3.2405043,2.3103068,2.2127523,3.8678343,7.9324503,8.751366,4.5227246,4.5497203,5.257686,8.286807,3.2115934,2.2127523,5.541516,5.9542117,2.2127523,2.2127523,3.4482737,4.9087753,4.0056353,10.099724,3.104085,2.2127523,2.2127523,3.1578863,5.1506095,7.893276,5.1665807,8.241402,9.294199,6.364563,2.758161,6.5543833,2.2127523,6.317571,3.3961914,2.3285937,5.228267,2.2127523,11.875997,4.223919,9.259854,3.6647358,9.099039,3.3748083,3.176583,4.339947,2.2127523,2.2968457,2.2127523,2.7065334,3.0894039,2.2127523,5.4280405,6.4860096,5.336691,4.326152,9.758112,6.7982006,4.473742,6.722587,2.2127523,2.3073266,2.4605174,2.2127523,3.6824987,10.763594,6.3066063,7.506824,7.526728,8.214278,2.6841838,4.656256,3.1340556,12.046689,6.10773,2.4901462,9.465043,2.2977893,6.139952,6.791721,2.2127523,5.887196,11.618148,9.507112,7.7589064,5.867734,5.7756147,6.25048,2.2312536,5.283444,5.546291,2.3481257,5.5333967,5.61678,2.2127523,2.2127523,11.852348,3.577689,3.0372157,3.2545087,2.2205548,12.426318,5.118143,6.0858235,6.526491,4.9405556,3.2122061,2.3706348,7.274689,2.2127523,2.2127523,3.4111967,9.232875,12.087847,9.119928,10.403879,9.912325,2.251507,12.912045,2.2127523,2.5651538,2.5729449,2.2127523,4.074654,2.2211578,2.2127523,2.681685,2.2127523,3.7748451,2.2127523,4.8539076,3.0196319,2.599001,2.2127523,2.2656987,8.091315,3.5336645,6.233995,2.2127523,2.2127523,13.29283,10.378536,3.5847263,2.2127523,5.390338,4.5712914,3.9708562,3.8991904,8.912497,4.4964314,4.3378034,2.6386306,3.8429177,14.209382,14.087432,13.755574,11.621107,11.646041,4.2386265,4.2777452,12.761273,2.2486794,9.588121,3.0143876,11.316607,11.107127,10.137527,14.414445,10.005343,4.0314736,6.0201163,12.128092,10.763744,5.1576204,12.307326,12.343146,4.5050173,8.053359,9.567555,4.116252,5.5046616,5.1079655,7.316718,6.8210707,10.23314,2.7381098,8.865011,6.2438903,5.9422035,5.456713,2.213638,12.234772,2.4239304,2.2127523,10.361203,7.1308618,9.021362,5.6266794,2.2127523,2.218452,2.2127523,8.362673,2.4603224,8.0566025,7.822402,2.2127523,12.460898,13.2217455,4.738139,5.117724,13.097568,2.3895924,10.973455,4.402215,6.9155,8.891559,5.6800857,2.2127523,5.3019385,2.2127523,7.88767,6.6706843,2.2134376,2.2127523,2.2127523,10.987462,8.18298,3.3394122,5.2021894,2.2131357,2.2127523,3.603175,6.2917156,8.569129,11.232577,10.850906,2.3443081,2.6304195,9.634717,10.500686,2.2127523,10.731806,9.245959,4.276329,8.176047,9.936116,9.94751,8.949285,10.826028,10.737721,13.368244,7.591705,6.3263965,6.155385,9.24157,8.352278,10.6073885,8.91769,7.185738,2.217663,8.042351,3.1269178,4.999314,2.2868304,7.102537,8.238066,7.2952943,9.748635,2.2127523,2.2446609,2.2127523,4.2311993,2.2127523,2.2617884,3.2661664,2.2127523,3.274015,2.2254913,2.2127523,2.3513722,2.2127523,2.2127752,2.2127523,2.4634428,5.0638824,4.227964,4.2234807,2.6931188,3.237544,2.780401,2.7719831,2.2127523,2.2127523,6.029098,3.7888834,5.5902395,4.815445,4.1005445,5.209695,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2928128,4.8948,2.319348,6.9249535,3.5609357,2.213465,2.2127523,6.8265815,2.2127523,2.2290106,2.2127523,2.2266266,2.2169185,2.2127523,2.9394798,2.2496293,7.8123903,2.2127523,2.2127523,9.237729,2.2296863,8.774304,2.2138834,2.2127523,2.2127523,10.57228,7.8775206,6.3334475,3.782693,5.6632476,4.214482,6.7847643,5.896764,9.182518,4.42624,3.1710591,2.2127523,2.2777011,2.2127523,2.6962242,7.5240564,6.4141707,2.958765,2.6228664,2.2127523,4.6620564,2.2127523,4.1896577,3.7711682,2.353373,2.2127523,2.458347,4.864253,3.3289378,6.6015377,4.4179535,7.8799257,4.2470226,4.1752205,2.2127523,3.5363498,2.2127523,4.7766,7.0789514,2.2127523,10.41786,7.3943973,9.810702,6.1272225,3.1943338,2.2186625,6.881568,8.844061,12.006891,4.3438334,5.009269,2.2127523,2.2127523,6.2532296,10.815732,9.983229,2.2261374,7.387268,2.2127523,2.2127523,5.782635,2.2127523,7.0189657,7.6789007,5.390525,6.0054035,2.2127523,4.9041004,2.2127523,2.2127523,2.2127523,6.167698,13.059275,13.325296,10.362686,2.8411405,3.562203,8.937332,7.1909537,12.026637,3.6770804,4.2465944,8.951695,3.9679549,10.943021,8.0432625,14.159954,7.472299,12.7267065,7.690215,3.414714,2.2297735,2.972055,2.787539,6.0652833,2.2127523,7.0379467,6.1721153,2.2127523,7.5085053,11.382885,10.968349,5.4275026,7.8456087,6.65689,8.455204,5.152298,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2272274,2.2127523,2.2127523,2.6890905,2.2127523,2.2127523,2.2127523,2.2127523,3.5308642,2.2127523,2.517455,2.2246609,3.0330667,8.575008,7.1918144,4.835288,5.7184258,5.702144,2.798545,5.4175453,2.9389615,6.387233,7.0581117,2.6341097,8.663801,2.2308543,2.2127523,2.2127523,2.2127523,2.2127523,6.3536205,2.2127523,4.2628803,5.490421,3.4922898,8.959752,9.033584,2.2127523,7.8138766,2.2941914,11.887214,11.114739,3.3404567,2.5815969,3.9994235,6.2796063,4.6579623,3.604774,10.571404,8.822781,8.656035,6.744087,2.6726606,9.859307,5.8691273,4.9316974,6.0382247,5.518787,2.3910966,8.837061,10.521956,9.399175,2.2127523,8.834922,3.8087015,7.5097594,9.394451,2.2648993,6.4447045,5.9173,8.400584,7.5922284,8.1304245,2.2127523,2.2127523,8.237645,7.623145,2.2127523,5.950223,3.2381275,11.368367,9.949286,9.209641,12.3229265,6.452147,6.4143567,5.571526,11.74643,12.300223,5.9314666,7.6527333,12.350514,8.321497,8.16518,10.9381,10.758327,5.948449,10.615336,8.341599,10.644358,9.107685,12.838225,3.107931,12.963475,13.010883,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.216286,2.2127523,2.2127523,6.0767937,9.340572,2.2127523,2.2127523,5.5091033,10.00704,10.793339,2.2127523,2.2127523,2.2127523,3.4648142,2.2127523,7.6853185,8.694857,3.6473014,3.2600594,5.917513,10.467509,6.587694,3.980897,5.6054616,7.4175487,5.935709,8.613109,6.148208,6.855014,4.8275123,8.058796,9.7265415,2.5021229,4.084601,5.0042887,4.855457,2.783194,2.2127523,2.2127523,4.878491,2.6933532,8.685501,2.6332266,2.2127523,7.232749,9.555943,9.574218,4.8522816,7.410018,9.201703,2.2127523,2.2127523,2.2127523,4.837969,7.6777596,2.2584345,7.005926,6.6645937,6.3070927,2.4686835,5.0829253,10.38519,2.677915,2.2127523,2.946257,2.272237,2.2127523,2.215629,8.939465,14.017757,12.214955,7.4078293,11.881917,12.577078,12.702413,8.34657,7.800134,2.9416988,12.123485,10.910489,2.2127523,8.624162,3.8888078,5.9510236,6.2963643,2.2214212,2.2127523,2.254365,3.9313583,2.2127523,2.2127523,3.1000144,4.5247626,3.4314868,2.2127523,7.698665,4.6336255,3.1489773,2.8263717,2.344424,7.58738,7.4953938,6.3382473,7.4692726,4.507121,4.6948357,4.849894,4.00512,4.461127,4.5570574,7.8491397,2.2427542,9.134632,9.787874,2.5328794,2.2403884,4.897112,2.2127523,2.2127523,2.2424471,4.476659,2.4597886,3.2434418,9.100226,2.3410926,7.4879055,7.838445,4.8036017,2.2127523,2.2127523,5.0366282,3.366809,2.2127523,6.2364917,8.302627,5.735977,3.3174915,2.2127523,8.949018,4.371987,4.5495863,3.8101716,2.2127523,2.9811766,5.521893,3.3366942,8.263371,3.3366013,5.430104,2.2127523,2.2174172,2.643946,2.2127523,3.6668637,2.2127523,2.2127523,2.8906176,2.3362224,2.2127523,4.0083327,2.2127523,2.2127523,4.231623,3.7944639,2.4315546,2.2127523,2.2127523,2.2127523,2.222661,2.2127523,6.0017943,5.185122,8.77702,9.383579,2.2127523,2.5772388,9.775635,6.8045006,4.5563006,2.2127523,2.2127523,2.2127523,2.2463124,7.185153,5.375128,2.2127523,4.4994774,8.217651,2.2127523,2.215561,4.492566,6.83186,7.761568,8.037871,9.525865,2.2127523,3.4518833,4.1273227,2.4079301,11.16263,8.639305,6.93355,8.976044,7.8869214,5.830492,6.072434,4.3953614,9.4436245,9.162175,8.373197,9.800821,6.962962,8.262613,6.38241,5.3746557,10.859022,2.2127523,2.2127523,2.2127523,4.8167,2.2246184,2.213988,3.6405635,5.1448894,5.1621923,5.9191546,6.940398,4.425797,2.2127523,2.2127523,7.166877,2.2396708,3.0373898,2.2127523,2.2127523,6.505959,7.253484,7.1833305,5.9105697,5.234431,5.79047,6.2431602,7.8846273,5.0574036,3.8423727,5.389088,4.5623927,5.717323,8.037045,4.825557,2.3384588,6.8955264,7.1667347,2.2127523,2.2127523,7.744064,8.415357,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2185733,5.4541903,2.2127523,4.362107,4.8377604,8.985495,8.205806,5.5823655,4.0018516,2.2127523,6.279294,5.2056055,4.1075478,4.893737,9.603756,4.6583686,5.761443,3.9362032,6.3767767,7.4096427,2.4442163,5.4344788,3.876908,4.952136,5.7108912,8.189204,7.837988,3.5446744,10.987812,6.668578,6.016112,2.2481084,5.0925617,3.720209,4.6108923,2.5079412,7.59753,4.6621046,2.663695,4.5405436,2.2127523,2.2127523,2.230514,2.2715414,2.2127523,2.2127523,2.2127523,3.6918204,2.2366412,3.8390908,7.1555376,2.786006,4.7785625,11.0589485,5.745182,2.7926047,6.128225,2.8913817,2.5896344,2.235777,6.988856,6.0912004,3.2363412,3.7779973,6.4699087,7.023995,8.941835,8.658777,3.3363504,9.207869,11.845565,9.153581,3.6072502,4.8078837,5.414093,9.168757,9.597306,8.742294,2.2127523,2.2127523,2.2127523,5.186061,7.6230454,7.508942,2.2167413,5.7663665,4.5854554,12.733214,2.2142692,9.701429,9.355257,8.568406,12.466721,11.970093,8.875266,6.5736814,9.175608,2.2127523,5.5421815,2.2681475,4.2670617,2.2127523,2.2482994,9.403743,6.1129627,2.2127523,7.5377326,2.2127523,6.4912076,5.6248894,3.9571512,6.1098213,6.740051,5.096755,6.1487913,5.1898723,2.5720103,3.3520572,8.298673,2.2774456,9.984256,7.3222966,7.1758995,7.9532337,3.008231,3.902152,4.12486,2.2143164,2.2127523,9.966316,7.9372993,9.28914,13.089498,3.9494927,6.1939273,5.6116877,2.4035134,8.608932,7.6181602,2.5636458,8.272964,5.273682,5.435147,7.664671,5.971263,3.8761663,6.218721,2.520945,2.243843,4.214973,2.4684896,4.7289634,2.2127523,7.683147,2.2127523,7.96771,13.919452,8.914058,9.063409,5.169663,2.3379753,5.7369366,5.486389,3.0442877,5.2200904,6.779867,2.3333573,6.1307125,6.074711,6.0985,2.2127523,6.658254,3.8426757,6.8734107,6.5673685,5.3665137,8.502706,7.8126106,2.9692266,2.2127523,2.2127523,3.6946018,2.2127523,6.9281583,5.640548,7.032453,5.6040835,5.9981074,9.750265,7.2344365,7.2049856,3.2243023,2.4518125,3.1631057,2.2127523,2.2208471,4.2161503,3.4984555,5.945546,2.2127523,6.8469257,2.2127523,4.596831,4.9446583,7.4008036,6.530945,3.529784,3.196606,6.160903,4.722873,4.1847115,5.991329,6.9691014,11.005856,5.0809746,4.3071012,2.214535,2.2128403,8.344033,4.7898946,5.1173224,2.278395,12.77163,9.090419,8.779574,6.1884007,6.5567446,2.2127523,4.988954,2.2127523,2.683561,4.8752007,5.5109034,2.8782296,2.2127523,9.927783,4.8625507,5.6650114,8.567653,3.9188378,11.955931,7.600823,10.094247,7.6185536,5.7246895,6.750708,8.03111,6.619069,6.683675,5.757131,2.6824687,4.6406636,6.06583,2.7943423,4.299143,2.2127523,2.2127523,2.2127523,2.2127523,6.4478636,2.2127523,6.056679,4.66201,8.255533,5.9749107,6.43122,6.166242,6.4774027,6.110752,6.0786877,2.2127523,2.2127523,2.2130291,4.7737894,2.2127523,3.1806076,2.2127523,2.2127523,3.1593695,4.1602163,2.2127523,2.2127523,2.2127523,2.2127523,2.2132187,2.4733796,2.7361948,2.2178257,2.2149875,2.6671913,2.2127523,2.2127523,2.2127523,4.8008447,3.078937,4.8204765,2.2127523,2.2127523,2.218517,2.2127523,2.2205546,2.5465715,2.2451031,2.2127523,2.2127523,2.7334716,2.2127523,2.2127523,2.2127523,2.2691157,5.4134564,2.2371242,2.2127523,2.3125632,2.2127523,2.2127523,2.227469,2.2149272,2.260633,5.4130816,2.2231686,7.8703313,2.251356,2.2127523,2.2127523,3.5393255,2.2459743,4.3608174,2.21604,5.5209723,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.017646,2.2127523,2.830895,2.2127523,7.87919,6.893877,2.2127523,2.2127523,4.070268,2.6083956,2.2189019,2.3778377,3.9815307,2.2127523,4.389086,3.307155,7.763845,5.229904,2.2471418,2.2278562,2.685634,2.2127523,4.276748,2.5421164,3.4356325,2.792696,3.5144086,2.2127523,8.685832,2.2129617,2.3231964,2.2127523,2.2127523,2.2127523,2.2152517,3.2803617,8.554112,5.2599344,11.558214,4.664615,5.4293275,3.6839862,2.2132564,2.7594688,5.2288055,2.2127523,7.2165284,8.809159,6.209048,7.3433495,2.2127523,5.974524,5.564177,5.2315326,8.939136,5.7046437,6.8172216,8.729945,11.096381,8.326854,4.584227,7.254419,4.6908875,8.234976,8.461447,5.8661623,8.578631,6.242085,4.5585237,5.816285,2.4480376,6.804118,6.5083613,5.56257,2.4303992,6.41351,5.0046997,5.4188485,3.046093,9.596983,6.675169,8.366742,2.2127523,2.2127523,2.2243543,2.69418,2.2127523,2.2127523,8.153905,2.451843,4.451819,3.0757277,8.597176,4.403711,5.175016,5.728267,2.2127523,6.6844077,8.138385,2.9225843,5.5731,4.15407,8.04732,3.8024695,5.583683,7.598011,9.257083,2.2280364,7.90329,7.4071507,8.577646,8.0570755,2.2224195,6.6114964,3.907724,2.2127523,4.2518077,5.9669356,4.325933,8.579454,5.0936,2.2623909,6.724887,3.197856,9.022104,7.1089134,6.5865655,2.216127,9.526612,2.213183,4.126969,4.13715,9.397994,7.464644,3.3790867,11.016001,2.2127523,2.7171454,4.241089,4.4430747,4.5097265,2.2170959,8.453094,3.4087107,4.575937,3.4500167,4.047085,6.1713147,5.9061103,5.0575633,5.431743,2.272129,4.632909,2.635874,7.5732713,7.7987156,2.23711,2.8803687,2.2127523,8.844556,6.3186855,9.218489,2.3084178,2.2127523,3.14393,3.848808,2.2127523,2.2811158,2.2127523,4.2503405,2.365611,8.243906,5.7520976,2.3188992,2.3698401,7.131381,4.5058465,2.2127523,3.8882527,5.7299843,6.743601,4.7025537,2.7606583,5.291087,5.252118,6.1885734,2.6596534,4.985382,5.7060127,4.6039896,2.9238713,6.7135158,2.3742175,5.56661,5.9377103,2.2170181,2.2147913,5.9545264,6.6416864,4.9731107,6.012372,6.766714,4.8266797,5.8758135,3.1789498,2.4811137,2.3394904,2.2155073,3.3999717,4.472624,2.215585,6.7611704,2.2127523,3.3523045,6.16334,7.460623,2.9759161,3.976048,2.2263327,9.8318205,2.2407372,2.3614774,2.2127523,5.6741033,3.381216,8.190442,2.692492,5.8776746,4.6054025,4.9103637,2.4556377,2.2174103,2.221494,2.2127523,2.3817883,2.2258751,2.2127523,2.2127523,2.2127523,2.2127523,2.214989,2.2127523,2.2127523,2.2401266,3.890442,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,9.434868,7.6181517,6.4148526,5.794259,2.7418158,2.823299,6.2164407,6.3123364,2.5843163,4.4940224,2.2127523,10.270651,6.6552863,4.2663965,4.386323,2.2127523,3.8908513,3.6171525,2.2127523,5.7670536,6.558023,4.766034,7.1210837,9.564346,8.7150955,6.034233,4.8177624,5.163,4.745236,7.621694,12.104605,7.6476583,3.8381586,3.717021,4.245016,2.2127523,7.8113227,5.7695446,2.2127523,2.217106,2.2127523,8.060068,2.878024,2.2479296,7.4365773,2.3603404,3.2228172,9.721438,7.0254474,6.9848166,4.0805945,4.498935,8.37711,2.2719471,5.1141386,7.0316677,2.7961853,3.201086,2.2640352,2.2127523,2.2367246,2.2127523,2.3169281,7.5949335,2.240284,2.2127523,2.253219,6.460974,7.3986926,5.7972727,7.931822,7.4785094,6.058567,2.350428,2.2127523,2.712547,2.441975,2.3769226,2.2896898,2.2394564,2.2208307,3.7192278,2.2168477,2.2127523,2.2127523,3.163354,2.5147567,2.2175925,2.2127523,5.595915,7.0153785,4.6473513,2.7580163,2.2127523,6.0727882,4.311135,3.869876,2.4396,3.7453828,5.495113,5.273541,3.652606,6.5831103,5.6628566,9.950539,2.2127523,2.2127523,2.6211898,2.2127523,6.6534123,6.4877963,7.1490526,6.9541397,11.864813,8.801946,7.5182624,7.158304,6.4200277,10.157335,5.638439,7.0982814,5.5772133,4.920021,4.318884,2.2127523,6.202663,9.317303,3.0416834,2.2127523,5.4272575,7.4365616,2.3873627,6.2014723,2.2127523,10.379435,2.2127523,2.2127523,2.2127523,7.677809,2.2188225,2.6420038,13.100058,2.3417323,2.2127523,2.2127523,6.2641335,2.2127523,2.2127523,2.2174506,2.2127523,2.2127523,2.2127523,2.3816168,2.2127523,2.2127523,2.2127523,2.2127523,2.2128541,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.3161054,2.2127523,2.2139876,2.7494159,2.7670403,3.9504383,2.2127523,2.2127523,2.2130685,2.2757967,2.2127523,2.2187862,2.2127523,2.8364704,2.298734,10.846585,7.677494,3.0653882,2.2127523,2.2127523,2.2127523,2.2127523,4.059222,2.6805477,3.5178442,2.3004413,2.2127523,2.2127523,2.2127523,2.2127523,2.2351432,2.2127523,2.2127523,2.2129054,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.21936,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2747912,2.2127523,2.3406358,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,7.773468,6.5263762,2.2127523,3.2233295,2.2127523,5.144225,11.522101,4.252351,2.2143867,5.649401,2.232052,2.4371064,4.656821,4.1494956,2.845335,6.0066414,4.550899,3.367894,2.2805285,2.2127523,2.8395197,2.2127523,2.2127523,2.2127523,2.2127523,4.9496913,2.2169504,2.2127523,8.319232,10.241591,6.860813,2.6525357,8.326058,7.693711,4.3527822,2.2127523,2.2127523,4.6841526,2.7776449,4.0785494,9.077196,2.2127523,8.284559,6.718996,2.2127523,9.397217,2.2127523,8.187134,7.723566,2.8685956,9.8416395,10.066175,12.020014,3.2016468,9.835411,9.909317,8.696726,8.737244,10.872919,2.2127523,6.784668,4.7259808,2.3145313,13.101025,13.255225,12.29663,11.877938,2.5484977,2.2127523,7.361207,4.211258,7.4892783,5.6064563,2.6829154,6.671767,8.795499,6.5681977,4.973979,11.505625,6.526399,5.369398,3.845159,5.8548656,7.5724235,2.8056939,8.618172,2.2127523,9.62509,2.2127523,2.2147765,4.436183,3.793931,2.3030035,3.5719428,2.2127523,2.2127523,2.2283933,2.2127523,7.8840175,8.504966,8.920947,5.1098003,2.2127523,2.253497,2.4694424,2.2127523,8.034306,2.2571485,2.2137582,2.2127523,9.87185,7.4575305,10.146435,5.5125704,2.2135854,2.2127523,5.793047,3.6518395,2.2998989,13.989071,2.2127523,2.2127523,2.2127523,14.487426,2.2852213,2.2197433,5.4099364,5.920219,5.0036097,2.2127523,6.1003494,8.278886,8.45429,4.844283,2.2127523,2.2127523,4.173204,2.2232168,3.2175932,2.2848868,2.2127523,8.309792,2.2796452,4.1773663,10.200458,2.2226937,5.7898355,6.7759852,3.3758702,7.6355267,2.2129138,3.5974257,6.398046,3.4539032,2.5593503,6.6657705,8.566677,9.054503,4.0257936,2.7857091,10.006062,5.8197093,13.452544,2.2127523,2.2127523,2.2127523,5.5274935,10.435127,12.790984,2.419893,2.2127523,8.699858,2.7185194,8.466945,6.156734,2.2127523,2.3337119,2.6785493,2.5695162,10.35269,5.0958843,2.2127523,4.546399,2.2127523,2.2127523,5.5677567,2.2127523,9.468024,2.2127523,2.2641335,3.5342577,3.1927419,2.2127523,5.8532515,2.2127523,2.2127523,2.2137048,2.2275908,5.4392624,2.2127523,3.3495638,2.2127523,2.213152,6.8719106,6.180328,6.905058,2.2127523,4.529549,5.0189033,7.9077363,2.285828,4.1670346,2.2127523,3.9013767,2.456349,5.6166096,6.353363,5.9795427,7.1593003,4.50781,5.7119465,2.2127523,2.2127523,7.667262,9.85436,4.6667676,2.5364716,4.4244556,6.315596,2.8564756,5.7080693,6.714329,4.430264,3.4503026,2.5170069,2.2127523,2.2127523,5.064216,5.860491,2.3130352,2.453757,3.854159,9.804575,3.0806355,5.462111,2.2127523,2.2127523,2.2131245,2.2127523,2.2127523,4.576651,3.0161693,6.879712,2.2127523,3.1729445,2.2127523,2.2127523,2.4346418,7.663745,6.494072,3.18086,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.504405,2.34252,2.2189934,2.483018,2.4798083,2.6738484,2.9685004,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6652613,2.2127523,2.2127523,2.2127523,4.538064,3.303408,5.9558372,2.2127523,2.519921,2.2127523,2.2692907,2.2127523,2.2269478,2.7982187,2.2127523,2.2127523,2.2127523,2.2127523,6.936409,2.2127523,2.2127523,2.2127523,2.2134113,2.2192993,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.7662308,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2187743,2.2127523,6.5131383,2.2401123,2.2127523,2.2127523,2.2802737,2.2127523,2.2127523,5.326378,6.6841044,7.6627507,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2565997,2.2147973,2.213441,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2156336,2.2127523,2.5207784,2.2127523,2.2127523,2.2127523,8.462309,2.2127523,2.2127523,2.4605715,2.2127523,2.2127523,2.6010618,2.831895,2.2127523,3.3299594,2.2127523,3.427106,2.4410203,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,10.572037,2.2127523,3.2171295,2.2127523,3.059828,2.2127523,2.7013483,2.2127523,2.2249923,2.2127523,2.2127523,2.2135386,5.5134125,2.2127523,2.2553763,2.2127523,2.490769,2.2265632,3.190397,2.2175014,2.2127523,2.2127523,2.2127523,2.238792,2.2811346,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2149503,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.495847,2.2127523,2.260497,2.2127523,2.2127523,2.2321932,2.2127523,2.3835127,4.8530073,2.2127523,2.3296335,2.2127523,2.2127523,2.2127523,2.2564242,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,8.461544,2.3831246,9.951255,2.2127523,2.2127523,2.2127523,2.5277145,2.212982,2.2127523,2.2127523,2.213396,2.2432098,2.2127523,2.2149463,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,10.513012,2.2127523,2.2127523,2.2129297,4.336558,3.3689334,2.2127523,3.4650004,6.2360077,2.2127523,2.2127523,2.2127523,2.8993,2.2127523,8.106542,2.2315881,2.2127523,2.2127523,2.370092,2.2127523,2.3147812,2.2127523,2.2588813,2.2127523,2.2127523,2.2127523,2.7109559,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.377397,2.2127523,2.2127523,2.2127523,2.2127523,7.157159,8.295196,2.2131705,2.2127523,2.2127523,2.2127523,3.4593077,2.2127523,2.2127523,6.917193,2.2127523,2.3434758,4.6343718,2.2127523,2.2127523,2.2127523,2.2176483,2.2192402,2.2127523,4.2085795,2.2127523,2.501907,2.2127523,7.086208,2.2127523,2.2127523,3.0295365,2.927,2.2127523,2.2127523,3.44628,2.2531266,2.2127523,2.2127523,2.2127523,2.3159473,2.2362328,7.2650213,2.2127523,2.2127523,2.2127523,2.2127523,2.785352,4.248399,2.3937542,2.2127523,2.213401,2.2697039,3.6013777,2.2127523,2.2127523,2.2127523,2.2135396,2.2127523,2.2127523,2.2127523,5.494345,2.7885492,5.1218696,2.2127523,2.309649,2.2127523,3.1313262,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.8797083,2.2127523,4.600705,3.5249655,3.9763987,6.5850463,2.2127523,2.79253,2.2672937,3.0841422,7.177621,2.2127523,3.0162525,3.2053256,2.755001,2.2127523,3.195072,3.1363118,9.951128,8.371169,2.2233417,9.323405,3.1303015,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6114082,2.3980036,2.2127523,2.2127523,2.2127523,2.473067,2.2127523,2.2315154,2.2127523,2.2127523,7.6909037,2.2127523,2.2273452,2.2127523,2.2127523,2.2318037,2.6382523,2.2127523,2.2127523,6.791129,2.2127523,2.2127523,2.2127523,2.2127523,2.2553904,2.2127523,2.2133234,2.2127523,2.2128062,3.618849,2.2127523,2.2127523,2.2127523,2.225772,2.8273182,2.2127523,2.2641852,2.2127523,2.2173147,2.2127523,2.361215,2.391068,2.2127523,3.3501475,2.2131882,2.6287684,2.267265,3.9610202,3.2131243,2.3544223,3.806648,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2509227,2.2127523,2.2127523,4.834679,2.2127523,2.2127523,2.2127523,2.2127523,2.2147124,2.2127523,2.2127523,2.2127523,2.2127523,3.4874542,2.2127523,2.22536,2.2127523,2.2127523,2.2127523,2.4771132,2.2132452,2.2127523,2.2295666,2.2127523,2.2127523,2.345368,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2788765,2.2127523,2.638128,2.2127523,2.2331228,2.5004706,2.2127523,2.2127523,3.8990324,7.070356,2.2127523,2.2127523,2.2127523,2.2127523,5.701405,2.2127523,2.2127523,2.2127523,2.426909,2.2148805,2.7768905,5.6602006,2.2127523,2.2127523,2.27992,2.2815135,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.796297,3.613844,2.2127523,2.2138703,2.2127523,2.2127523,2.2127523,2.2127523,2.4310846,2.2127523,2.2127523,2.2127523,2.2127523,2.3733225,2.2127523,7.827267,2.2127523,2.2127523,2.3329616,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.6272752,2.2127523,3.971911,6.686503,7.6171308,2.2132401,2.6569948,3.2916226,2.2127523,2.2425952,7.102518,2.2127523,2.2127523,8.018007,8.262924,2.2127523,6.853931,2.2127523,2.2127523,2.2127523,5.0605483,5.300013,4.786469,7.7008433,2.2127523,5.737375,9.584596,2.2127523,5.8383884,7.3454175,4.913184,6.7447395,7.850884,5.5395064,4.3288255,3.2444494,6.985313,4.784756,7.7637963,7.1486106,2.6909297,2.2127523,4.3402934,2.2127523,5.1736274,7.629856,4.394201,6.25153,2.2127523,3.5443823,2.2127523,2.2143462,3.8272798,2.24228,2.2127523,2.2127523,4.8103766,6.0136213,5.085295,9.550772,5.5138397,2.2127523,2.213832,2.2127523,2.2127523,2.2127523,3.3589604,2.2127523,2.2526164,2.8459003,2.2127523,2.2127523,2.2127523,2.3496902,2.248098,2.2127523,2.2127523,4.0825925,2.6888688,2.2178056,2.2127523,2.259918,3.8460913,2.2999158,5.8542776,2.2127523,2.2127523,2.2127523,2.246807,5.175429,2.2127523,2.2127523,4.073091,3.487889,2.2132351,2.2626183,5.8922186,4.249388,5.903454,2.2127523,2.2127523,2.2234347,2.5468166,2.2127523,2.2127523,7.9464035,6.8434277,2.2127523,2.2127523,2.6134765,2.2127523,2.5900586,2.2127523,2.2127523,2.2127523,8.768537,2.2139,2.2628515,2.2127523,2.2127523,2.2127523,2.8152196,2.2127523,2.2127523,2.7610934,2.2204866,2.4266822,5.778278,2.2127523,2.2127523,2.2127523,2.2136018,2.4127896,2.2127523,2.2127523,2.2127523,3.6332936,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.209898,2.2127523,2.283246,2.2127523,4.848112,2.2127523,4.197614,2.3225818,2.2127523,2.2127523,7.0539155,2.2127523,2.2127523,2.2127523,2.2127523,2.2434173,2.2127748,7.1916056,6.677251,2.296447,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.336045,2.2127523,2.2127523,2.5104082,2.2127523,2.2127523,4.3131175,2.2127523,2.2127523,7.1880836,2.4987347,2.2172487,2.2127523,2.328274,2.2127523,4.7105565,3.5626001,2.2127523,4.815723,2.3030484,3.7530391,3.2894156,2.219045,2.2127523,2.372546,2.2127523,2.2127523,3.537718,2.2127523,4.93852,5.552034,4.8349833,3.5760396,10.141886,7.014872,2.2127523,2.2127523,2.2127523,2.4544883,2.2127523,2.2127523,10.611944,3.055569,2.2127523,2.2127523,2.3884673,4.1271577,2.6946886,2.2127523,2.2127523,2.2127523,2.2127523,6.110911,3.7998912,5.419509,2.2127523,2.2127523,2.468975,2.2127523,2.2127523,6.906971,2.2881937,4.600328,2.2371893,2.2127523,2.2127523,2.2127523,2.9822993,2.2332423,5.446935,2.2127523,8.773517,2.2127523,2.257741,2.4116356,3.237616,2.2127523,3.0999706,4.8368363,2.2127523,2.2561316,2.531874,2.214159,2.2584186,2.2127523,6.9972067,6.2298517,4.360931,2.2281773,6.637849,4.1836667,2.2127523,2.4229994,6.67281,3.5854177,2.259661,4.2657757,2.5762188,2.4016142,4.776329,2.2133949,6.148355,2.2197514,2.2127523,2.2127523,2.2127523,2.2127523,2.513265,2.2127523,3.9465153,2.2843874,5.556099,2.2127523,2.2127523,2.2127523,4.823861,3.9545572,2.282729,2.2127523,4.1659255,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2178526,2.2127523,2.2127523,2.2127523,7.7454333,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,6.4579535,2.2127523,2.2127523,2.2127523,2.378309,2.2208595,2.2127523,3.6641178,2.2127523,2.2127523,2.2127523,8.175146,2.3128579,2.247274,4.7422233,2.2127523,2.2127523,2.2127523,3.797405,2.2127523,2.2424269,2.2127523,2.2127523,7.711574,2.2127523,2.2170575,7.9934545,4.3546176,2.2127523,2.2127523,2.2127523,6.1110387,2.2127523,2.4185307,2.213541,2.3345423,2.2127523,2.2127523,2.3612437,2.2127523,2.2127523,2.2127523,13.116466,2.3035192,2.2127523,2.2245142,2.2127523,3.951432,2.2127523,2.2534106,3.6369517,3.7476895,2.3895066,2.2127523,6.0083356,2.2127523,2.2147417,2.395808,2.2127523,2.2127523,2.2313342,3.4386,2.2127523,2.2303782,2.2127523,2.2127523,2.2148044,2.297014,4.5176625,2.4081295,2.5686157,2.746719,2.83596,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.7921433,7.6766596,2.2127523,4.9934816,3.375358,2.2127523,3.206996,3.9590652,2.284811,6.9192634,2.2127523,2.2127523,2.2127523,2.2127523,2.6211407,2.2127523,3.0505886,2.4291496,3.2994664,2.2127523,2.5024664,2.2260184,2.2127523,2.3223522,2.2127523,2.2139478,4.4635415,2.2127523,2.2127523,2.2127523,4.776747,4.120513,2.2127523,2.2127523,4.479494,2.2127523,7.8090634,2.2127523,2.9350283,2.2127523,2.2150056,3.3576121,2.2127523,2.2127523,2.2127523,2.2127523,2.402026,2.2127523,2.2127523,7.788512,2.2127523,2.2296233,2.2127523,2.2127523,2.4539583,3.1588535,3.3854783,2.2160506,2.2127523,2.2236567,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.5326464,2.2127523,2.3310883,2.2127523,2.2127523,2.2127523,3.3394346,2.2127523,2.2127523,2.3521445,2.2127523,3.5899117,2.7968202,4.265127,2.2127523,2.2144477,3.5402734,2.3418236,2.2127523,4.4858456,2.2127523,3.123933,2.368838,2.9967444,2.2127523,2.2127523,2.6945932,8.20771,2.2127523,2.2127523,2.593411,2.286968,5.3525205,2.2127523,2.2146199,2.2237737,2.2127523,14.705486,2.2127523,2.2127523,6.66264,2.2127523,2.2127523,3.018303,2.5068178,2.298002,2.2127523,2.997356,2.337144,6.546074,2.2127523,2.2127523,4.721738,2.2355118,2.219479,2.2127523,2.251001,5.2793536,2.2127523,2.7891617,6.5842614,2.2161818,3.374115,2.2127523,2.2773566,2.2127523,2.2207277,2.2127523,4.7837806,2.2415214,2.2127523,2.3390825,4.2066383,8.34197,2.2127523,2.2127523,3.7907088,2.4574654,2.2127523,2.2127523,2.2127523,6.609409,2.2127523,6.5018473,2.2127523,2.2285254,3.1095185,2.2127523,10.6186905,7.596569,5.302207,4.0751867,9.067663,14.006165,3.9984186,2.2127523,3.1515372,2.2380896,7.121405,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2181945,3.2290883,4.9963207,2.2127523,5.228677,9.041417,8.03383,11.802197,9.550052,2.2127523,2.3931353,8.434809,2.2233436,2.2127523,2.2127523,3.2798507,6.8765163,2.2127523,4.4291615,2.2493522,4.8921804,5.613033,9.659008,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6934733,2.2127523,2.2669342,2.2127523,2.2127523,4.8788075,2.2127523,2.21417,2.2127523,2.2161856,2.2139587,4.892097,7.16353,2.2127523,2.2127523,3.217497,3.9280438,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.0148103,2.2127523,5.042516,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.225934,2.7855482,2.2127523,2.2127523,2.2127523,2.2127523,2.7159133,2.2412093,2.2127523,2.221401,2.2127523,2.230665,2.2127523,2.2127523,2.2127523,2.2127523,2.2169664,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2142878,2.2127523,2.2127523,3.9157948,2.2127523,4.4506497,2.5409439,3.8349943,2.8214004,2.2127523,2.2127523,4.316454,2.2252479,2.2127523,6.167848,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6031926,2.2127523,2.2127523,2.2127523,2.2127523,2.27941,2.584066,3.7255666,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.4109857,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.583378,3.5386782,2.2127523,2.2127523,7.000534,4.5329638,2.2127523,2.2127523,2.2127523,2.2127523,2.2332199,2.2127523,4.7666955,2.2225018,2.2127523,2.2127523,2.2135475,6.589997,3.8195083,2.2127523,2.2794,2.2127523,3.5921493,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2971048,2.2274485,2.2127523,2.2220118,2.2127523,2.312299,2.2127523,2.6244059,2.2127523,2.2127523,2.2662866,2.2392383,2.2127523,2.2127523,2.2127523,2.5540664,2.2127523,2.2127523,2.2127523,2.9305682,2.2564762,2.2127523,2.2127523,2.8457406,4.84056,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.284705,2.2291963,2.2127523,2.2127523,2.2127523,2.2127523,4.269353,6.5786247,2.2127523,2.2127523,2.2509444,2.2127523,2.2127523,2.2127523,6.424304,2.2127523,2.2141633,2.2281702,3.6387339,2.2188044,2.2127523,2.2127523,2.2127523,2.2127523,5.765538,2.2127523,2.2127523,2.2127523,2.2595758,2.2127523,2.2127523,2.2127523,2.2807672,2.3030353,2.2130523,4.3494744,4.954482,2.2193415,2.2127523,2.2127523,2.2127523,2.2127523,2.8813522,2.2127523,2.8837795,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.451243,2.2127523,2.2127523,2.2127523,2.2127523,5.2832823,5.053336,2.2127523,2.6818871,2.2127523,2.5375195,2.2127523,2.2127523,2.2127523,2.8801637,2.2127523,2.2127523,5.5923266,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2251394,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.1607075,9.660857,2.2128968,2.2271068,2.41156,2.2127523,2.2127523,2.2127523,3.2201152,2.5253508,2.2127523,4.124165,2.2127523,2.2127523,3.1604362,2.2937882,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,6.7324777,2.2127523,2.535924,2.2127523,3.1433756,2.2127523,2.2127523,2.2127523,2.396865,2.2127523,10.72836,2.2127523,2.2289217,2.2127523,2.3491645,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2256367,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.7030284,2.2127523,2.2127523,2.22484,3.1117227,2.2127523,2.2127523,2.2207744,2.2127523,2.2127523,2.2127523,2.2488523,2.2127523,2.249385,2.2127523,2.2127523,2.2127523,2.4459953,2.2127523,3.3981977,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.3120668,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.8183894,6.801629,2.2727787,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.200218,2.2127523,2.2127523,2.2328124,2.8786416,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.6953475,2.2127523,2.2127523,2.2127523,2.2127523,2.2619383,2.326251,2.2127523,2.2127523,2.2127523,2.2386591,2.2127523,2.2127523,2.2127523,2.2300603,2.2127523,2.4401062,6.8963695,2.2127523,2.2336204,3.70128,2.3406696,2.2127523,2.2127523,5.9908023,2.2127523,7.434793,8.02172,2.2127523,2.35622,2.2577455,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.421597,9.528493,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.220681,2.2127523,2.2127523,2.2127523,3.8943672,2.5395272,2.2171128,2.2127523,2.2127523,2.2261138,2.2127523,8.390835,2.2127523,2.2127523,5.5940943,2.2127523,2.2127523,2.2149734,2.2127523,2.2127523,2.363734,2.213608,5.1491094,2.2127523,2.2127523,3.2475958,2.2127523,2.2127523,2.362271,2.2127523,2.2127523,2.2127523,15.265955,13.288642,2.2127523,2.2127523,2.359045,2.2229018,2.2250197,2.2127523,2.2152317,2.2127523,6.641928,2.2127523,2.2127523,2.2385051,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2265787,2.2127523,2.2127523,9.864587,2.2127523,2.2127523,9.154294,2.2127523,2.2127523,2.2127523,4.3529487,2.2127523,2.2127523,4.9334693,2.2127523,2.3621552,2.2157705,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2401788,6.8472924,2.2127523,2.2127523,2.2127523,2.372611,13.49771,2.2127523,4.994479,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.6217957,2.2156343,2.2127523,2.2127523,2.2127523,3.174846,2.2127523,2.2127523,2.2812355,2.2127523,2.2127523,2.2127523,7.285937,2.2127523,2.214809,2.2127523,2.2127523,2.3354027,2.2127523,2.2127523,4.3371577,2.2127523,5.2917733,2.2127523,5.9154444,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,6.794593,2.2127523,5.6376643,2.2155926,2.2127523,2.252051,2.2127523,3.9623864,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.399774,5.876912,2.2127523,2.2127523,2.2234216,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.7296803,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2310445,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.293505,2.2127523,2.2396681,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2202027,2.2127523,2.2127523,3.5663996,2.3863773,2.2127523,2.2127523,2.295386,2.2127523,2.2127523,2.2127523,2.50754,2.2252946,2.2127523,2.2279246,6.774647,2.2536557,2.2164958,2.2127523,5.7238874,2.2127523,2.2127523,2.4454627,2.2127523,2.2127523,2.2788715,2.2127523,5.03485,2.2127523,2.2127523,2.2127523,3.5402744,8.02744,7.2078967,2.2127523,6.1888022,6.1319194,7.912228,2.2148178,6.753673,6.235847,6.5824485,2.719309,5.9818325,2.2127523,2.2127523,2.2127523,2.2127523,2.390479,2.2127523,5.4477005,2.2127523,2.2127523,9.39071,9.06886,6.196714,6.44765,2.4574401,6.2877645,2.3330295,2.2127523,4.318086,2.2127523,2.213064,2.3574011,4.900363,3.200395,9.871716,5.846226,7.8077354,2.2127523,2.2127523,2.2210407,4.7559266,2.2127523,2.2127523,2.4129982,2.273301,3.259691,2.215879,2.3149352,2.308551,2.2127523,3.9167945,2.3212917,2.2127523,2.295357,2.2127523,3.565207,2.2127523,2.2127523,2.4109693,2.497665,2.2127523,2.2127523,2.2127523,2.2127523,2.5591877,2.2127523,2.2127523,2.2127523,2.2127523,2.2272046,2.2840345,2.2127523,2.2127523,4.6114826,2.2127523,2.2127523,2.2127523,2.2127523,2.2171369,2.2127523,2.2127523,2.2127523,2.2127523,2.2607715,2.3898854,2.2127523,7.4050746,2.2127523,2.2127523,3.1962304,5.172145,2.6505442,2.2127523,2.315832,6.8275285,2.2127523,8.410629,8.7268,5.549805,3.4268363,4.514899,2.8145547,2.2195256,8.168133,5.9644246,9.937603,3.3069742,2.2559717,2.2745435,7.845451,2.4162822,2.2127523,2.2127523,2.2127523,3.6075537,2.2127523,6.1512947,2.2127523,3.0344877,2.2127523,2.2127523,4.458244,2.216154,5.399693,6.7937694,2.2127523,4.157982,2.348707,6.3605742,2.2648978,2.2127523,10.594444,2.2127523,2.2127523,2.2181714,4.5284295,2.2127523,2.2894895,2.5786772,2.2127523,6.894079,2.5983326,2.2312152,5.033704,2.3575654,2.2127523,2.2127523,11.132943,5.331907,2.3901553,7.244751,2.2127523,2.2153919,2.2127523,2.2127523,2.3092015,2.2127523,2.4821846,2.30183,2.2286747,2.29214,4.818049,5.5358377,2.2127523,2.3380182,2.2531104,2.2127523,2.2127523,4.039378,2.3395455,2.2127523,2.2127523,2.2127523,2.2127523,3.1780696,2.2181096,2.2127523,7.060521,2.2127523,2.2127523,2.2127523,2.2263274,2.2127523,2.2127523,2.2127523,6.9890857,2.2127523,2.2127523,2.2127523,2.231766,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,7.0588346,2.8577218,5.528265,6.1755786,2.2952514,2.2127523,4.6812024,2.2364647,3.205491,2.2127523,2.2127523,4.772385,2.2127523,2.2127523,2.2263758,4.178001,2.2467492,7.780399,2.2127523,2.2127523,2.2127523,2.2478163,7.243958,5.595837,2.2127523,2.2127523,2.2988682,7.648028,2.2127523,2.792516,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,7.87268,5.4333053,2.2127523,2.2127523,4.0293093,2.6516674,2.2170458,2.2127523,2.2142687,8.6778145,2.2264047,2.2127523,2.2127523,2.2127523,2.2127523,6.408092,2.2127523,2.2127523,8.02088,2.2127523,2.2127523,2.2127523,2.2127523,2.250545,2.219862,2.2127523,2.2127523,2.8423786,2.2127523,2.2127523,2.2127523,2.2127523,2.4034612,2.2166533,2.2127523,2.2127523,2.2127523,5.1219044,2.2127523,4.4673085,2.2127523,2.3820186,2.2219117,5.5358796,2.651292,4.569086,2.2130203,3.7941494,2.559949,2.2127523,2.2127523,2.2127523,2.2127523,4.619571,2.2127523,2.2127523,7.014181,4.617978,2.2127523,2.4706914,2.2207766,2.2127523,2.2127523,2.3743095,2.2127523,2.248671,2.2127523,2.2127523,2.2127523,7.382286,2.2127523,2.215871,2.2127523,2.956835,6.3507657,2.2127523,2.2127523,2.5151322,2.2127523,2.2127523,2.4004734,7.486509,2.2127523,2.3431144,2.2822883,2.2682862,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.1171107,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.9585004,2.2127523,2.2127523,2.2127523,2.2127523,2.337138,6.7721415,2.3669882,6.9836607,2.2127523,2.2127523,2.2127523,2.60185,2.2127523,2.2127523,2.2264848,2.2127523,2.2454247,6.3568716,2.2127523,2.2127523,4.705605,2.2127523,2.2127523,5.7864213,2.2221432,2.3003654,2.2226763,2.2127523,5.9429226,4.1957946,2.2127523,2.2127523,2.2127523,2.2799277,2.2127523,2.2145228,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.7635193,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.6402955,2.2127523,2.2127523,5.7511396,2.2127523,3.011959,2.2127523,2.5603907,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.537456,2.2127523,2.238684,2.4861867,2.2127523,3.965154,2.2163045,2.2388854,2.2127523,3.8162787,2.2127523,2.2682724,2.2896144,7.1796713,5.213647,6.034265,2.2127523,2.9568675,3.166776,10.293059,2.2127523,3.2191856,6.2140584,2.3673832,2.2127523,2.2925742,2.3324058,2.2329671,2.2182617,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2157114,2.2127523,2.2127523,2.2127523,2.2127523,3.1597393,2.3410873,2.2135346,2.2127523,2.2127523,2.2127523,2.2127523,6.5992513,2.2127523,2.2127523,2.2127523,2.3253999,2.3432164,2.505092,2.2127523,7.0032787,2.2127523,2.2127523,2.2127523,8.825083,2.2127523,2.2127523,3.2316723,2.4478595,2.2447093,2.2147732,9.149238,2.5931394,8.979832,2.2127523,2.287599,2.2127523,2.214771,2.2127523,3.0022159,2.2127523,2.2127523,2.2127523,2.7946956,2.2127523,6.77978,8.77287,2.2127523,2.3485804,2.543088,2.2127523,6.876359,2.2127523,5.369112,2.3304224,2.2127523,2.2127523,3.671522,2.2127523,2.2133012,2.214334,2.2127523,2.2127523,2.21797,2.2127523,2.5933108,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.377294,2.2127523,5.6473145,2.2127523,2.2127523,2.2127523,4.1788487,2.2318687,4.677711,2.2127523,2.2277677,2.2127523,2.2127523,2.2127523,2.2127523,4.336775,2.2378037,2.2127523,2.8354132,5.4283895,3.200565,2.2127523,4.916923,2.2127523,2.2127523,2.2420988,2.2177675,2.2127523,3.2584372,2.292771,5.5207105,5.9715023,2.2127523,2.2155528,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.4007752,3.9690962,2.2127523,2.2127523,5.6857443,2.2127523,2.2127523,4.843289,2.2489321,2.3100073,2.2127523,2.2127523,2.2127523,2.2274208,2.2127523,2.401738,4.8338885,6.4724855,3.110739,2.2127523,3.1558723,2.2127523,8.658578,2.2196,2.2127523,2.2127523,2.2127523,2.3031914,2.2127523,2.2154863,5.872708,2.2127523,2.243586,8.392727,5.7626143,2.2127523,2.6360986,3.5989785,3.2680502,2.2127523,2.447551,2.2127523,2.2127523,3.011542,2.2127523,2.2127523,2.2127523,2.2483978,2.2127523,2.2127523,2.2127523,2.2127523,7.5099497,2.2215853,2.2156632,2.2127523,2.2127523,3.6508324,5.2550488,4.502986,3.5509374,3.8624275,4.558483,2.831208,2.5286782,2.2234282,2.2127523,2.7368217,8.359429,2.2127523,2.2127523,5.439997,2.5733533,2.4428825,4.6992645,2.7633955,7.768704,2.2127523,2.502882,2.2127523,7.500733,5.444,3.4232786,8.783786,3.963546,7.6607018,4.900661,2.2127523,4.7185745,2.2127523,6.641432,4.360727,3.1353164,5.6995606,3.9341993,2.2127523,2.2127523,7.039504,5.0824723,2.4858851,2.2127523,4.517385,5.344486,11.547975,5.300871,4.47244,3.0632658,8.213782,5.4566483,2.6770759,6.555949,8.774979,2.2127523,9.620405,2.2127523,7.5242867,2.2442458,5.8322563,4.3713255,2.2700195,3.0511467,2.2658765,6.813923,2.2127523,7.8487916,5.442224,3.5499742,4.9710364,4.9982643,2.4256094,2.2127523,4.043902,2.5021698,2.4195354,5.9689445,2.2127523,8.809311,4.435335,4.2921863,8.893965,2.2127523,6.9180617,4.2269573,8.496322,9.5345335,4.547565,2.2127523,7.078203,2.2166178,3.1393259,2.4797769,3.80104,5.010129,6.985772,4.5019493,5.8994846,8.19612,5.361106,6.029853,3.3428793,5.569486,3.5177212,5.0006003,4.5886974,3.103879,2.716628,2.2127523,2.3399696,2.2127523,5.480517,4.595863,5.524321,3.3976607,3.6560178,2.2127523,2.370338,2.2127523,4.3014193,4.8058624,5.700845,7.259979,2.2127523,5.3556757,5.01905,4.9913907,2.2127523,2.2127523,2.2127523,2.2156804,2.9084346,2.2127523,3.0274653,8.695286,4.916737,7.7232237,2.2127523,5.839822,2.9604633,10.372618,3.6920075,3.9977648,2.2142096,2.479881,2.2127523,2.2127523,5.0508394,5.733585,7.046178,4.9599476,4.237675,4.6327224,2.2127523,2.2739747,4.64721,2.2127523,2.2156663,2.2127523,2.2191517,2.4813385,2.9839962,5.1396756,3.9076278,8.460096,4.624415,8.602179,3.4959848,6.728078,2.2127523,8.234365,4.498383,2.2127523,2.3001335,4.1944695,10.629015,5.106209,2.4281743,3.75789,8.665221,3.221486,2.2127523,2.3397622,2.2127523,4.868804,2.2127523,2.2127523,3.0884953,7.464335,3.3600886,2.2376292,7.1130714,9.612994,6.897987,3.284686,7.5893087,9.524335,2.9674785,7.6103806,6.889665,6.950888,8.664387,9.487935,11.228527,6.175547,2.2611983,2.2127523,7.541137,8.864546,5.0559072,8.179297,2.2127523,5.469232,4.3824744,2.2127523,6.117165,7.5595827,6.4874625,4.597429,3.7553115,4.9611273,8.620785,6.038325,6.269604,5.421903,7.4309864,8.717749,9.758858,10.637417,3.6009345,5.5445833,5.6754293,2.214804,2.2127523,11.460872,9.119326,2.7663527,8.967679,2.2127523,2.2127523,2.2127523,2.2487752,2.2163544,2.5009394,8.437803,8.788143,2.2127523,4.690621,10.357214,7.356964,2.5604205,2.2127523,7.2796087,9.556142,2.586977,2.2127523,2.2127523,2.2127523,2.233114,2.2127523,2.2127523,2.2127523,11.339803,6.564093,9.570196,9.54069,5.844913,6.8661532,10.726792,6.8977847,5.07493,4.907262,9.750082,9.693366,4.6302686,6.099421,3.470669,12.433311,2.2127523,2.2127523,2.225319,6.614138,4.4945383,5.962208,2.3725436,2.2127523,5.5299377,8.849859,2.2127523,9.371361,2.4493446,2.2127523,2.2127523,2.2127523,8.804945,8.44714,5.510672,4.195178,2.2127523,7.018848,5.9269047,6.9632683,7.7599206,2.4389348,5.8298583,2.2127523,5.506126,7.9134126,10.2678,7.993955,5.796596,2.9973364,5.8575745,2.226214,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.213686,2.2127523,2.2127523,2.2127523,2.2231536,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,10.691072,7.577088,3.587972,4.140274,2.2127523,7.4348,7.959091,2.2127523,4.3278656,2.2127523,3.2930143,2.2127523,3.8236978,2.2127523,6.8123856,2.7103457,9.438268,3.6495545,4.640918,9.092073,2.6698983,7.596794,7.932423,12.859956,11.975986,11.8857,2.2339554,8.0463705,7.0754547,5.1734085,2.4813175,2.3329692,8.720655,5.150132,3.5393193,4.0520506,3.339151,8.921572,4.9625797,7.727927,5.1165586,4.6645145,9.49611,6.97938,8.806826,7.9517198,4.1751556,7.621351,8.076951,5.3147836,4.765205,9.164945,9.457219,7.347046,3.178141,2.2127523,3.019518,6.396843,7.0022545,10.657175,2.2127523,5.9915743,5.639473,4.4584637,9.622866,5.3072195,7.4179544,3.9384327,3.777684,4.241973,7.2458224,3.097436,4.3112144,5.228737,7.239397,4.6027856,2.8378165,2.2127523,2.3019803,6.4252706,5.68413,7.4960876,8.417376,3.913124,7.6384625,5.492552,6.7801347,4.3957353,6.6126785,4.492675,3.9844313,9.194915,6.82368,7.9449887,3.0326543,3.220039,6.3143306,8.22481,4.4952273,3.7290392,6.7055335,6.579481,3.7153902,2.7696705,2.8085284,8.558935,7.7804637,2.2822149,9.058297,7.5373244,2.2127523,5.9501953,3.818412,5.161741,2.21503,2.6304097,3.1085463,8.057115,7.693951,7.199383,7.846542,7.7109847,6.147807,4.0936193,10.012479,5.844158,3.4076471,2.6405463,2.8739436,4.7666006,4.999194,5.9764314,2.2127523,2.2127523,7.128612,3.4120328,2.2127523,7.840506,7.4701366,6.14782,5.0684385,4.056085,8.449438,10.477083,9.379446,2.9133878,3.929494,6.801708,5.3673205,7.903872,3.064633,5.472905,7.9002323,4.086731,2.2127523,2.2149255,2.3775725,2.2130966,2.2127523,4.5546083,5.5206976,6.24123,8.120967,2.2127523,9.879612,6.713481,5.0197935,2.2756777,3.3426344,3.9574683,3.2934678,7.344988,4.5172343,7.7042475,6.914655,6.348143,3.3215277,3.45501,6.142933,6.4526057,4.4988856,4.9635944,2.2127523,2.2127523,9.774131,4.610188,6.987524,11.104914,2.2127523,4.129335,4.664658,8.684018,4.536977,6.678845,4.7173815,7.7373285,8.473006,5.2737904,4.6656675,7.642633,8.224081,4.1108766,3.5252554,2.2127523,2.2127523,2.2127523,2.2127523,5.8932934,6.3867216,5.253172,7.8446264,5.6414866,8.594337,8.956479,2.2127523,3.0259798,2.3436472,2.2127523,7.1133494,2.2127523,2.2127523,2.2127523,2.3298047,8.27052,2.966204,7.790014,8.308358,4.2150664,4.326274,4.2445555,6.4915447,6.0495563,9.0485935,4.059382,3.761543,5.9390645,2.2277093,7.293861,5.5845046,3.4137144,3.6824195,6.716131,4.502738,5.7379107,5.554492,6.919968,7.935765,5.6090665,4.6354604,7.4190254,2.781788,2.2127523,11.225425,2.4441864,9.3003025,11.38245,4.9890676,5.0054007,7.0257683,4.5011683,8.087975,2.2296636,6.1982613,8.948082,4.6696253,9.418765,5.2740617,4.399145,5.895477,5.932406,7.9614944,7.8254313,8.713483,4.8872166,2.2127523,9.533917,7.292006,4.9941916,2.2135277,7.9265304,2.2127523,6.027792,6.1981173,7.9738746,4.78296,5.639806,5.686779,4.5687375,5.054103,5.532781,5.2066126,5.541167,8.594127,6.0697513,6.0609436,8.406878,6.958442,6.3393636,7.697814,3.3028946,3.3163433,7.26788,2.9377398,6.025387,6.5431757,2.2127523,5.9848614,2.2127523,2.510168,2.2127523,2.2127523,2.2127523,2.2150373,5.710352,4.6202993,8.993194,3.681279,2.2506533,4.64269,6.951596,2.227283,9.082004,7.1352544,6.8431125,2.8669546,3.7661471,2.3534787,2.2127523,8.369461,2.2756765,4.6558323,6.2341676,6.5409594,3.4835286,4.228421,9.247263,10.0989,9.539311,3.1783497,9.246103,4.596873,8.135205,5.9112678,4.492289,2.8264585,7.9504375,7.783937,3.6192248,6.780328,8.479889,2.4320178,5.480137,6.4225283,2.2127523,2.2127523,4.7324715,10.999051,6.796841,9.65585,2.5198522,8.750328,5.265424,6.812558,2.9925532,6.502445,7.040482,5.8098745,6.858898,3.741474,2.2906353,3.8059092,3.1569881,8.930258,8.357014,6.119587,7.6742506,4.7985177,8.360094,6.8364434,8.073579,3.8068051,3.4167721,6.979931,5.0599504,4.4251204,11.637268,6.929605,8.35062,5.240527,4.75939,4.1292615,6.1150737,7.633445,8.192612,2.2660701,2.4284985,5.5901785,3.8894436,6.09991,2.245271,2.3468788,7.9374967,7.600167,3.3189957,5.34708,8.925611,2.2127523,5.662448,2.2127523,2.2611725,4.3747826,2.2355778,2.2127523,2.2127523,2.2127523,2.2127523,2.2179332,3.3174896,2.2127523,3.1576123,7.1612062,2.2127523,2.2127523,2.6195304,2.2127523,2.2127523,2.2127523,2.2127523,2.244047,2.2127523,2.2127523,2.5881054,2.2127523,2.2163548,2.3364153,8.426257,8.431601,9.110829,9.488927,5.9334803,8.7648325,8.226423,10.14198,7.745081,6.309045,2.8932168,6.206401,2.2540479,7.7360506,4.793822,2.3132954,8.835843,3.6004484,7.9298615,7.780648,5.779525,2.4110694,4.435122,6.072067,5.3437324,7.980312,8.0072775,11.013304,4.968179,5.6773715,8.469735,4.5179534,4.2360463,6.88796,4.726356,8.647463,6.1781898,3.6997395,5.7485247,12.644289,8.568998,6.369613,6.2511506,6.103845,5.3860583,7.6582956,9.080455,7.2249665,8.408711,6.585688,2.2127523,2.2127523,6.342525,4.093533,3.3686056,2.2127523,5.915159,2.878175,3.40673,4.139164,9.439113,2.2127523,12.408794,8.471503,11.181681,2.2127523,2.2127523,4.3749843,4.542875,2.2430303,8.406513,4.9781084,2.2127523,5.1677403,3.4234464,2.214822,2.2149446,2.2127523,12.883746,4.0897913,8.779171,2.2127523,8.216745,4.7284336,2.2127523,2.2127523,2.2127523,7.762735,2.3373928,7.467606,4.4978223,2.2127523,6.1936097,2.2127523,6.6826696,5.502362,2.3880835,2.2865298,2.2127523,5.6977987,2.946088,5.587374,2.2127523,7.60833,7.596975,8.733943,9.106369,2.9391685,5.995505,9.844642,8.326029,2.352999,9.9944,6.334243,2.2127523,7.302922,3.5050597,10.697053,6.167087,5.1329618,2.216131,4.883724,11.664964,2.2864444,10.216336,4.7160945,7.919561,2.2127523,4.7128425,4.7515693,9.044036,9.175177,8.506359,8.376578,6.6643195,9.178182,6.6400523,10.296571,2.9794092,5.304823,5.6002307,8.688605,7.446121,7.9276876,2.4627082,6.339641,5.527477,11.443833,2.2127523,7.146607,8.028209,2.289384,5.6114445,2.2127523,12.597525,7.312691,4.451141,7.5268626,2.7444715,8.783661,5.3257685,6.0196395,7.2041974,9.92097,6.2790346,2.2664077,3.6371293,8.129324,3.0442722,4.4245615,7.3771343,6.5981417,2.2127523,10.74937,9.36343,5.3174715,2.2127523,8.598433,2.2127523,3.5542164,2.2127523,2.2127523,2.2127523,9.518149,6.5139446,9.148365,9.786531,4.524861,2.2127523,2.2127523,8.244784,5.298531,5.0333033,6.233387,6.134241,3.7120042,5.061082,5.767242,2.2228498,2.2127523,10.716892,6.287071,6.6282926,5.190285,2.2127523,3.883841,4.9422345,3.8903074,3.694468,12.345695,5.494696,2.4149761,4.478973,4.6444516,9.047211,7.450762,3.620966,3.788627,11.3064165,5.4951253,6.372029,3.448138,2.5651553,2.2127523,5.662711,4.5583563,2.2127523,2.2127523,2.3013737,2.7577903,6.913215,3.2822773,2.2127523,2.2127523,7.530912,2.2127523,5.7095327,2.2349367,7.905954,2.8446567,3.699579,8.454884,3.6005085,6.227562,3.6860914,10.605402,4.0326314,5.877624,3.45505,2.291204,9.95411,9.630349,9.7867565,7.081921,6.5576477,5.9726186,5.7636485,8.541433,2.2661073,6.2350183,3.1669648,6.8377714,4.051753,2.248202,2.222218,2.213045,2.2127523,6.9228578,3.268055,4.9799356,7.120887,4.977457,2.2127523,2.2127523,6.82836,7.776281,9.279495,6.156553,12.317776,9.477605,2.291454,2.2203972,6.763371,2.2589602,3.3621967,2.6564932,2.2127523,7.7064753,6.896344,6.596862,6.8469114,2.2127523,5.298726,5.596081,8.970298,5.2913175,7.5212207,8.548828,8.527934,9.175339,5.581915,7.790079,5.8462954,4.1623187,2.2174187,9.868688,6.2812066,2.2136858,7.398133,2.2418494,3.2164006,2.218784,7.9599085,3.6925826,4.5298877,7.69717,2.214202,2.2968714,6.1621523,2.2127523,7.9422264,6.8922005,4.4776855,3.407632,2.224864,6.698692,9.663963,5.812603,7.8053646,2.2127523,2.2757905,5.163705,12.540119,8.1053915,3.8417819,8.442844,13.296147,10.158238,4.1463933,4.8091645,2.2127523,2.2127523,2.2127523,2.2127523,2.2136352,5.7422085,4.624764,2.251404,4.2172623,2.5090375,2.2127523,5.213894,2.213142,6.805389,2.2215965,2.2127523,9.495648,10.019214,6.971876,5.2741776,9.066762,11.99642,9.449573,8.533188,9.288972,9.591682,6.9498396,4.3482428,5.592674,7.275727,9.184015,5.5814815,9.800241,10.406166,6.9237633,12.317793,7.372307,10.988838,12.225096,7.9358406,5.6638246,11.116851,8.469496,6.0677094,5.858585,6.985401,9.600113,5.368205,8.217462,8.181052,6.930181,8.568132,2.5719159,7.645703,6.023268,8.037228,9.190524,2.2127523,6.551456,9.76737,6.724769,9.291754,5.22881,9.359532,10.106285,9.38269,5.808183,2.2127523,3.803148,8.35585,6.49515,7.521405,8.504487,7.228384,9.617844,4.8733206,7.7298183,11.756467,8.164833,8.047223,11.10314,8.4961405,5.1852274,5.6305346,9.318815,8.294475,6.633982,9.498811,7.8965406,8.892512,10.262566,5.88716,10.507242,9.406026,7.6853266,7.8882127,5.5703373,5.749834,3.9842901,2.2127523,5.1778574,2.2127523,5.290242,6.642557,4.3715963,2.2127523,9.24721,2.2127523,7.260567,2.2127523,9.55906,2.2127523,2.7517838,9.850574,8.512622,4.4045568,5.987057,2.4056532,8.151251,7.590446,2.2127523,5.0185065,4.1875787,6.6233435,2.2127523,6.1734557,7.083056,7.1505594,4.971229,10.009297,2.3002331,10.519444,7.483195,6.5152225,4.0331903,8.030737,6.3209734,4.895548,2.812227,7.8775907,5.7485046,7.4106283,7.0116854,2.2127523,7.330561,3.1611006,2.5587285,10.332509,8.584446,5.5336814,9.317079,10.854354,3.5997605,10.371725,11.977075,2.7794886,2.2127523,6.259828,2.2127523,4.833794,4.2742186,2.2257614,11.121296,8.91289,2.4214945,2.376709,9.226885,5.642211,6.6691422,9.029506,7.1741104,6.6553893,7.7484393,6.378281,6.5926685,6.2375965,6.6791034,2.846307,5.156578,3.7548275,9.190955,2.2127523,6.2216115,3.9425893,4.8929944,4.733282,2.611268,9.232838,5.844421,2.2127523,6.546627,6.7827644,5.0534496,5.578235,7.191136,5.1188464,6.358591,6.2298713,8.402735,6.8544154,2.2130103,5.994173,4.1398387,2.2127523,2.2127523,2.2127523,7.5520124,4.2288666,8.152893,2.604282,9.19517,4.590975,4.6527166,9.879613,4.7661524,2.3109565,2.2127523,5.1820784,8.211371,2.6974397,2.5788925,9.344282,9.249397,6.763981,10.736755,2.2127523,2.311943,4.123561,5.2836123,2.2947874,2.2127523,2.2127523,5.1490817,2.216626,2.894927,3.184106,3.767522,2.2176597,3.0325005,2.493784,2.2127523,4.1451836,2.2127523,5.879765,5.953259,2.4552307,2.268717,7.5952625,2.2127523,4.1872163,8.422612,7.084988,7.047043,3.2803323,2.8733733,4.3399215,10.394516,10.716705,8.963881,6.2591867,5.7955728,9.051657,9.476838,8.3074875,9.225314,7.8886733,2.2127523,4.0942903,5.077692,4.716732,4.2745256,2.2127523,2.216373,2.2266665,4.7484803,10.678264,10.290851,2.2127523,3.211947,2.2127523,2.29096,2.2127523,3.284917,10.715945,4.8349714,3.1382997,6.176858,2.2127523,2.2127523,2.2127523,8.203816,4.402726,2.2127523,4.3096642,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.3812363,2.218026,9.318758,2.2127523,4.1113105,9.184732,12.696106,2.2127523,2.2188644,2.3378632,2.6912427,10.050569,8.653505,2.2127523,5.029856,7.6262364,5.221919,2.8875616,2.2127523,2.21547,2.2142348,2.3948932,4.3901367,4.5980077,2.3684905,4.7090106,2.2127523,6.1118007,2.2127523,6.1263633,2.2573583,8.405833,8.4615965,4.2984223,6.207255,7.9774957,9.464211,2.2127523,2.2132282,2.2127523,5.193065,5.844617,10.92436,10.039666,2.777765,2.9233625,7.735032,5.465597,2.6108766,2.616261,2.9070313,10.843631,2.2127523,2.2127523,4.641797,2.2431004,2.2127523,3.4330907,2.2127523,2.2144248,2.2127523,3.7799156,2.8575716,2.2127523,2.938045,5.3481026,2.2130768,2.2127523,3.6709635,7.625481,6.897833,5.474318,11.010853,7.6921945,7.59626,8.107437,4.4638844,9.1768465,2.2733216,2.530259,3.3680837,6.3754234,7.714568,2.2127523,4.9653077,7.121817,10.40008,6.815866,3.8987012,8.964137,8.986634,6.5454445,8.0786295,7.141565,6.9635115,2.4644792,4.001046,2.2160506,2.2127523,5.921073,5.194364,7.2567463,6.166597,12.032879,2.2818913,2.3340003,12.030637,6.7453814,5.84586,6.0890965,8.168932,2.2127523,4.752526,10.932531,8.089677,2.2127523,7.530829,10.371376,4.343514,6.350019,4.907719,3.76582,5.60827,2.3595006,2.228747,9.779597,2.359947,5.9851775,2.2127523,2.222535,7.113188,6.310138,11.425415,5.551237,4.7271996,4.761389,4.957517,8.405333,2.238729,6.656658,6.813357,7.6840444,7.974802,13.537291,7.4337463,7.8299155,6.181175,10.234625,12.078579,7.7569838,6.0632625,7.306412,7.207096,5.8584847,3.5193129,2.2127523,4.746216,2.2127523,2.2127523,2.9198678,8.486056,4.05492,5.444265,5.0188384,2.2127523,2.2127523,2.2127523,2.2127523,2.7155583,4.7531967,7.849164,8.809578,5.2835755,2.2127523,8.7856045,8.2871065,7.7561364,11.7013645,7.2359,4.7124453,2.2127523,2.8605802,7.5784364,4.663249,3.520381,2.2919354,2.2127523,2.2127523,6.9931865,5.8086786,5.788637,7.38497,2.4964132,3.297393,5.7307982,8.949027,8.696141,2.6602004,9.140386,5.848563,9.339925,6.790828,7.3451133,6.957319,7.6592045,7.887543,8.066487,2.9896455,2.7206023,4.627949,7.41755,3.4946556,7.7513657,6.1194096,8.315682,12.410344,5.625657,5.6063495,5.405415,7.0604916,4.1428127,2.2127523,3.2203367,2.2127523,2.8012834,2.257623,2.6185403,6.291012,15.103246,15.499927,15.399266,11.051041,6.778746,5.322999,4.6181,5.62994,9.362474,6.7185426,8.062956,9.806953,6.085382,3.958964,2.2127523,10.744066,4.6590304,7.50144,2.6445541,4.431449,4.4399405,2.3060408,2.2127523,11.00641,3.7733536,7.8743024,11.752746,11.079573,5.2061334,10.762416,13.016381,4.9003415,7.463762,9.681325,5.506299,9.887222,8.951391,11.991187,8.201491,8.959361,9.025327,5.5382943,4.610669,7.2560673,7.985056,9.184851,10.880044,7.764527,2.4356089,9.788466,7.876978,11.696299,6.813314,7.161819,9.279951,11.9273205,8.267192,8.816512,6.1454015,10.952013,9.360751,6.319544,10.8073635,11.268738,5.136357,6.0646186,10.535504,3.1019058,8.44186,2.2138484,5.3419504,2.2127523,2.2418888,2.2127523,7.0070453,8.760384,6.303119,7.0517707,6.0398426,7.2693644,11.862929,5.8898544,2.2285202,2.3873422,2.2127523,3.2086596,2.2127523,3.5735738,2.2127523,4.5826273,2.2429724,2.818331,4.372104,5.3374214,5.641768,3.0387895,4.7326593,11.317618,5.5124655,5.023558,8.606113,9.126436,2.2168941,8.23762,9.527433,4.331554,4.236832,3.571211,12.872829,2.5710285,5.4437475,8.555085,2.236095,2.650422,2.2375956,2.3294368,8.5418005,2.7342331,2.736637,4.0840545,2.2127523,2.3356032,2.2127523,2.2127523,2.5300922,2.2127523,2.2233024,6.2776976,2.2127523,2.2127523,3.4231067,4.806259,2.2647529,7.217883,4.0063562,4.735065,8.594574,7.082799,3.7508411,2.3837283,9.821633,6.723275,9.3260145,2.2127523,4.9922705,2.2127523,7.971742,7.1658607,8.956783,4.1133475,8.725512,4.2896576,7.3347073,4.073212,2.7727585,7.46434,3.2402565,11.253981,4.8839297,5.346649,10.06865,7.9310102,7.400523,5.8780336,6.784079,5.4379745,2.2127523,2.2432127,6.5688915,4.2710834,3.8960268,2.2127523,11.005635,7.5584307,9.875757,2.2127523,4.699863,2.2127523,2.2127523,4.17337,7.810433,2.2324085,10.973936,5.7820363,6.5943427,4.195793,3.3292887,2.2127523,6.8438177,3.6479661,6.2647505,9.015106,6.4904556,4.6608357,2.715594,4.879441,8.742442,8.467531,6.1818194,8.7334795,5.840694,8.184365,5.5021577,5.008773,2.2127523,9.028619,7.61266,9.440748,2.2705808,6.4596996,7.3216825,9.061388,2.2127523,2.2127523,2.2388887,2.2414708,7.2284846,2.2127523,2.2544734,3.0058749,5.576981,7.765741,7.9461255,7.0466237,2.2127523,7.3575163,6.834044,2.2127523,10.516869,4.0257006,2.2127523,2.2127523,2.216605,5.0232906,2.5906258,2.468173,2.2144976,2.2127523,2.2298687,2.7586308,2.2127523,3.314513,2.5261357,3.2651012,2.2385159,7.076353,5.621272,6.299963,3.2580888,8.910077,3.1199,2.2127523,2.2127523,2.4267843,2.2127523,2.2127523,2.505703,2.480137,2.2127523,2.2127523,2.2127523,2.2135086,2.2127523,2.2127523,5.988393,7.1147127,2.2127523,9.747098,11.164701,13.463301,7.4880795,9.524665,6.0536056,7.692302,10.669878,2.2127523,2.2127523,10.689824,3.4361825,2.6803615,3.747302,8.059281,8.0689,2.244333,6.572794,5.363573,6.490311,2.4645176,2.2127523,2.669698,2.3126438,6.737539,8.28059,8.451223,8.383216,2.2127523,4.5609837,8.598257,6.559994,8.651797,5.930731,2.4036105,7.055192,4.8989105,10.859301,6.858939,7.353477,2.2127523,3.2557776,4.246255,9.889441,9.4979725,6.535168,8.420447,7.9319277,6.732165,11.2265005,11.44224,6.971018,2.6459568,4.898394,7.6217313,11.072052,5.3075933,2.2159312,3.75001,2.2127523,2.2621095,8.157725,2.6409562,5.10485,8.721413,12.5647745,5.043183,3.3761687,4.8222165,3.0464563,2.8056262,2.9124212,2.2307782,2.2127523,3.324008,2.2127523,6.451758,2.2127523,2.2127523,2.2127523,2.6846893,3.3077414,2.5108767,2.2127523,6.0362353,2.6292214,2.2127523,2.2127523,5.201062,2.2127523,12.690993,2.2127523,6.8782454,6.454985,10.339241,2.4458125,2.2277792,2.2127523,2.6632123,6.794812,2.2179437,2.578454,2.2701952,2.2127523,7.603633,7.337436,8.837919,6.716658,9.538994,12.364536,5.858494,4.5210443,6.482407,6.050588,2.917128,2.2730222,2.2127523,3.6368682,5.0151386,4.65856,5.7100997,4.4790726,2.2127523,2.2127523,12.583979,6.619241,7.2538476,2.2534645,2.8041158,2.2127523,2.2127523,2.2127523,3.609309,2.2127523,2.2127523,2.2127523,9.094947,5.3554997,2.2127523,3.1787224,4.049453,9.858995,7.5514107,6.71178,2.2127523,2.2127523,2.2127523,5.7386045,7.1441236,6.3846865,2.2127523,2.2127523,8.30705,5.709163,5.634361,4.995143,4.9776864,9.854649,7.1144657,4.7278037,8.226616,3.2367454,2.2127523,2.472424,3.3260791,2.2127523,5.9339867,8.850903,8.449222,7.9146614,2.7578433,9.019292,3.1320646,5.8455954,4.3047934,2.5887458,2.366915,2.3961623,2.2131376,2.2127523,5.8606105,10.9176655,2.253943,2.72298,5.360514,2.3089263,4.7031794,2.934571,6.8146553,6.2784724,5.7955832,2.2127523,3.1656086,2.2127523,2.215592,2.22407,3.1478846,2.2127523,12.318293,2.2127523,4.91274,6.3589277,4.0458612,6.3721833,2.4903064,5.44479,5.605286,8.937672,8.102111,6.624901,2.8740952,9.244404,9.170445,4.610235,6.034042,9.285635,8.842707,5.746194,3.5242581,4.2416067,2.2127523,2.2127523,9.767938,10.577109,10.757326,9.327272,5.7943106,4.012948,2.5047207,2.2869372,7.0644917,2.578294,9.3356,3.3924675,6.793244,2.2537618,6.804266,2.841051,2.2920644,10.216843,2.2127523,2.6912174,2.2127523,5.982385,2.2127523,2.2898836,2.2761798,2.2127523,2.2127523,2.2127523,2.2127523,9.659079,5.865703,8.323098,6.7233834,8.376983,8.703272,8.067103,9.731709,6.653757,6.3810406,4.761254,3.9315925,5.497755,3.6498823,2.9328542,5.1686935,5.611991,4.398509,8.562198,5.399254,8.12053,4.0413485,4.1587524,4.183998,9.69013,5.5927925,9.027921,4.8599067,7.366906,5.3806696,9.294094,5.831346,3.8816288,8.5420265,2.2127523,6.7878323,5.349878,9.022661,5.3909206,4.693227,7.193697,9.26785,7.259774,5.381166,7.9102993,6.6514306,4.055622,5.723688,6.998437,2.2163193,2.2127523,5.125324,8.871761,5.706888,6.5339932,5.9214244,2.2422438,5.360253,5.7771163,4.2851834,9.241575,6.226208,4.724014,6.926752,7.941435,7.63317,10.610107,7.804945,4.9693975,7.469114,8.065515,2.2189722,2.2127523,2.3994462,2.2127523,2.2127523,7.5083556,2.2127523,8.833906,2.2127523,2.2409823,2.2127523,11.238276,2.2768264,2.5991123,2.256648,5.740359,2.2127523,2.2127523,2.2127523,3.51907,2.2127523,7.778079,7.4988756,2.2127523,2.2127523,4.03473,2.3046312,2.2127523,4.3551965,8.196459,8.502639,7.019623,6.7247596,6.4620166,11.757998,13.979509,7.509481,2.8948696,7.0332837,2.2127523,2.2127523,2.645118,3.441536,2.9764476,6.4000974,9.016707,11.509522,2.2568452,8.446522,2.2127523,2.2127523,6.087946,2.3068616,11.256452,2.2245445,6.1842914,5.1924405,2.537669,3.9592779,2.8173895,2.2127523,2.345562,2.2141173,8.707252,5.546656,2.2127523,6.71804,4.9413915,7.1669264,5.944977,8.325304,2.2127523,5.4760857,3.183419,9.313875,3.616989,9.467405,2.2127523,2.2127523,2.2127523,7.6889644,2.2127523,4.6956387,2.6383653,2.2613378,2.2127523,3.333694,3.6990886,4.7430058,10.105419,2.2127523,2.2127523,2.2127523,9.291814,8.561648,3.656894,2.2127523,2.2127523,2.5392065,2.2139673,2.3941605,2.2127523,5.9419975,4.5663,2.2127523,2.2246304,10.478839,5.8871784,2.2127523,8.664688,2.2127523,2.2248704,2.2127523,2.2127523,2.5726702,2.2127523,2.4568033,8.775925,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.3291848,2.7195792,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2352018,2.2127523,2.2127523,2.2127523,2.2258565,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2373757,2.2127523,2.2213473,2.2177713,3.914111,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2787137,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2270486,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2209067,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2133057,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2343938,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2173986,2.2127523,2.2127523,2.2127523,2.2127523,2.2200036,2.2127523,2.2127523,3.5293767,3.7117615,5.2218223,2.2127523,2.2127523,2.2127523,6.6394787,3.0020876,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.6214967,5.3135915,7.832069,3.0561988,2.2127523,7.152532,8.46987,7.6100187,5.307391,7.139664,2.2127523,2.6595879,8.051898,9.382858,7.556541,9.521389,6.8534975,2.363459,4.11638,8.248304,6.354637,7.289639,7.918395,5.092406,2.7858446,3.8720636,7.8180737,10.695197,6.270529,3.379758,6.664694,3.326036,6.8555093,7.771217,3.6873884,4.843028,3.917456,10.178105,2.217806,5.3476605,11.487778,11.128609,2.2127523,9.112203,6.0253,2.2127523,2.2127523,2.2127523,2.2127523,2.4483645,2.2127523,2.2127523,2.2127523,2.5832734,5.448003,4.213355,5.699284,7.1915355,7.4258685,8.128964,2.6396546,4.9092474,4.0716634,2.2127523,2.2127523,2.2127523,2.2127523,5.4669666,4.0719085,2.2127523,4.505283,2.2127523,11.053183,8.019643,2.4381628,2.2127523,2.2127523,8.409338,7.898327,6.496492,7.3911104,8.2004385,2.2127523,3.2454925,3.1038332,2.2653027,2.2127523,8.174144,6.765145,11.8181505,2.2127523,6.1132293,2.6816757,3.916705,2.7255557,4.3811755,5.8943877,6.169945,2.908581,2.2127523,7.2858953,6.877285,6.0838227,3.0132413,12.321283,8.878549,8.390844,2.2465498,6.092513,9.684739,6.0175314,2.2127523,3.1386762,14.19502,3.9912882,2.2127523,6.1761394,2.5391276,2.2127523,4.5129437,5.3393574,6.312517,2.2472954,6.5668964,3.3446639,2.9035592,3.438719,2.2127523,2.218987,2.2919397,7.821388,8.766484,5.797233,8.262299,5.7634583,9.889772,2.2127523,2.218202,2.2127523,5.9228206,7.4770374,10.541068,8.264748,5.9314637,5.9038525,7.6209397,6.7579885,2.5386956,5.77779,3.605861,2.2127523,7.463234,3.510851,7.706416,6.110854,2.2441406,6.5624223,3.2650783,11.4021845,6.6557083,3.093238,7.409696,9.9277315,3.8974001,5.232833,4.065981,7.0626507,6.289253,5.546879,2.6727173,2.2127523,3.9100733,9.1093445,7.1472073,9.141423,4.8805046,8.708127,2.2127523,5.4301577,4.212299,2.521699,10.022175,9.533544,5.055219,7.078775,3.6020486,4.017579,4.746395,5.5716233,2.2127523,7.227823,3.0380254,2.848993,5.1484356,4.4105773,3.7995977,2.464969,6.506831,2.2127523,12.5998,5.458324,9.7690735,8.08682,9.978442,4.375313,3.9547362,8.90043,6.822557,3.066858,6.407384,8.628972,7.932639,10.487664,7.7143054,9.247984,8.939276,2.7116344,4.9617476,2.2926068,5.685921,3.840249,5.211471,2.2127523,4.3336773,2.2127523,2.2127523,5.670922,5.2309813,4.19744,6.2937527,2.213952,2.2127523,2.2127523,2.2235768,2.980914,2.2127523,4.043671,2.6126902,6.612274,2.3969216,6.713533,2.5385857,8.597897,8.39477,5.1259856,8.061296,4.496315,4.0671043,2.2127523,6.3794904,5.518347,5.0253386,7.8556056,3.3907053,8.1015215,5.405291,2.2133641,2.2127523,3.1894326,2.2127523,6.159685,10.111207,6.8587093,10.639955,3.908272,7.424333,2.2127523,6.0259237,4.144547,5.1543055,3.7536445,2.2369955,2.2127523,5.8944945,2.2127523,6.264711,3.7544134,3.5474024,3.106537,2.2127523,4.7294946,3.8279397,2.2127523,2.215754,2.2127523,2.66179,2.2127523,2.2127523,2.2127523,5.759311,2.3135192,5.135805,4.4846487,7.6544876,4.61352,7.7757397,2.2127523,2.649394,6.0910735,2.9508936,3.0156198,2.9727995,2.2230363,4.0566363,2.9746997,2.3193266,5.069243,2.5327575,2.2127523,2.2737849,8.965634,4.938726,2.2127523,2.697369,2.2127523,2.2127523,2.2127523,2.5438175,2.2127523,2.2127523,3.270629,4.876163,6.981559,3.145886,8.572287,2.2127523,7.930717,5.6462765,5.727157,6.695899,6.680936,7.6594996,4.734799,2.2127523,8.089048,3.136091,7.0175223,10.098895,7.089163,7.066397,10.838419,2.2127523,9.452379,5.1120605,5.1346636,3.7483723,4.9398885,3.6213455,7.936151,2.8879669,8.811286,2.2187455,7.0637107,4.446855,2.2127523,5.029268,2.5262623,2.333744,7.9464574,2.4204857,7.049151,3.2804613,9.3959055,9.342344,5.3114805,5.9733753,2.508389,4.3958044,5.846911,2.2127523,2.2357745,11.318799,5.9707375,2.9347367,4.797481,6.8682475,2.8321126,2.2984896,6.2464023,8.727459,9.58796,5.8984866,7.236969,2.2127523,9.564581,7.7431607,2.8016205,2.2129006,5.171293,2.9719663,2.2127523,2.2725792,2.7146044,2.4861872,4.5471444,3.9137743,5.622256,3.841422,4.887688,3.5760822,2.2127523,3.1861262,2.2127523,6.01899,2.2354574,2.2127523,5.2527375,2.4584384,6.447964,2.3821313,4.006497,4.759544,3.1587849,7.7883554,3.4435706,4.036621,5.80111,5.733687,8.224506,2.2127523,10.066361,8.814762,2.2127523,8.668181,7.6579647,10.654364,9.494209,2.2127523,6.8505716,4.7701263,5.711119,5.839971,10.818847,4.992468,4.4169426,3.698874,5.9478445,3.1912222,7.876727,6.0326595,2.3206387,4.2572923,5.8421206,6.4724216,8.87512,4.334613,3.8056746,7.1173267,2.3338501,4.457203,6.184942,5.748158,2.2127523,9.510506,4.525109,2.2127523,2.9433196,2.2749407,6.099916,2.2127523,2.2127523,8.640757,4.8240333,2.2425435,2.2127523,9.808117,5.3277392,3.4213474,10.727639,8.659451,6.778713,7.643301,8.251288,3.9850187,2.5123518,2.8172212,9.537681,4.7924943,5.6372676,4.3365436,7.521488,5.423636,2.576951,7.6158094,5.344928,4.9662867,2.2127523,2.467091,5.7153826,4.792457,10.242292,5.345202,5.1887503,7.2751203,5.152441,4.258808,9.286349,3.6723132,6.819344,5.832327,5.236293,5.4379077,8.488685,6.053085,3.203382,5.4629335,5.97495,2.2127523,3.4308429,6.3310304,3.6663587,2.2127523,7.234479,7.7773194,10.290086,6.3871675,12.455599,2.2365365,5.0105834,10.26105,2.599053,7.753134,7.476688,5.908506,12.249589,7.152616,2.2190363,12.275162,2.2127523,2.7306216,4.197056,7.0401154,6.6997356,3.5188196,4.2573786,2.2127523,2.9654438,6.073807,4.6124825,7.671511,7.6343794,9.018286,2.2127523,5.9993267,2.2127523,2.2127523,2.2127523,2.2127523,9.742903,7.1254163,5.502952,6.9316564,5.41592,8.524139,4.270116,3.831909,3.4176047,10.031669,9.9841,6.977833,4.1918,6.6834064,2.9041374,8.259831,8.21256,9.4016075,12.393723,2.2173038,6.677656,6.049409,6.021988,3.4249072,2.2127523,6.733489,8.190242,6.979183,4.219491,6.7907,6.5870395,6.390585,5.28109,6.326517,5.579267,5.9335775,2.2127523,7.9802446,2.225866,7.4212084,8.849461,5.9033484,2.5333405,3.2661145,3.4942374,2.2127523,6.6629944,5.812895,4.1342235,8.495568,2.2127523,3.4860106,8.11066,5.6267896,7.8413377,2.2127523,3.5085354,2.407332,3.5841475,3.9559114,2.3028607,7.6602473,2.2127523,2.2127523,9.261404,3.93805,4.2532,5.444317,9.266744,6.579102,2.2127523,2.2127523,5.8031635,2.6575668,2.5559933,2.237586,5.5440392,8.46627,8.874194,7.6956525,2.2181273,8.792719,4.0453005,7.97782,4.6995187,8.5880375,8.376792,3.3520522,4.209327,4.024209,4.2482514,2.8713236,3.0011868,4.668646,9.316249,6.566717,7.085475,8.705929,4.49447,5.5441003,5.4307294,4.212447,6.753411,4.9325814,4.293504,10.836888,3.0252414,8.894124,6.011945,7.1790066,6.5269217,7.6312566,8.067383,5.8107767,9.798197,5.554501,9.376751,2.9708946,2.2356272,2.2127523,7.230974,7.719904,5.827529,2.2127523,4.43477,6.2244353,8.117732,5.153054,6.2234464,6.061385,8.690069,4.2352605,4.366955,4.1118875,3.5341926,2.2978394,4.7557306,7.059763,6.694799,6.208877,10.53997,8.406252,2.4726095,6.709035,7.1592245,4.5395384,4.0486774,6.6742425,4.092226,8.037572,4.843137,7.974581,7.0811954,3.8836527,6.280953,4.4617214,4.323408,7.7649603,2.2127523,4.204515,8.47686,8.344307,9.957954,3.6378021,5.6220956,3.379297,5.291756,5.478606,2.2127523,2.2139175,2.6272752,2.2127523,2.2127523,2.2150447,4.4745083,7.871736,10.001741,2.2127523,2.2135274,4.286326,8.001519,5.036754,2.2421021,6.1672163,2.2127523,2.2127523,2.5752466,3.8289683,4.825251,4.7545667,8.998873,2.2395618,7.9331884,2.7535715,6.551534,7.6353188,4.4721484,5.155998,3.1966906,2.212809,2.7007349,5.423069,4.8152337,3.7301157,8.600509,3.6596155,6.3416014,5.702655,3.8657818,3.797853,2.2127523,2.2127523,2.228784,4.138946,9.521079,4.475981,2.2127523,2.4824677,3.5469553,4.495376,9.6860485,2.737134,5.9302473,2.2127523,2.2127523,9.398727,2.3741937,3.3898966,3.4094012,8.378628,4.4818892,12.640346,9.624685,8.775588,2.9730568,9.241375,4.678526,4.193752,4.0178404,5.302191,2.9867904,6.6464567,4.3534684,2.2127523,2.8084457,6.785011,2.3496783,4.300449,5.3844457,2.2127523,2.2127523,3.337714,5.3663106,4.004909,3.3862085,2.5048957,2.2127523,3.6829123,6.7251863,9.573412,2.285752,2.2127523,2.2127523,7.334797,8.3152,5.1462975,9.08073,7.0067444,5.8852553,2.5206504,8.426627,4.1305685,7.1111007,3.3344657,4.9689784,3.9440758,7.6160736,5.5252666,8.152176,4.314729,4.42605,2.8783288,4.901147,4.23504,3.367287,2.2400453,3.0255804,3.8278396,3.6126215,5.019946,5.986106,4.664738,5.2612553,7.3299227,7.063917,2.5911083,2.231958,4.215901,2.2127523,9.531098,2.2127523,6.970305,9.90405,2.9260485,2.7278192,2.221126,9.239059,4.4877725,3.8043787,2.2127523,7.847002,4.261802,7.5011415,7.551317,8.443491,2.2127523,11.686235,7.631026,9.22438,14.509188,8.231954,2.2180324,7.564171,8.014228,7.2209053,5.813501,5.4499283,7.489104,5.8324676,5.7438884,3.2710643,2.2468157,4.44755,6.0237927,3.2041824,7.53753,5.219566,2.2127523,6.1470957,8.976626,2.3484702,2.2127523,2.2127523,2.39324,8.775852,8.887897,2.244269,4.527046,8.239876,7.786364,2.2127523,5.4787154,9.42519,6.4076023,3.5320084,6.918912,4.6210136,6.847786,6.627696,2.2127523,8.280297,6.4726243,8.575856,2.2251177,8.565707,5.368188,6.509015,2.2127523,2.2127523,2.218703,2.2127523,2.2127523,9.102069,5.919637,2.2902975,2.2632065,3.1202195,2.2127523,3.2497873,9.531458,7.874626,2.446946,8.21446,2.2127523,4.8123913,2.34731,5.6809554,2.2127523,6.7953324,2.9693396,9.100579,7.8859544,9.011526,10.467704,10.161453,3.0021281,7.8696547,6.89248,3.9464834,6.7415376,7.392741,2.216475,3.0874727,3.835815,3.507182,6.706975,6.3241806,9.989695,5.725859,3.8727734,5.699438,6.3302636,8.068668,5.9863496,7.9028163,7.506564,5.0350056,5.044655,5.091101,9.375994,5.581425,6.249272,7.7668195,4.6160364,7.440852,7.4936132,5.5026093,3.0681572,3.1997356,2.2473168,4.3838305,6.5164237,7.3584485,10.107328,4.5026217,3.6416433,8.660916,8.462561,8.118458,10.014789,6.0709257,9.622578,9.9081745,9.364075,8.714591,3.669611,5.6707196,10.590668,7.8388963,4.8498554,4.0065613,7.9477787,5.4582634,2.2127523,6.9198265,2.682986,11.319903,4.6925592,4.9124937,3.5815153,8.744692,7.6113925,2.2127523,2.2127523,2.2153168,2.2127523,2.2127523,3.000033,2.2218347,4.4399943,5.5195436,2.2127523,10.164639,4.8174767,3.7398195,6.411125,3.5814655,10.162624,10.834199,2.4251533,7.0025687,9.6709175,7.585236,2.2127523,2.2127523,7.5514674,2.621901,7.1295323,7.621875,2.2127523,8.62399,2.2133029,2.2127523,4.4225683,3.4489777,2.67389,2.2893126,3.2723932,2.2127523,2.2127523,2.224428,2.2127523,2.2127523,6.4112086,4.3570113,2.2509832,2.2200954,2.2127523,3.2227015,2.311204,2.2127523,2.2127523,2.2127523,2.9063075,2.262518,2.2383065,2.3611119,2.2127523,2.2127523,6.8960195,13.413083,6.9086685,5.7713594,8.088126,8.21191,8.144889,2.2127523,8.025133,8.267537,2.7740252,4.4263544,4.5840263,7.3002496,2.2970457,3.0797799,6.924119,10.227693,2.2127523,5.8074718,9.098507,8.500341,2.2127523,2.2127523,6.9974413,2.2370033,2.3890538,2.2456007,4.918609,6.849869,5.571761,8.197362,2.2127523,4.042561,12.17527,9.125132,8.853742,5.6633997,2.5932245,7.748067,8.790201,8.173779,9.132264,4.553288,10.001704,8.116847,2.5413687,3.9437056,9.248481,8.904802,6.6826634,9.59664,6.834818,2.2127523,4.8992004,7.7098436,3.8114145,2.4970033,4.857098,3.1854553,6.442617,2.2127523,5.6819005,3.7711692,10.771244,11.102542,12.043886,6.3183208,5.5787005,9.3155985,4.7593637,3.7234824,6.4569445,6.27735,4.7543225,9.558084,4.933364,9.051089,6.449655,10.589989,8.037594,5.900088,2.2127523,8.291421,2.251112,8.564855,3.4443772,7.325529,7.43274,7.6357245,3.0291524,2.3667374,2.2147527,2.2127523,7.7431984,2.9428463,3.1645865,2.2127523,5.482192,3.739616,5.2665944,5.6221976,2.751902,8.594094,9.351871,5.682775,4.7254453,7.1414723,7.2716484,8.711788,5.5861692,6.575822,3.7311628,2.2928853,6.86729,7.916924,7.3294845,8.99883,6.733789,5.885635,7.4867845,5.4597616,9.43716,10.849687,8.819297,4.0235653,4.4546294,5.2399917,2.2127523,9.288726,10.355539,2.355513,7.1298738,2.2406716,3.810138,2.742664,9.47242,7.689305,5.772027,7.234536,8.437584,12.60696,9.43239,7.2258644,2.2127523,2.2127523,7.980902,4.490292,3.2642663,5.9431567,2.2127523,5.73268,6.778272,3.6059382,3.8079011,2.2127523,2.2137084,2.2257032,2.2300158,2.2127523,3.8502605,5.794462,2.2672286,7.2042546,8.276127,2.2448306,3.9578507,6.0766497,4.615244,2.8201933,4.503069,2.2127523,2.2636535,3.536884,5.1805563,4.6030965,7.9527,2.6824844,4.2656956,2.2127523,2.214221,2.2137446,12.129815,8.230728,6.5435514,11.619475,11.886886,12.019235,7.2478933,8.454503,5.020664,5.869956,4.9621606,7.4800506,7.2897153,3.0519986,6.580839,2.2567585,2.2127523,2.5088086,2.2127523,4.2314944,5.759898,5.0384507,2.2127523,2.798843,11.828473,7.1803303,4.1189866,2.624273,7.731443,6.768285,3.828753,8.480307,5.851091,4.537972,6.753863,2.2127523,7.712548,4.625287,3.9422204,2.2161207,11.930262,2.2127523,5.7223225,8.616071,7.5758753,4.930168,4.4208155,8.207119,5.7282457,3.1916127,2.2127523,5.7923326,7.7870407,5.2124386,2.841695,8.376915,7.296523,3.1244519,7.205638,6.4392543,6.5511203,3.59838,2.2127523,2.5909154,8.020095,9.194817,6.0558405,2.3406308,2.317304,2.2127523,3.4619577,2.2127523,2.3417282,2.2596776,2.2127523,2.2127523,6.074891,7.2517214,7.7850146,8.591115,2.2127523,7.3010874,5.6865835,2.2699459,7.4755993,2.21327,10.166122,2.2127523,3.5250547,4.1571403,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,10.153837,2.2127523,2.2127523,2.2127523,2.2127523,2.9379416,7.668137,2.6820095,2.2127523,2.3868718,2.2127523,2.2127523,2.3490994,4.1294966,2.2127523,4.074152,2.2127523,8.655667,6.304576,7.286499,5.1265154,2.2145722,2.3002393,4.0852475,2.3737593,2.2127523,2.2127523,5.5200553,8.8333435,10.023931,6.084133,7.739471,9.229657,8.608538,8.524091,8.744057,4.518414,9.568533,7.4839044,10.145305,2.2127523,2.2127523,7.6630583,2.2127523,8.897119,7.1667905,9.235953,10.239917,4.509824,10.80019,6.802033,4.308178,2.4937992,8.889187,3.521142,2.2276719,2.2387547,2.216493,2.2127523,2.2127523,6.5428953,2.2127523,5.7228494,3.706746,2.2161374,2.2168088,2.2127523,9.565344,2.2138734,2.8929327,4.4758725,2.6036015,2.2127523,2.2127523,8.899593,6.4080944,6.0509257,9.41285,5.8359075,2.9625962,2.2139049,8.6478195,3.1973066,2.7470453,3.8279576,7.040472,5.1439614,2.2127523,2.2127523,3.3370042,6.393107,2.4363601,3.0083697,6.12392,5.853931,2.2127523,2.538239,2.2127523,2.2496903,3.3163033,2.4800313,2.2127523,2.2730865,2.2127523,2.219422,2.9141111,2.2127523,2.2127523,2.283219,8.0067215,5.5966606,4.725649,2.2127523,6.0268464,3.4692585,3.5730023,12.9817915,2.2233562,6.4061313,2.3084793,2.2127523,2.2841666,8.056344,5.789318,5.0005064,5.930992,8.281218,2.2398665,2.2914574,2.2127523,2.2127523,2.2272885,2.2934875,2.2210658,2.257846,2.3001802,6.9627657,5.4281263,2.2127523,11.427941,9.788019,7.0977745,11.876445,7.230968,10.857781,2.2127523,8.562087,9.597987,5.560732,11.081022,11.502068,10.391855,10.315874,8.765882,10.553047,5.9777794,11.2061615,10.549184,9.40291,10.253916,6.1753817,9.056947,11.170019,12.176796,9.650637,10.47634,8.288488,10.129758,9.296781,10.544157,10.051139,8.929383,9.245527,5.8204923,5.579413,6.2989836,5.641663,9.614996,11.097607,5.750109,12.310176,7.2048173,7.8602204,6.8382044,9.009295,11.163015,8.294965,4.839793,4.0821986,2.2127523,2.2127523,2.2127523,6.277921,4.687062,2.357678,5.16317,3.1028285,5.0528665,6.634818,6.4408755,8.307266,5.12239,5.2112694,8.81248,6.1040316,6.9983997,3.7548711,2.2127523,2.7396162,2.2188232,2.2127523,4.397899,2.9225166,8.948658,4.9942055,7.663158,9.11439,2.4303756,2.2127523,8.910268,3.3251302,5.386349,2.3105652,6.828775,2.2127523,4.50535,7.1429996,13.164908,2.2127523,2.6905985,7.2209663,3.2549005,3.6917703,8.046662,6.7625294,5.61282,7.73792,2.2127523,4.5120296,4.1249285,2.2127523,2.3668644,8.350714,3.906863,4.883076,4.8819904,12.636325,2.2127523,4.3405313,4.1548996,7.251202,2.2173162,10.018804,11.5767355,5.7425365,5.615427,4.401468,11.266005,5.2327094,8.345955,7.2819653,3.5679212,6.4542484,8.296115,9.53931,5.5078197,6.9244347,5.11788,2.2127523,3.5177567,5.1934404,3.4760478,6.1599703,5.409277,2.215358,8.639509,4.120114,3.881325,4.9878597,3.8298695,7.9135523,7.034412,2.6887813,5.5846095,7.049765,6.609179,3.278876,5.655641,4.199827,6.133674,7.8818617,2.2436814,4.161453,7.14439,2.2251413,3.771859,2.2127523,5.029138,3.3260067,7.433093,3.9910042,9.285757,10.130081,9.056853,9.104375,12.362523,2.2127523,2.2770743,3.4575608,9.475739,10.683424,7.7846303,8.3569355,2.2127523,2.2186203,5.096052,4.2045016,8.156189,7.9723115,5.512192,4.8500533,2.6586893,3.794801,5.980884,2.9984503,6.7902255,7.5550146,2.2127523,4.7775884,3.579019,7.755862,8.484355,4.046264,2.2134073,5.2148843,3.5296845,5.6854415,6.2654285,2.2127523,4.897907,7.0461555,4.409458,5.0702767,3.01331,2.2127523,4.6540837,9.092369,2.2127523,9.226764,3.03367,2.2127523,5.3458514,8.320875,2.2431266,2.4819808,4.866579,4.460813,7.3290486,4.4247336,2.2127523,2.7397487,3.2056155,4.9171762,7.310122,7.502629,2.2203336,6.0270247,2.2127523,6.0090156,2.490023,4.5030274,7.416213,6.2970037,3.9403093,5.8113256,3.6834311,7.3247905,4.7869153,5.5524306,9.094444,8.623447,2.2127523,10.495995,9.623702,3.3069875,3.6909266,9.828021,7.905003,6.237696,4.5073028,6.817687,6.5745516,11.624424,8.3446,3.8007257,7.0006385,7.572529,9.089227,8.75559,5.4729066,6.2088895,8.674455,6.3534446,8.024805,11.304885,2.5402272,11.146937,5.059906,7.471687,6.846842,8.016883,9.2926235,4.098403,7.1879964,11.156204,5.11095,3.1548316,7.709379,5.8362246,3.6603348,4.5763125,6.7952814,2.2127523,2.5147107,4.7510867,2.2127523,2.2127523,4.8806577,7.287032,2.7898266,5.1248164,4.4426923,2.2306888,2.2127523,5.811957,4.561774,6.449374,3.6361804,8.268708,3.0451727,7.959849,8.022559,9.0235405,6.6710324,3.5416074,5.5856023,6.7614083,10.070367,7.601581,9.474249,3.2290688,2.2127523,9.122798,7.172311,4.642098,4.985624,7.8466115,7.164881,4.7166424,7.3615627,9.647702,7.2054434,7.87257,7.0006776,5.699965,10.7271,9.988861,2.8149648,8.783835,2.2127523,9.249433,8.269541,4.0935726,11.111513,2.2127523,8.995023,10.273847,7.878606,3.2410505,5.537877,7.1924486,2.5698562,7.0870457,2.8162885,4.303552,2.9329007,3.213657,2.2127523,4.869895,5.0497646,2.2229457,4.061111,6.30995,2.2127523,8.266992,2.6643353,2.2127523,5.0295916,10.952291,4.8901443,9.140658,9.00016,7.568197,5.281471,5.6276507,6.1595817,8.975422,3.7843056,6.4949284,8.91081,3.2006483,2.2127523,7.7908893,5.0087028,2.3248615,3.3769984,11.315492,5.0259557,3.8258336,3.8367996,8.82261,4.488813,2.2127523,9.701914,7.926367,5.558727,6.9419684,5.3657913,11.7085705,4.8072596,6.526774,7.2356153,5.976982,6.256511,9.327584,2.2127523,6.072186,6.644072,2.2997124,3.335534,2.2127523,5.2348313,7.0753665,3.803316,9.958008,2.2130604,4.73273,6.203944,2.2627766,3.9945767,7.5327687,9.10266,9.841785,10.450624,5.2113614,9.581549,8.291735,3.832358,2.2127523,5.122714,6.2237506,9.140965,2.6262987,9.152024,5.342871,2.2191536,2.2127523,2.8333137,3.4348257,9.163595,3.4424472,4.997753,2.4482708,2.9265802,2.2217474,2.2127523,2.2127523,3.3258536,4.412733,6.139682,2.2127523,6.2724724,7.193493,8.374969,6.7288303,7.762415,5.720794,7.214387,4.4393163,4.3151364,3.5915418,4.50366,3.6687875,2.2127523,2.7431517,7.9510307,10.836011,6.778863,2.2152777,8.402704,2.2127523,7.7298384,6.375397,10.7722435,9.536257,2.870174,4.9316645,7.967823,4.325388,2.2127523,2.2158473,7.2077413,2.2127523,6.922384,3.687012,9.039154,5.66653,2.4744618,8.852453,8.416942,2.2127523,5.3582726,6.127899,5.8494873,6.5692883,6.7365546,7.0579243,3.9031413,2.268324,9.160097,8.917492,2.6747284,8.078567,5.4570537,4.304084,8.515057,7.9934273,9.410703,3.9077566,2.2145474,6.2960258,9.364079,8.889978,7.6155343,7.8472953,4.974851,4.480808,2.2127523,6.9240913,6.0213046,5.6201262,8.363301,8.028569,8.353736,7.850935,9.745791,5.632882,4.9308596,2.2127523,6.063607,8.766061,2.2127523,2.2127523,3.0143425,4.4772844,2.5471497,2.2127523,5.3745804,3.9752214,2.8120847,8.542977,2.2127523,6.7681174,5.4859405,4.554682,2.2127523,11.274514,6.58602,6.912124,5.6972904,9.02246,5.197745,4.33322,5.10161,6.605698,9.533433,4.7451673,4.5272317,7.4902277,5.4308715,9.551951,10.047962,5.6459694,5.6287155,2.342195,6.970453,6.006465,2.2127523,2.2127523,5.5663834,4.9751787,2.2127523,5.596859,2.2127523,3.0079782,2.2494798,2.3058906,7.0800233,3.4688833,4.546843,4.007473,9.451888,3.8646886,3.082373,3.795393,3.0066187,8.946341,6.0259705,10.738661,4.6431074,2.2143152,2.2127523,2.2127523,2.2290087,2.2127523,2.2127523,8.080737,7.2327867,4.559228,6.422911,2.5043843,3.3597739,4.8988004,5.3670073,2.2203937,2.2148817,5.211678,5.038818,4.9802947,3.430203,8.319811,3.6876702,4.406879,2.2127523,6.518089,2.2847674,3.9299397,2.2127523,7.889274,6.5998197,7.8117175,3.535939,2.2127523,5.304913,6.9821296,7.9914317,7.880749,5.2730618,8.381877,7.5967517,3.4855194,2.6666837,9.217961,7.209348,2.2127523,2.2127523,2.9848993,2.2233403,2.2127523,4.6082454,6.6255956,2.6455944,8.418146,5.9971333,5.1596465,3.5907838,3.538502,2.2127523,9.069665,2.2127523,7.1236873,8.678701,4.7686167,2.2127523,4.9242253,6.6921015,8.231132,8.419549,4.598786,2.2127523,2.8775375,5.592934,2.2191043,5.9771156,8.017057,3.5025089,6.974185,2.2127523,2.2177653,8.088671,5.65977,2.2884915,3.1802008,4.721982,4.19232,2.9454362,5.241453,6.332654,8.501964,10.123959,3.6158817,3.1226914,5.2916136,2.8377702,6.3335137,2.2241614,2.2127523,2.2127523,4.5841265,2.542832,2.2127523,2.2127523,2.2127523,5.584918,7.7384253,5.4838405,6.370524,7.5057406,2.2127523,3.031164,2.2127523,2.2185729,2.3303413,2.2127523,2.318669,6.1979475,2.3423016,7.3915024,3.2845013,9.54608,4.729931,6.845717,6.0464387,7.415867,5.669355,5.4781294,4.778018,8.097579,7.0628963,3.3303287,7.662062,8.613651,7.001516,9.885304,3.5278225,2.2253826,2.2127523,4.5348167,2.2313643,6.9266415,2.630644,2.3333893,2.2362404,4.808499,7.961984,3.0357325,8.211978,4.501262,7.9861035,2.2127523,2.2127523,2.904209,4.4501443,2.2127523,2.2130802,4.426907,2.6658642,4.8852706,7.5749016,4.004639,8.823363,9.142369,7.344926,7.070076,8.78569,6.944051,7.7602787,6.552407,5.279919,2.2127523,2.2127523,4.741558,8.183874,2.2127523,5.870089,2.2127523,6.253271,7.904467,2.3801382,4.3472085,2.2127523,3.3743305,6.7947907,4.116505,9.166874,9.079507,5.5528297,7.6978874,5.720184,5.1550937,2.2127523,2.2127523,5.316477,7.488949,2.2127523,5.5774226,2.2127523,8.170709,8.651015,2.2127523,7.689664,5.565567,6.255167,4.3504744,6.2506933,7.531018,6.113739,3.828498,2.4380054,7.8839674,8.098341,5.579389,11.439967,9.1131735,2.217883,7.630406,9.6818495,7.8472133,2.2127523,5.200043,2.9783182,6.804221,6.381458,6.6876116,4.458215,9.928368,11.078038,4.170271,2.2127523,10.048865,8.70774,9.920107,3.6103144,6.5335793,5.5533204,4.376246,2.2127523,5.172493,7.8607745,8.557193,2.2127523,10.735305,2.695986,2.9026177,2.2127523,2.2387488,3.5416527,2.2127523,7.3764424,7.6139245,5.7870684,8.304305,2.2127523,6.4838862,3.0449555,3.8236659,11.69337,3.0382276,2.2508516,6.340559,2.2127523,2.2127523,3.468534,10.200553,7.6515107,4.639392,9.906763,3.6317587,2.2127523,2.3173208,2.2127523,7.9007487,4.874522,2.2127523,5.6831136,4.003369,4.8994284,7.3185053,3.6236594,5.5134654,5.3553405,2.2188413,2.4422185,5.7698965,6.7993846,2.6562848,10.236536,3.5511198,6.5181823,9.375772,8.434734,6.1727934,6.009729,8.003545,4.595153,7.751148,2.6278362,5.677138,5.753156,4.2393346,6.632423,6.394989,7.978177,10.512171,5.4305997,9.187039,2.2359254,14.671136,7.8250875,3.0304716,5.851852,4.1822314,5.946971,6.920883,7.1693707,2.3287542,9.656845,2.2127523,2.2127523,2.2244465,3.338278,2.2127523,2.7247357,4.249376,2.765582,6.462822,4.8162355,2.2127523,2.2127523,3.246758,3.842363,3.5943954,6.0736394,2.2127523,2.2127523,2.6657696,2.2127523,2.9488487,2.2127523,7.332581,2.2127523,2.2127523,2.371815,2.6275575,2.297669,3.0056205,2.2127523,2.2127523,2.2127523,2.2127523,2.2880557,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2146463,2.5410247,2.3564508,2.2127523,2.2127523,2.2127523,2.2662542,2.2127523,2.2127523,2.2127523,4.9482884,2.2127712,2.2127523,7.322951,2.2127523,2.2127523,2.2127523,2.2211692,3.1220138,3.9255667,4.128246,2.2127523,3.2923405,2.2127523,2.2127523,2.3862314,2.2127523,2.2127523,2.2879786,2.4021895,2.2127523,2.2416632,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.248052,2.675774,2.2127523,7.1926827,2.2127523,2.2127523,2.2127523,2.2127523,2.347622,2.3005435,2.2127523,2.2239034,2.2127523,2.2786527,3.3217762,2.2127523,5.7323456,2.2201314,2.2158823,4.763241,2.2677548,6.1873913,2.2127523,2.2127523,2.2127523,2.2194936,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.903437,3.0804791,2.878577,2.2127523,4.458424,2.3792484,6.4866085,5.525396,2.4588451,2.2127523,2.2704418,4.627101,2.2127523,2.3002436,5.700848,2.2327845,2.2127523,9.358706,2.2127523,2.488605,2.2127523,2.2392352,2.2127523,2.2127523,2.2287195,2.2127523,2.2127523,2.2127523,2.4218686,2.2127523,2.2127523,6.221344,3.1799438,2.2127523,2.2127523,2.2127523,2.2593868,2.7881577,3.9070542,2.2127523,2.232791,2.9446847,2.2127523,2.2127523,2.2127523,2.549508,6.743841,4.0541286,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,7.2737274,2.2127523,2.2127523,2.2127523,2.2127523,4.754168,2.3785112,2.2127523,4.4021587,10.008771,3.5946424,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,11.490778,2.213729,2.2127523,2.621687,5.2068753,4.243777,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.460872,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,8.180806,2.2127523,2.2261512,2.2127523,4.2427397,2.2127523,2.2127523,5.375141,3.6328053,5.5374217,2.2127523,2.2127523,2.2268794,2.6563308,2.2287138,2.2221975,2.2127523,2.2127523,2.2783427,4.4782696,2.2127523,2.2127523,6.459867,2.2127523,4.5402617,3.0112247,2.2138321,2.2127523,2.2127523,7.874062,2.2127523,2.2127523,2.3070364,2.2127523,2.2260807,3.4530184,2.2138345,2.225428,2.8854234,10.403434,2.2127523,3.521635,2.2127523,2.8860085,2.2127523,2.2394419,2.2127523,4.59728,2.2142806,2.21401,2.3201795,2.2127523,2.2127523,4.498942,2.2127523,2.2260687,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.3173747,3.1841774,2.2127523,2.2127523,2.5981662,2.3226166,2.2127523,2.2127523,2.3955016,4.8430786,2.410918,2.2127523,3.1677701,2.2127523,2.246883,4.1525583,3.3627007,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.408152,2.2127523,2.2127523,3.120998,2.2127523,2.2127523,2.218812,3.5474281,2.2127523,2.2127523,2.5103362,2.2143729,2.573691,2.2127523,2.2127523,2.2127523,3.3997219,3.1847281,2.2127523,2.2127523,7.062108,2.2127523,2.2127523,2.2127523,2.2127523,2.8677676,2.642707,4.143969,2.2127523,2.2127523,3.19289,6.370284,2.2127523,4.5594516,2.2127523,2.2127523,2.2127523,2.315863,2.2127523,2.2127523,2.2127523,2.2127523,3.1459203,2.2141957,4.8146296,2.2127523,2.2127523,2.2165928,2.2127523,5.8479404,2.216439,3.6585321,2.2127523,5.65147,5.8973055,2.2127523,2.2127523,2.6192257,2.2127523,4.28181,2.2127523,2.3725789,2.2127523,2.2127523,3.6959453,2.2127523,2.2127523,2.4572499,2.2127523,2.2127523,5.222246,5.6512175,2.2127523,8.8523,5.5343323,2.2127523,2.2253344,2.2127523,2.2127523,2.215799,2.220447,2.2127523,2.2127523,2.2127523,4.2575836,2.230951,2.2127523,2.2127523,7.8816485,11.191419,2.2127523,2.2127523,2.2127523,2.21389,5.497407,2.2127523,2.2127523,2.2127523,2.2127523,2.2522705,2.3887372,2.2127523,2.2127523,2.2127523,4.3230104,2.2127523,2.2127523,3.4698527,2.2127523,2.2243047,2.25177,2.2127523,2.2141762,2.2127523,2.2127523,2.2127523,2.3678706,2.2127523,2.2127523,3.098214,2.240664,2.652172,2.2127523,3.575809,2.6431787,2.2127523,2.2127523,2.2127523,2.2127523,2.3537667,2.2127523,2.2127523,2.2283952,2.2127523,2.213601,2.219998,2.8395615,2.2127523,2.2127523,2.2127523,2.2162392,6.5470295,5.9629188,2.2127523,2.2127523,2.2192287,2.2127523,2.2127523,5.3918414,2.2127523,2.2127523,2.2127523,4.1510215,2.8407984,3.9863474,2.2157865,2.2127523,2.2127523,2.2127523,2.2127523,2.525035,9.937919,2.784391,2.2127523,5.1204386,2.2127523,2.2127523,2.2127523,2.2127523,2.2361803,2.2504354,2.2127523,2.2127523,5.5995417,2.2127523,2.3858755,3.8179054,2.3539233,2.2442045,2.2127523,2.2127523,2.2128077,2.2127523,2.2127523,2.2127523,2.8810918,2.2127523,6.9748945,3.770579,2.2127523,2.2127523,2.2127523,2.2900674,2.2127523,3.548069,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2400596,2.2127523,7.882948,2.3755312,2.2859366,2.2127523,2.2127523,2.2150934,6.2697616,4.870375,2.2127523,2.283333,8.264906,2.2182522,2.2127523,2.2127523,2.2127523,3.9951274,2.8495088,4.329595,3.5107567,2.2127523,2.2127523,2.2533858,2.2127523,5.191706,3.9668922,2.2127523,2.2265491,2.2194052,2.2127523,2.3010092,2.3975399,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.397297,2.2127523,2.2127523,5.5112505,2.4864726,2.325457,2.2127523,5.609307,2.2127523,2.2127523,2.2426715,3.596202,2.2127523,2.5057871,3.718277,2.2127523,2.2127523,2.2127523,2.213634,2.2127523,2.3019555,5.328646,4.01974,2.2127523,2.325933,4.3955393,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2138178,2.2127523,2.2127523,2.2127523,2.2184036,2.2127523,2.2127523,3.3896146,2.441026,2.2127523,2.2127523,2.2127523,2.2293227,2.2127523,2.2175796,2.2189417,2.2127523,2.2127523,4.826787,2.3652496,6.6995015,2.2127523,2.2127523,2.657558,2.2127523,2.6743588,2.2127523,2.2675061,2.291758,3.3839383,2.2127523,2.879031,2.2127523,2.2127523,2.2127523,4.1187973,2.8322163,2.2127523,2.2127523,2.2334816,3.313301,2.2833486,2.361647,2.2127523,6.858272,2.2127523,2.2134013,2.2127523,4.0134077,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,5.2306633,2.2127523,2.2127523,6.3143473,2.2127523,2.5149283,2.2127523,2.2127523,2.2127523,2.2127523,4.334621,2.3618839,3.1318681,2.2127523,2.2127523,2.21307,2.2127523,2.2127523,4.133813,2.2130547,2.9312499,2.2127523,2.2127523,2.2127523,4.4938507,5.3295016,3.5488846,2.366594,2.2902257,7.457546,2.9614305,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.9763467,2.2127523,2.2127523,3.0554059,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.9215636,2.2127523,2.2127523,2.2127523,7.710135,2.2127523,2.2127523,7.28149,2.2246382,2.2127523,2.214777,2.271164,2.234337,2.2127523,2.2127523,13.673629,4.6731744,2.2155657,2.2127523,4.164042,2.2648163,2.2127523,2.2127523,2.2127523,5.420713,3.1650217,2.9262116,2.2127523,6.397693,2.2127523,2.3238182,2.2836337,2.2127523,6.945431,2.2127523,2.2127523,2.2127523,2.2127523,2.2195556,2.2127523,6.325752,2.2649348,4.306993,2.2127523,2.2127523,13.448633,2.2127523,2.2127523,12.033553,2.2127523,2.2127523,5.271276,2.389612,7.029586,2.5882123,2.2127523,3.2689176,2.2606723,2.2127523,3.0138762,2.2127523,2.4915,6.808375,2.2127523,2.226038,2.2127523,2.2127523,11.921946,3.6585317,2.2127523,2.4091718,2.8055184,2.2127523,2.2127523,7.0414324,2.2127523,3.233873,2.2127523,2.2127523,2.2127523,2.2127523,6.9030814,2.5032163,2.2127523,4.3273835,2.3948827,5.344312,4.868082,2.2945988,10.44924,2.5291328,2.5135021,3.5988405,2.2127523,2.2822473,2.2127523,2.6546652,2.2127523,2.5447772,3.4332356,2.2449281,7.508159,4.7501326,8.052245,2.2127523,2.2127523,2.2127523,2.2261612,2.2127523,5.369206,2.2127523,2.2127523,2.3745565,2.2127523,2.2127523,3.3073287,2.2127523,14.480313,2.2127523,2.2248356,2.2127523,2.2127523,5.6245723,10.432855,2.2127523,2.2127523,2.2127523,2.3670187,2.2127523,12.659938,2.2127523,2.5759041,2.2127523,2.2127523,2.492868,5.887511,6.469288,5.88385,7.1043396,8.318263,10.317643,9.019523,2.2127523,4.372022,4.849932,5.6248913,8.267297,5.9427094,6.560059,2.2127523,5.2796297,7.891867,6.9491954,2.2533178,3.571506,2.213445,3.8984714,7.899184,7.3646007,14.352047,5.220825,14.329027,14.910566,10.390941,14.464773,4.127728,11.931023,10.676813,7.373941,8.629976,8.332947,8.339146,7.8604045,14.562111,8.926485,13.046509,11.792734,7.046602,10.3954935,15.133024,2.2127523,7.639925,2.6306446,14.602206,9.557076,14.8598,9.245633,5.8758287,14.063728,8.133803,14.02165,14.883469,8.544131,14.810954,5.7138004,14.920597,2.2127523,14.379527,9.11916,5.7829695,12.987655,9.122869,13.819098,15.721488,10.861539,12.348169,15.444748,4.993739,2.2127523,14.357406,15.471546,9.011437,14.369398,11.934786,14.26354,6.7534103,9.328009,2.9213777,11.002642,14.945541,14.684871,14.286736,2.3588908,8.938075,7.0504956,2.8289773,10.088414,11.969772,5.937006,8.326702,3.5642257,7.979534,4.4530993,7.838577,5.98464,2.2127523,7.3357015,4.4440947,7.744834,2.215445,2.260066,12.958126,10.511926,7.3622584,3.984219,13.210659,2.2127523,14.635337,5.536175,8.586728,7.879495,12.614092,6.7508664,14.78787,9.622328,15.533007,10.443622,9.349329,13.596154,11.253378,11.346925,11.776812,2.4680796,12.194409,14.71831,10.559748,8.037505,9.432506,8.011651,9.829939,2.6324544,2.2164266,14.871148,12.409612,14.844136,14.085077,7.0597987,4.2434254,3.0093927,14.382582,11.759547,6.5443673,8.404457,2.2469532,10.732937,3.3894873,5.240877,2.3828688,8.866124,5.314379,8.099135,2.2127523,14.334793,8.68728,9.568255,14.608851,2.2127523,6.463232,2.8289773,4.4645915,4.1624007,8.652842,4.673929,5.422546,10.025217,4.2717667,6.6170745,8.58271,8.342711,7.5299883,5.9790616,4.92366,2.2127523,8.530266,8.316337,9.923326,8.527815,4.102609,3.3309493,2.27077,5.521361,5.4988623,3.5185566,7.371816,8.340126,5.727065,5.7525096,3.3995159,4.437963,9.316445,2.2127523,2.2559972,7.2530165,8.959547,8.080008,9.33736,7.2534957,3.2829025,9.006756,11.600534,3.6048028,4.171691,5.479439,2.306011,2.3623612,2.8306718,2.2127523,2.2127523,2.7478566,2.258886,8.728985,5.3903265,9.790923,5.2729864,2.2127523,2.2127523,6.4420247,9.983068,2.2147737,3.3257434,4.9206634,6.1641283,6.0797744,4.1451902,3.4289262,5.2615767,5.8345513,9.321764,10.778173,4.3789845,2.2213106,2.73075,2.2127523,2.223748,2.2127523,7.7645345,2.2127523,3.677685,5.272429,3.384672,7.160469,7.0126834,3.0006306,2.9525466,7.023946,5.760694,4.1729107,5.4623127,8.521299,5.7204795,5.025147,7.6814723,6.9056964,6.97508,4.499714,8.009511,8.58984,8.019944,2.369784,2.2127523,2.2127523,7.3409343,4.779169,2.228434,7.7323956,7.5370626,5.2613025,3.5154889,2.2127523,2.6142612,11.17756,12.563497,11.305365,2.237062,7.036896,10.789007,11.730288,11.764029,3.7321742,14.468965,3.3516393,11.131047,2.2127523,3.0838013,6.7009077,10.080944,3.964018,2.2127523,4.6338563,7.9437065,2.2139368,7.8391733,4.33783,3.028134,6.304933,2.4692225,3.1724145,2.2232642,2.3169067,6.214536,7.9130616,11.080956,7.2445874,2.2127523,6.8422356,5.969179,6.7091208,2.2127523,2.2127523,2.2127523,6.5160766,2.213149,5.3766093,3.5719373,2.4009972,3.373812,6.186479,2.2127523,2.4055707,2.2524135,3.9280145,4.746054,5.256833,2.9909172,5.243871,8.16695,8.733492,9.429743,5.4082575,6.8919764,7.271347,9.50994,4.528823,7.6489916,4.3716116,2.742021,5.4489875,8.130254,7.5358243,12.63272,2.5575871,3.7517936,8.495682,8.011976,4.3221507,5.0076675,6.55785,7.2496195,7.4888735,4.902181,10.377359,9.683634,6.2816615,4.08817,2.2127523,6.3078637,4.7569427,10.088036,4.713048,5.562435,2.291646,5.0430913,6.2926307,6.989301,2.2127523,3.2042992,7.7932186,3.426912,9.012378,5.605022,9.864005,5.6464314,7.0680876,5.858737,8.746361,9.090513,3.4406483,2.3046732,2.2127523,8.215405,2.3643732,7.6200786,5.074931,3.5309691,3.138265,3.5287688,9.572497,7.4301558,2.5783749,3.2472894,7.543141,5.1859884,2.6179745,2.2127523,6.8706813,8.018785,4.278415,7.8726287,6.928462,6.2380605,2.3565838,2.2127523,4.1667957,9.336165,2.2127523,2.2127523,2.2127523,2.2127523,2.6158233,9.232845,11.153387,2.2127523,2.2127523,7.6063724,2.8855543,5.8319607,6.359783,8.067428,3.107817,2.2164786,2.2575164,2.2134564,4.7554603,2.2127523,2.2587721,2.302377,2.2127523,2.2127523,3.1273553,2.4583704,4.8733277,2.417997,5.0166416,9.496632,6.0724707,6.808356,2.2127523,5.5610776,7.1064353,9.750704,10.2268305,2.21418,12.354115,2.2127523,6.006818,2.2127523,8.131858,8.671773,4.7761683,5.127704,2.2127523,2.2127523,2.3076494,2.2127523,5.795621,3.711898,4.3246293,9.016857,7.6454554,6.1191263,12.240299,6.535189,3.881743,12.870391,12.827829,2.4424455,5.3746567,5.7511497,7.9143167,7.66456,7.877904,9.211719,10.384884,7.169521,7.931989,10.75336,5.9518514,8.500414,6.88527,4.243196,2.8692791,10.123963,12.094308,8.487072,3.4788916,2.2127523,3.2752557,5.6477203,2.6796134,12.16834,7.33665,10.742954,6.519583,4.183163,2.2129772,2.2139816,4.5336185,2.3043604,10.58743,2.862705,2.2721996,6.4873605,12.581685,5.2215223,8.999464,8.992518,6.0512867,7.473806,10.651989,4.1057973,9.3731575,6.948482,11.389899,2.306717,9.664856,2.2155223,7.20683,11.773235,10.057982,7.1806293,9.335772,9.088901,6.7682524,8.20072,6.5480695,2.2127523,7.086582,2.2127523,9.983982,11.648274,7.337985,9.247299,6.2296247,4.474999,7.07432,9.714807,4.365794,2.2159202,6.7680845,2.2184558,2.4178767,4.375661,2.7244656,4.4096837,8.6176605,8.667403,6.1910987,4.025185,2.2555144,5.020274,2.4645176,2.7153957,2.5409813,2.4151428,2.4914112,3.783812,2.2127523,2.2127523,2.2127523,3.8071425,6.6008244,5.844728,4.8297343,7.8007574,6.339334,3.7534456,12.571903,7.306614,11.11314,6.818302,8.597649,4.661775,4.5341473,5.637946,9.4676695,6.4809794,2.2127523,9.183827,3.7891448,2.9919043,2.7782075,6.814289,5.806542,3.6076875,2.7032883,6.5187654,7.839723,12.142546,3.4108598,10.030312,7.552315,4.432791,3.4509904,2.369706,11.756023,3.307804,9.962238,9.576818,11.379882,2.4937272,7.4941506,2.3763666,2.285726,2.2127523,3.6032906,2.2127523,2.3390138,2.2127523,2.2127523,2.688859,9.738097,2.2127523,2.2127523,2.2127523,2.2128496,2.2127523,2.2322037,2.2127523,2.2127523,4.0733824,2.2127523,3.6770792,6.15435,6.260312,2.2127523,8.336992,4.511734,4.739874,10.86097,5.919834,10.259663,10.1360035,6.9675207,2.213841,6.4553723,9.640351,6.5988917,5.6166606,7.255304,2.437352,7.1551085,5.1071734,5.320083,7.3146195,11.99752,6.2195625,2.7144914,9.914151,7.5417604,5.755763,4.5241346,6.19226,6.9375477,6.865966,6.392908,4.390604,6.427168,5.329314,3.6272695,7.452706,2.2127523,2.3020327,4.237918,6.2181525,8.336589,4.6759367,8.099298,10.513981,6.1203423,6.7298813,9.147097,9.661183,7.1101537,4.8707805,3.6601357,5.2767544,10.515804,8.552551,7.6564384,2.3983152,9.513501,6.1452994,2.2154043,6.209634,5.6314483,8.181876,6.069959,7.566856,11.830512,13.534612,15.351315,13.143184,12.461499,12.076027,6.3967624,4.763758,8.54375,5.6126766,2.76506,2.2127523,6.033473,2.894377,7.20236,2.6319664,2.2127523,2.3591793,11.395448,2.2323253,8.663173,7.075264,10.839859,8.265283,2.2612078,4.9385467,10.319664,8.99393,8.270398,7.20664,2.7002738,6.5861406,6.9781923,4.2771206,5.3263097,5.513553,7.241674,7.813282,4.2765408,2.2873454,2.977389,5.912118,3.6521773,8.6465025,2.2127523,2.2127523,4.499415,11.340502,8.104794,10.604086,2.7307642,4.4671235,5.1448298,10.216695,6.0056224,6.032641,7.157872,5.6098485,7.91517,2.3852675,2.2127523,2.2311478,2.2633636,9.76926,4.13588,8.419368,5.851437,2.2127523,4.4721293,8.060628,2.376252,3.798784,2.2127523,4.387546,2.2127523,7.018042,2.2127523,4.409898,2.2127523,2.3579402,3.8247068,8.0324335,5.576803,2.2127523,9.346871,2.2127523,6.452669,4.28254,2.2362523,6.356145,2.2127523,2.2127523,8.091124,2.4202957,10.033989,2.956123,7.1972775,13.132887,4.193473,10.449721,2.2127523,2.2127523,2.2127523,2.2127523,6.1964207,2.2351997,7.5438843,9.77081,2.2127523,2.486199,3.6930046,7.4042697,7.039851,4.060979,2.2937522,6.539311,4.7403502,2.2127523,4.490826,7.73248,6.7889366,2.2127523,6.2494016,8.963021,6.2242727,7.468935,3.752403,2.7411852,2.9299126,2.2127523,3.3058226,2.6830547,2.6751132,3.9918242,2.3191316,3.356241,2.2127523,7.347361,5.4913187,4.9100637,5.7807875,4.9859056,7.997481,2.2127523,2.3786395,8.013499,4.862989,4.548177,6.8243933,4.4128737,4.8962035,4.064805,5.3377404,3.3142083,2.420827,5.001058,2.3573625,2.784394,8.857736,5.6230845,5.066853,2.2486174,3.7250648,5.147848,8.487319,5.6946106,5.9613237,5.629491,2.2794907,4.0845246,2.2127523,5.276813,4.2614603,2.2127523,2.2127523,9.671683,3.1016304,5.0844574,7.4236846,6.193986,2.2188075,5.665631,7.120207,7.214416,11.184217,4.739682,8.676535,3.6200159,9.9545355,6.396079,3.0505304,2.2599752,7.9487486,7.9067016,2.2142873,10.720889,2.230487,2.240379,7.2282443,2.2127523,3.6304836,4.8709226,7.456818,2.456773,8.602134,2.2127523,3.663225,2.224244,5.3275847,9.91656,5.9740877,4.2633853,2.2127523,2.2669191,2.2137015,2.2127523,2.2127523,3.4305656,2.2127523,5.910624,7.973521,6.622119,3.2344959,2.2133086,3.1539106,2.5158854,6.251702,3.424477,7.167034,8.386475,6.002496,3.610807,2.7283382,8.341216,2.2127523,2.2127523,2.2127523,2.2790565,8.180635,2.2127523,2.6456761,2.2127523,4.325052,2.2127523,4.2172747,2.2127523,7.5981846,2.2127523,8.365257,4.4942865,6.4962125,2.2657814,5.5217395,5.168275,8.457998,2.2127523,2.2549438,9.549691,5.272604,2.2127523,2.2127523,2.2127523,2.2127523,4.899069,7.569007,2.7233078,2.6980462,8.509388,4.249872,2.2127523,2.8286617,2.2127523,2.2158132,8.525342,9.505133,3.9259975,3.7366214,2.2127523,2.2260447,2.223506,2.2127523,2.2195902,2.9297564,2.496419,3.335953,2.2127523,3.762464,2.2211955,6.8898907,4.7625246,6.110024,6.523958,5.465211,5.7341986,3.947546,7.0269732,5.830919,2.2127523,7.3187532,2.2127523,7.5220923,2.2127523,5.4283533,5.8049374,6.3501534,6.162473,6.146254,7.715525,3.6870317,5.7929735,4.98297,10.143727,3.6072123,2.2127523,9.150334,6.424403,2.2740076,3.938132,7.041416,4.96552,7.6867704,7.9485226,2.2127523,5.522351,6.4687567,2.2137806,3.162349,9.366632,7.8559465,5.7058287,9.141184,2.2417517,2.9445307,12.3273325,2.5646887,4.392363,2.267319,12.177386,2.2431455,2.2127523,8.907366,6.797112,2.2127523,4.418215,2.2127523,5.6554437,2.2592392,2.2127523,7.7668004,4.7633286,7.3106394,6.087785,5.238617,3.4295185,2.2127523,2.2127523,2.2127523,3.2527387,4.9114094,2.7789156,6.365781,2.2818744,3.852806,2.217475,6.299981,4.118839,3.099433,3.3141868,6.40176,2.2127523,7.3313704,2.2133832,4.317353,3.0110013,5.2939315,4.7159204,2.9633741,6.827559,2.2127523,2.2127523,2.264846,3.2106977,7.785456,6.00747,7.5052233,2.2127523,8.189699,9.35424,9.935623,2.2127523,7.224928,2.2504785,13.000123,2.2137132,9.471048,5.595627,9.654633,8.10856,9.091554,9.724363,7.0469084,5.956911,4.358746,5.3948684,4.9809713,6.7897077,2.2127523,8.18814,7.275228,5.208779,2.2127523,6.0903773,2.9110801,2.2127523,5.593258,5.5372715,2.9740894,4.4022384,2.2127523,4.9426904,5.538467,4.635296,4.0268683,9.1831045,5.8310585,6.8126855,4.5917554,2.2127523,11.24345,2.2127523,2.2127523,2.8711424,8.940893,4.9089622,6.6228037,8.308031,10.458167,7.7458496,2.2127523,6.011398,4.074255,2.2127523,4.1506047,8.724592,2.4975035,8.737207,7.4510374,5.4503684,4.398491,2.418006,7.7995105,12.381849,7.2479486,6.669747,6.2421904,5.8827453,5.508513,9.169709,7.3197255,7.6483717,5.3593655,6.3012586,5.52365,2.2127523,2.2127523,3.8914733,5.428186,4.057958,3.6965997,6.0524282,8.049375,2.2127523,4.098112,2.556775,2.2127523,4.818578,8.985344,6.8463755,2.2127523,5.826317,2.9848578,4.6852674,5.8502526,2.493094,8.689474,2.2127523,2.2127523,2.396562,6.9305353,2.6860626,2.6686666,2.2315173,2.2553465,2.2127523,2.275152,7.763358,2.2127523,3.9949448,5.1560845,2.2127523,2.2127523,2.4967108,2.4309902,3.7596228,2.583155,2.2127523,2.4366565,12.585389,2.2127523,2.7851884,2.8556192,2.3107712,2.5964012,2.2367518,2.611021,2.2249687,3.1293511,2.2151675,4.2738824,2.2127523,4.805592,2.2954981,3.802205,2.2127523,3.8443985,2.2127523,2.2127523,2.2127523,2.842486,2.2127523,2.2127523,7.1519637,6.680658,5.818094,8.980601,4.868083,2.3228853,3.1100092,2.2127523,2.2127523,2.232338,4.878802,3.7914004,2.2127523,2.2127523,8.132391,3.3676088,2.3080215,8.658586,4.2104716,5.9060316,4.5182104,2.4712708,2.2127523,2.2524052,3.917924,11.644761,2.2127523,6.232733,7.898951,5.916221,2.7074165,2.3069723,3.7863884,4.4251766,2.2127523,7.6001616,5.63555,5.3915224,9.154064,3.0366895,2.2127523,9.770764,2.6608229,7.1809587,2.2127523,3.7270436,2.2127523,2.3170643,2.2127523,2.2156243,4.854751,7.9025774,9.82376,6.03787,2.2127523,9.14879,2.2128677,10.059053,10.453707,9.445056,2.2127523,2.3435183,5.234417,7.11742,9.806054,2.2127523,7.254895,4.646898,6.444661,8.016137,2.2127523,4.1821756,7.8611116,5.580955,7.9730263,9.153687,8.874938,6.554028,6.0220456,6.9404964,9.146213,8.9567585,7.7467804,5.345671,8.1755085,7.482932,4.5895014,7.607688,4.7386227,7.649802,2.964098,5.467903,2.2316866,6.25684,7.4004126,8.957457,6.865173,2.7154644,6.710018,6.5314403,2.7737312,9.856393,2.2127523,2.2127523,5.6832476,7.59164,6.4374514,6.3989587,2.2127523,6.7654123,3.2448459,4.812169,2.2127523,8.2221575,4.0898724,5.8098125,3.7354033,4.2605696,5.063073,4.6089015,8.882815,11.59467,2.22041,10.02786,2.2127523,2.2279441,8.475704,2.2127523,2.4599795,2.6918242,7.8711767,3.4430215,2.2446387,2.6683416,7.604924,5.158436,2.3184767,2.2127523,8.970321,10.711221,2.5756743,2.4099348,4.260771,3.605527,3.8083448,5.025304,2.2304604,6.190196,3.5028431,2.2127523,2.2127523,2.2127523,9.77152,4.7130723,2.2391944,2.5776029,6.620062,3.987008,5.3855,10.885567,2.283759,5.979446,7.2748976,5.572246,4.82559,2.8090253,2.2127523,7.6418567,2.2639797,2.2488315,8.708584,2.2919548,6.646204,2.2127523,7.1845794,4.7255993,7.535018,6.281219,7.1156926,10.624709,4.8607125,2.5712335,2.2127523,3.5127072,8.382223,4.8959284,3.649201,9.906854,9.695256,3.1745737,5.3811755,8.266541,11.629035,6.0319524,4.4067984,10.255501,2.8771725,5.3073325,2.2127523,11.950941,13.587068,8.728351,11.118071,2.7348757,4.9636145,6.0178976,3.8512409,7.5366745,2.2196364,2.2127523,6.023276,2.2127523,2.2127523,2.2127523,2.2127523,3.7316017,2.2127523,9.393903,2.2127523,2.2137337,2.2137544,2.2136102,7.120115,2.439008,8.364321,3.9197335,5.3554587,2.2127523,7.6349726,8.765964,9.860236,8.011252,6.6751595,5.5519896,5.6630707,8.79289,8.523899,10.560617,6.753382,8.328801,9.106787,13.137571,10.895081,8.233571,9.633804,12.562279,7.7889476,4.118671,5.2449675,4.638271,2.2127523,2.2127523,3.715731,8.00057,9.78383,7.0300274,10.644189,7.5357857,5.94125,7.2157373,7.159601,7.1791306,6.307448,9.472374,6.5930743,4.9853635,7.9459014,4.131343,8.395364,3.3636348,4.2842813,5.20207,7.9580235,2.8548093,2.3224,3.2914374,10.801897,11.263884,2.2127523,2.2127523,5.910982,8.538122,7.753031,10.897578,8.82428,3.7948248,7.6785984,8.350499,2.2296443,2.5040803,3.9723454,5.397168,2.2127523,5.8511853,5.881216,8.033196,6.377561,6.2713084,12.096886,8.829103,4.1254473,3.0920181,2.2127523,2.2127523,10.5760145,5.1233716,3.2672937,4.798755,2.2127523,6.014711,2.2127523,2.2127523,5.123959,9.7066765,6.974237,6.2241116,2.2127523,5.1835685,2.2127523,3.415753,2.2127523,7.7832026,2.2129319,2.2127523,2.2127523,3.8274643,3.643919,2.2127523,2.7310247,3.7044308,2.3265073,2.6680038,2.2127523,2.2127523,2.2570937,2.2127523,2.2196589,7.500294,2.766847,2.2127523,3.31966,3.7545488,2.26126,5.2013974,2.2127523,6.6821294,7.023602,8.143247,4.221478,4.027975,5.8071465,2.2127523,5.8622475,2.3683746,2.2667472,2.2261932,2.2127523,6.203077,2.2127523,2.2127523,2.3102086,2.6541493,2.2127523,8.056176,2.2463527,3.921469,3.1957567,6.302566,4.78692,2.276442,2.6764486,9.550566,3.5508761,9.1567,2.2127523,2.483942,2.2127523,2.2127523,11.5568905,10.363888,7.4632792,2.251788,2.2127523,4.6920314,2.225993,2.9669473,3.7864416,2.2127523,5.2987785,7.922763,3.5098224,2.2127523,4.4515963,2.2140088,8.228653,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.3598578,3.0415862,2.2127523,2.2127523,2.2127523,2.2127523,3.2177107,6.6396794,3.435489,3.114704,5.8389235,2.2127523,2.2189338,2.2186189,6.1296034,2.2347028,6.453603,7.0891433,2.447341,2.3949032,12.412221,9.753867,9.233664,4.2852955,5.931267,2.2127523,6.162652,6.4939666,3.2209082,8.048382,4.1304474,3.7875082,2.2360988,6.445176,2.2127523,2.265144,5.1336455,5.083271,2.215218,8.729777,2.2127523,4.260301,3.3513696,2.2127523,4.999063,2.4655685,2.2127523,4.136314,4.6671734,10.159778,4.391667,2.5616841,7.1002274,11.213732,9.109929,7.490647,2.2277756,5.717134,2.784577,2.2127523,6.7154565,11.259056,2.2127523,7.461965,4.622161,2.2127523,7.3537126,4.566545,2.2158618,2.2127523,3.126314,3.3803625,6.1314383,2.2262008,5.047649,7.7970314,8.820497,10.162297,2.220681,10.024103,3.994759,2.2127523,6.017874,8.208228,9.369447,5.3884478,4.4146323,3.5402172,2.2127523,5.0694127,11.090635,7.0576596,2.2127523,2.2712033,9.138403,3.0498888,6.404344,6.1061873,10.203088,2.2127523,6.6360145,6.1786647,7.761562,5.484315,3.8255491,6.543261,6.297833,7.173835,2.2127523,2.2127523,5.752607,3.9068632,4.538694,2.3683414,2.768981,2.5349743,8.470424,5.3274937,4.9261107,10.810725,2.2176666,7.286486,2.2731159,2.2193491,2.2127523,5.2188816,11.223598,7.316314,8.373082,6.5955176,2.7898448,2.9129596,2.2127523,3.158214,6.030679,2.3586416,8.50705,2.2127523,4.837543,6.44582,6.2515025,8.132659,4.4390507,8.270403,3.4327533,5.576505,6.469649,6.6241956,2.4554417,10.956694,9.526345,2.2818408,7.5964303,2.2127523,2.2127523,13.173413,6.664244,10.398736,10.8905325,8.604636,13.4678135,7.894436,7.1509867,2.3184988,9.125348,7.4647245,6.329691,4.0727587,5.464463,7.0398602,9.175619,5.7933006,2.3396845,2.4250057,2.2127523,2.2127523,5.76116,8.889578,9.194121,8.3402195,2.2380502,8.844844,7.1904325,9.676126,9.83212,8.390987,8.102604,7.241705,2.2127523,8.071083,8.425346,9.374916,10.483611,6.4724813,4.436129,4.7272644,4.832009,5.6480756,6.297941,5.6552906,4.679528,3.4082892,5.6633215,2.2127523,7.540088,4.5239997,5.15953,9.793099,12.712744,8.475388,12.756787,2.2127523,8.152864,6.699846,2.2127523,2.2127523,3.0472374,2.3073564,2.2127523,2.2127523,2.5562243,2.2127523,7.969028,2.2127523,2.2127523,2.2127523,2.217842,2.2127523,3.9342368,2.2127523,2.2127523,11.029677,6.150556,2.2127523,7.1459227,6.178842,4.824039,2.4895525,2.7298894,3.2742605,11.601957,4.3470073,8.064212,6.0224586,2.2136517,2.29105,6.6423197,2.5331075,6.4701033,3.2168174,5.608691,2.2494736,2.214676,2.2127523,3.1889713,5.4363976,2.2127523,2.2127523,5.8239574,2.5818024,2.2127523,4.0181355,2.2130346,5.7329283,2.2127523,3.584559,6.332669,10.419047,2.4779296,5.3417954,8.823699,5.3678594,5.269358,9.51587,2.6344461,6.54061,5.3279886,5.400614,3.5194445,7.725851,2.2127523,5.3623543,2.94664,2.2127523,6.327992,9.346057,7.0722055,2.536775,2.2127523,10.49946,2.2258017,4.6719403,2.2526777,11.817068,6.4080153,7.925528,6.7833347,5.9120917,4.935897,7.5953817,2.2127523,10.821224,5.004835,2.2127523,2.8554454,2.8199165,2.3223693,6.21236,2.2525058,4.0086575,6.7712035,7.3920207,2.214329,2.2127523,7.0887413,3.5568101,6.6426435,9.6145525,5.9941163,4.2305765,5.308594,4.971208,6.4000607,6.8274665,5.7832747,5.6779075,2.2285397,7.0026164,5.8061824,2.418707,6.7905097,2.2127523,4.3861895,3.1192474,5.2361803,5.302349,5.257182,7.6445456,10.15674,5.400386,2.2127523,5.818364,2.2127523,2.2127523,4.795326,2.2127523,5.3080244,2.2127523,7.9784093,3.8191726,9.593074,2.2138298,4.109373,3.4520116,3.2582676,4.452422,3.0074234,2.2127523,9.571433,2.6687799,6.14644,7.607655,7.7541947,6.898068,6.0273843,2.35943,4.8565273,6.1163774,4.652593,6.986711,9.00274,11.4781065,9.879897,8.47936,5.0548186,9.699217,8.9267025,7.5530653,7.895444,2.2127523,2.5132453,2.780235,2.2127523,4.984183,5.4600887,7.6046076,6.2999306,6.5396404,6.3224106,5.584722,11.992502,5.2767787,6.2122464,4.922145,4.587584,2.2127523,2.2133088,4.8245525,4.327784,6.7785482,7.005527,10.000464,10.371745,6.295426,10.245871,2.4722998,6.8524623,10.762567,7.066121,2.5395603,4.0385356,5.353559,7.6933823,9.581222,2.4902987,3.343329,8.78274,6.1781335,7.2915845,8.118692,2.2602236,5.603709,2.5102806,2.2133667,2.2127523,5.1775084,2.2127523,10.060339,9.721563,10.16559,4.163384,10.118299,10.646053,4.6587567,7.8750367,2.2127523,2.5002003,5.991624,9.0796175,7.072713,3.7964127,9.063374,8.121151,5.7142873,7.1272473,7.086295,7.915141,3.7968242,10.456161,6.267295,6.9921823,7.1795506,2.2677376,6.662742,3.3587172,4.386584,4.501536,7.4846396,3.8355567,7.1586847,8.034667,3.3337395,2.2247431,2.2127523,2.7581859,6.6794486,8.295843,2.2127523,2.2247772,8.947208,7.2616305,5.642255,6.1660895,9.116993,2.6973312,2.2127523,4.187734,2.2127523,2.9135132,2.2127523,2.2127523,2.9442556,2.2127523,2.218147,3.8808694,6.0517864,6.3661323,8.115756,4.70313,2.2133906,2.2227778,2.2127523,6.6434007,7.8260064,2.725817,2.2437932,3.8570158,3.841126,2.2127523,5.3299465,2.5042436,3.312937,6.8834615,2.3573093,2.2127523,4.3534493,3.1164582,7.349707,3.1722517,3.928203,3.670193,2.217584,2.2127523,2.2127523,7.543859,2.4196506,9.997388,2.2335742,7.0764036,2.252756,3.799969,3.8495696,2.2127523,2.2127523,3.158606,2.2185745,7.099087,2.6043527,2.2127523,2.2127786,2.2127523,2.2254822,2.2323246,4.092494,2.2127523,8.072794,7.6957893,3.9276533,3.7899442,2.661998,7.1901994,3.5936356,2.6632967,2.2127523,2.2802875,2.2127523,2.2127523,2.2127523,2.2127523,2.2139068,2.2127523,2.919528,8.888805,4.5381927,2.2127523,2.2127523,5.54921,5.267768,5.656173,4.5445604,2.2185724,2.2127523,2.2138982,4.8133416,5.300963,3.6166062,6.5884047,5.1868668,4.962323,4.7599087,6.637244,8.354865,7.5459476,9.139355,7.211736,2.2131512,6.4091163,3.8807802,8.146528,2.2659788,6.975408,5.8058577,5.0677996,4.381337,5.5145483,7.7217393,8.106244,2.8849213,11.230797,2.2127523,3.2918117,6.4200935,6.0130215,8.58534,11.098588,2.2127523,3.1399329,2.2127523,11.698241,4.924894,5.4274354,6.799601,7.160416,4.110522,7.6538057,8.060431,3.6605146,7.455612,11.528267,5.9922166,7.1993227,8.6678,4.9736443,7.7043962,9.340146,6.302923,8.9265375,2.2127523,4.9801893,6.591796,4.929912,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.6165812,2.2127523,3.653779,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,4.055627,2.2127523,2.2707932,2.2127523,2.2127523,2.2127523,2.2127523,4.9760776,2.283801,9.609169,4.9479213,6.251455,11.61817,5.194953,2.2127523,3.4871376,5.4186916,6.9105077,6.4010525,5.1210275,5.0739913,7.91377,5.7864857,4.414937,6.367703,2.639024,6.833662,6.195717,2.2388253,6.832997,4.449169,8.816775,8.827934,4.285049,2.5176423,2.2127523,2.2996902,4.579767,3.3143294,3.883111,4.4534893,8.625955,7.5780234,4.0415425,4.3751383,5.057424,7.8534994,8.134912,12.714309,2.2127523,9.724262,9.497561,7.1499085,4.411013,7.776779,4.6192055,9.27492,8.885382,4.189756,4.9468036,7.7711797,2.2127523,6.3656983,4.221447,6.6186976,7.6241064,2.2127523,5.849071,3.615773,3.6415167,2.2180579,4.3412504,2.2127523,2.8313468,3.8103178,2.2127523,2.980881,2.2127523,3.2995985,2.768401,3.4041922,2.9842784,2.2263706,3.9984977,5.2654586,5.645289,6.355672,2.2127523,2.6099522,6.29293,4.477318,6.4773464,8.537885,2.2127523,8.775082,10.205546,2.6065776,8.841961,2.4111462,2.7475564,10.49969,2.5890374,9.670046,5.112747,5.057738,2.2143989,5.9959965,2.2127523,6.182932,2.280825,4.0150275,5.04738,3.5461366,2.2127523,7.236554,5.1546392,8.364594,5.904648,4.131434,7.8372245,6.613603,2.2127523,4.667753,8.959792,2.2896173,2.2127523,2.2127523,2.2127523,5.8645864,4.2383432,2.2127523,2.2141778,2.2127523,4.426901,2.605453,2.2127523,2.8100832,2.217212,2.2127523,2.2127523,2.2127523,3.1060457,2.2127523,6.174319,2.2127523,2.2127523,2.5188196,5.934999,5.3020153,4.2083077,6.877528,10.147561,2.2769186,2.634683,3.3328118,3.4569812,11.386023,2.2127523,4.943369,2.2127523,7.072362,2.2127523,3.27342,9.299044,2.9516094,2.2127523,4.7183347,6.3352957,5.628739,4.110634,5.9997916,4.763375,2.520382,6.118814,4.488072,5.491676,2.2127523,2.2127523,2.2127523,2.2271888,3.8089683,8.475031,3.9501753,2.372306,2.775729,2.2425272,2.2127523,4.9174137,6.323145,7.6946416,2.6281745,2.232861,2.2238119,2.3413754,8.211383,5.150293,8.269879,4.417429,9.121406,3.0136116,6.914929,5.0054584,8.067557,5.11057,3.4812617,2.5740294,6.1138353,4.7701616,6.38518,2.2160242,2.2127523,2.2127523,2.2127523,2.2127523,2.3542843,2.2127523,2.2127523,7.678307,2.2127523,2.2127523,2.2127523,7.588133,6.292844,2.2127523,2.2127523,6.1907587,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.223662,2.4009054,2.2127523,2.2127523,2.8515742,2.234286,6.586276,4.3303375,2.2127523,2.2127523,4.9155807,2.2127523,2.8976588,3.5719514,4.2885537,4.021302,6.9228673,5.0769315,5.735391,6.5095963,2.2712126,5.482759,6.0755296,5.779198,2.6428938,2.2127523,6.7489843,5.8448524,4.9367847,6.112539,13.558374,7.102763,2.7092273,7.276536,2.2127523,6.245563,6.5660324,5.592038,6.276329,2.5753005,3.7828217,11.525841,6.9035153,8.138132,4.4245462,5.002483,6.2256994,4.128615,2.2127523,2.2127523,5.0742574,2.2127523,2.2127523,2.819809,2.2127523,2.2127523,8.447898,5.842784,2.2127523,4.2815757,7.2645955,5.1399097,8.284592,3.6578958,2.8760576,7.69721,7.580102,6.8459253,7.8538065,2.2986405,4.833632,3.7695363,10.457975,7.8387876,4.8392,9.349305,6.6589527,7.4153438,2.2127523,2.2127523,7.3957877,2.2127523,5.3118644,5.864871,7.776131,7.293184,3.881614,4.4730277,8.714203,8.973894,5.095873,5.4402375,10.921695,6.0237513,2.2127523,3.0795653,5.47369,2.944536,2.7097225,2.342412,2.7380466,2.8662312,2.294383,7.3535476,10.319171,6.6839275,7.588419,6.321794,6.445678,8.271678,8.925228,2.2127523,6.6531053,4.927996,3.3478658,5.9242992,6.98419,2.231948,3.1333377,4.6024117,2.2300622,7.547386,7.057128,2.2127523,2.2229354,5.9949408,2.2463627,5.784031,7.3982606,2.2127523,4.3895426,8.232267,7.253512,5.408935,7.03972,4.598674,8.360854,4.9246316,7.3684425,6.897984,5.271713,5.7558465,3.357205,8.968519,6.9383793,9.769848,12.767654,10.382395,10.318633,2.2876496,3.480792,3.8064137,2.2152836,9.297536,10.76234,2.213887,7.0830073,8.427786,3.7563372,5.546591,9.322604,5.3866906,6.571285,7.345241,5.505066,8.543213,2.2127523,2.2127523,7.6189036,7.105048,3.1127799,2.2127523,5.600728,6.7786775,6.287753,5.2691646,2.376521,4.3379292,5.536849,2.2127523,2.2127523,4.6063924,2.7718792,7.1716175,4.0284543,9.527555,6.4540367,6.243831,7.4802814,2.4535346,5.242094,6.537532,2.2127523,2.2657976,2.2127523,2.2131133,2.2127523,6.976023,8.931805,9.60425,7.02228,4.013923,2.2127523,2.2127523,2.2127523,7.011016,2.2130754,6.530728,8.467801,5.9042087,11.55505,7.4168453,7.338527,6.9832273,13.707966,9.647705,11.916296,2.2542949,2.2127523,9.990317,6.336269,8.481782,2.6506357,6.287601,5.570509,2.2127523,2.9095833,6.0007095,9.977476,2.2127523,5.385925,3.819897,2.2127523,2.2127523,5.8194,6.8387794,9.064225,11.602276,4.065323,6.0678983,6.355437,5.0092363,10.000889,5.6839957,10.839264,4.0784945,6.0625925,7.4563165,2.397349,5.1342807,2.578556,4.590663,9.131498,5.9158893,4.7663765,3.5223672,2.2127523,7.7508388,6.3669057,4.4646616,2.2127523,6.790102,3.8371658,3.6346393,4.4844375,6.5821104,2.6515229,2.2503772,11.580977,9.084242,9.819279,6.429207,6.340748,9.671564,5.223929,3.6883543,6.4773703,3.2408814,2.2127523,3.1693432,2.3015554,2.2127523,9.612592,4.1526604,5.5010376,3.3571694,8.971218,2.8083856,7.649001,7.773191,3.8012567,4.0152764,2.2127523,9.897863,8.83481,11.904114,11.824169,8.154827,2.215502,7.5172567,2.2127523,2.4767456,5.6357613,2.3034768,3.0170517,4.307709,7.237638,6.326984,6.3077645,6.075246,4.6917934,6.194167,5.238653,11.074095,10.55527,8.839479,2.2294075,5.1223683,5.7772355,6.3982253,2.3242972,8.678583,2.4690704,6.369757,8.584348,6.1229424,4.0773034,2.8816617,2.3909945,2.2127523,7.9445353,4.744587,8.884475,3.3750732,8.40138,4.3533897,8.186074,7.810998,5.7743244,5.628464,7.0785117,5.657095,9.199627,6.277775,3.4409108,4.8461695,8.421542,4.9692016,2.782871,2.2127523,2.3139951,3.9303718,6.960201,7.478442,5.741884,2.5766387,3.5929163,9.299619,5.198276,9.196157,4.778474,2.2127523,8.844159,6.794588,4.9969687,3.06084,8.085297,6.8828936,2.2127523,4.2951436,5.8762774,5.5362663,2.2607396,7.3787136,3.905507,2.2127523,2.4803638,11.311005,4.3668394,2.270324,3.4259431,2.2127523,5.4263525,2.2127523,2.2452807,8.315157,3.4542158,11.008061,9.116881,3.2205775,9.422139,3.7101774,9.357585,2.2127523,2.2127523,6.2780623,2.2130795,6.560111,4.533431,8.529163,2.233349,11.834942,4.5042863,2.2127523,8.474341,8.295836,2.2127523,10.366592,3.4208758,11.466134,5.77496,2.2127523,7.9117107,2.6166878,5.034357,2.213842,5.624584,5.755147,4.7065787,2.4459593,7.703156,10.333192,6.8906097,6.493108,3.5547988,6.122945,7.435891,8.20023,5.9479403,8.301242,10.19106,7.3780303,5.631587,2.2127523,2.2666967,2.3856554,5.54452,6.8358717,6.7022753,8.025734,6.6113834,7.450443,7.444137,2.3800178,9.88069,4.5878563,5.2379937,6.5411744,8.410021,2.4824061,5.6854944,6.540352,2.9965196,6.8263416,9.429534,6.8822227,4.924203,2.244614,2.3844113,2.2127523,3.815092,2.2127523,4.039118,5.769457,3.3221383,2.8424592,7.693787,8.591767,4.9638567,7.47064,2.919512,9.316081,7.1073318,3.6714172,2.247741,6.3861914,6.4123163,6.3104587,9.423923,8.102715,2.342395,4.6469345,3.1558738,5.4910116,3.2020059,6.398396,2.285374,6.499035,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.8695726,2.5879202,6.142731,3.2162619,2.9835482,2.2127523,2.288069,14.6146965,3.3713565,2.2127523,15.843906,3.4879124,3.1458914,6.216271,5.653276,5.118115,5.6860394,7.6692247,10.109383,10.25297,6.145284,6.8738203,2.2127523,7.110222,2.412383,6.4317617,5.274443,8.733534,5.300865,7.2538977,5.817372,6.6539626,2.2127523,6.6766076,8.437363,4.412589,3.9902303,4.3358316,4.314733,8.518062,3.215367,2.2127523,7.9349456,7.648461,4.5851793,2.9249587,8.962439,7.59757,10.463083,4.090246,2.4580138,2.4081733,3.959826,10.304172,2.3782442,5.56576,8.722738,9.312147,9.62384,2.548835,7.4091854,2.2127523,4.918292,3.3966596,3.2909796,6.2197385,6.1556606,6.780283,3.5839338,4.0168357,3.0993347,2.2127523,3.9752321,6.6728663,6.2181664,11.909589,2.2302027,5.363944,3.0042603,2.2127523,2.2175198,2.3848655,2.8576372,7.223904,2.4653347,2.5040948,2.2226698,2.2127523,8.078854,5.8057613,8.404409,2.2209313,3.3287375,5.8398705,7.582214,3.3924832,6.3756137,6.3706493,6.5216184,2.3993442,4.2724204,2.2189276,11.471131,3.6390536,5.8693595,6.505957,4.5841756,2.4397542,3.4905684,10.330865,3.9172482,13.455236,8.12215,6.7642016,7.711994,6.746844,11.22412,9.182147,2.508903,10.441722,3.2493646,2.2127523,6.40052,10.899322,4.0805583,3.5081668,8.861757,9.369751,6.6585417,6.3292546,6.901495,5.424262,8.75575,3.6340506,7.5846124,5.4174623,5.0905533,3.7433658,8.149894,9.081526,7.3513813,2.2241764,5.138119,9.022662,4.1798472,5.696277,8.785882,5.44589,6.974639,6.073674,6.945802,2.2127523,2.2127523,2.2574356,2.219121,4.5430727,6.5504236,2.5075922,5.430245,4.198213,8.682709,5.7612004,8.642573,2.2127523,2.5281963,6.3237643,2.2127523,2.2127523,9.322422,5.953251,6.314778,3.8439353,5.4421654,3.283013,6.577601,5.448816,2.2127523,8.596647,6.0816035,4.3325777,2.220871,3.2612193,9.028603,5.7935653,3.5360987,7.819989,9.702579,5.8718066,2.2127523,7.269946,8.241807,2.2127523,2.2127523,5.5457673,8.747758,3.683787,2.228926,2.2127523,2.4692357,2.2277086,2.2301881,6.0977674,2.2127523,8.514077,8.055383,2.8326716,5.3976436,3.1408632,2.2938697,8.044598,6.478402,3.1468368,3.4405127,5.4505525,4.4030595,4.7735157,2.679731,5.525484,5.4711986,5.2309566,5.4339705,5.136801,5.9636507,2.2127523,6.640692,8.898744,11.796733,6.0544987,5.758295,5.3494997,6.0970006,5.5327954,8.764663,6.495115,6.522342,7.066164,5.5691705,6.3683224,5.343116,8.949998,2.6957278,8.343059,7.7602587,8.5334215,10.433439,4.2973766,7.221516,2.4446752,2.2127523,2.219024,2.2127523,2.2127523,10.59706,2.2127523,2.357783,3.710022,3.4594722,2.2127523,2.9940276,2.2127523,9.422915,9.146779,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.9687011,5.07417,2.2127523,2.7673545,2.2255926,2.2127523,8.980746,5.868557,6.104482,4.9675884,4.8239,2.2127523,5.116966,7.450282,5.8750305,2.4325266,6.047384,2.2127523,6.4910016,6.9844775,12.367152,5.874512,7.5470223,6.748305,7.064317,8.089141,9.943943,2.6157284,2.8392296,2.2127523,4.9508243,7.376931,6.9873214,5.6980343,2.9555018,6.194823,9.520416,7.1379285,6.327427,2.2127523,7.538764,2.2127523,2.2127523,7.151565,3.1887681,2.8557718,8.634938,2.2127523,5.72875,2.2127523,2.2742867,7.0330844,3.4588284,2.2127523,2.2152858,8.077483,2.240567,7.7215,7.9418006,4.2199407,3.310101,2.2307901,4.469307,3.81487,2.2127523,6.6906486,4.5741353,2.8081658,2.2127523,2.4557872,3.0803354,2.2127523,6.0314837,2.2157466,2.2127523,2.2127523,5.8621907,7.6626954,7.8229666,5.004128,8.883312,6.019159,9.361869,10.1158085,6.2847075,6.933998,5.46197,4.4553823,3.6741626,8.632919,8.007369,11.60646,7.1885757,7.8045745,7.7094135,5.149864,6.1597104,2.386892,9.43497,8.00755,4.3108115,7.6317763,5.5881834,5.0587654,2.466432,7.3723626,2.9476855,2.9775789,6.931928,2.2127523,3.1924653,2.2127523,2.2449152,2.2127523,7.813702,2.2127523,2.2127523,3.7321014,2.2127523,2.2127523,3.5713906,2.2127523,2.2559602,6.087888,2.2127523,2.2127523,4.2883744,2.2157278,5.5757375,7.6019583,5.549006,2.2127523,2.5989218,2.2127523,2.5424678,6.1251836,4.0917025,8.557151,2.2154598,2.2127523,8.934654,5.665973,5.9543266,2.9751856,4.51817,6.5845504,4.375973,7.4347644,5.0421247,9.201678,6.11022,5.4410834,9.957058,4.1461062,2.2127523,2.2127523,2.2127523,2.3048654,8.438591,3.1584098,3.919971,2.8644638,2.2127523,4.840458,2.2127523,8.291101,2.4901316,10.636909,2.3011565,5.5319567,3.6823473,3.7529511,5.4043694,13.2586,6.802554,5.2033525,2.6374602,4.9363737,2.9320557,2.705541,6.8674555,9.092444,3.2001102,6.4669933,9.218127,5.9682508,9.81593,7.3780894,5.5405746,5.125381,2.261704,10.149562,2.2127523,2.2127523,2.2127523,3.2664185,5.937031,6.9562526,7.752507,7.49985,2.2733662,8.272937,7.2814536,5.2084413,7.2412567,5.260396,2.2127523,3.0327604,2.2127523,2.2127523,2.2742593,8.534662,8.730449,10.129254,2.2127523,2.5239444,3.8136542,8.120639,3.3300223,9.321952,8.216449,2.516838,6.9443617,3.309195,8.566177,3.2717733,3.2770638,2.7611282,2.2127523,2.4446933,4.092279,4.8227487,5.815647,3.6351583,2.2127523,10.253698,5.5788913,5.554455,7.675848,7.773392,5.062363,2.6917534,8.865757,4.2025714,3.1740296,5.8214774,5.3850417,6.746706,2.2127523,5.2022777,2.4632008,3.6384704,6.302538,7.142894,4.6143603,3.2054057,6.700455,8.4138,4.88917,8.584896,5.1114793,4.1467834,4.921993,6.1373825,2.2141764,3.1594875,2.303445,6.8525004,2.2127523,2.2127523,3.0877864,3.3978486,2.824537,2.2127523,2.3625257,2.2127523,2.3325331,2.588927,3.65475,2.2127523,6.6510277,2.350863,2.268598,2.2127523,2.2127523,2.2127523,2.2127523,2.6168027,2.2307155,2.2127523,2.224582,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.953197,4.5559635,3.1044362,12.096404,11.259191,12.400231,6.814035,2.331643,9.62342,5.7228465,2.2127523,2.2127523,11.775488,2.213104,9.165128,3.4091058,11.161898,2.2127523,12.013098,2.2127523,10.002737,2.2445998,2.475533,3.278672,7.9886055,4.9952626,8.967869,5.449573,5.8852,6.4189754,6.687286,6.093887,3.133735,6.4149017,9.6052885,8.609894,2.2127523,2.2127523,7.884228,4.6608863,2.2127523,8.598319,7.358408,5.058046,2.2127523,2.2127523,2.2363105,5.569756,2.266618,10.72598,2.2288446,5.3864965,7.4379845,5.6336303,2.977371,5.0806785,8.292211,2.6444533,4.6160073,2.6735365,5.9917126,2.5197558,11.186147,8.028519,8.44415,10.436897,9.137277,6.83355,8.555093,2.231887,2.2127523,8.171787,13.428496,7.804426,6.669455,4.806298,6.646849,4.173019,3.353527,8.813156,8.105044,5.863426,2.5597646,2.2277822,2.549147,2.3495443,11.158467,2.2127523,2.762334,7.556779,7.3077035,4.563876,7.307624,6.21809,5.183251,7.3833594,3.1859074,2.2127523,2.2127523,2.2325478,5.015853,10.916192,3.8996682,6.003563,8.4100485,11.415385,8.510438,13.789054,5.8132176,7.7898064,7.979201,2.218952,3.0294056,8.316682,8.742613,2.7580407,8.972915,6.2368736,2.512259,3.9531925,14.564739,8.817651,3.8055968,14.331893,9.874803,7.261933,8.822582,6.7159495,8.654599,9.428394,5.830953,2.2127523,12.687596,8.738407,2.2127523,10.298639,7.9112735,9.376979,7.1018586,5.5677905,5.545242,7.8539767,5.2131953,9.811888,8.394454,11.20506,7.933116,9.579429,9.390401,3.7905777,6.4080925,9.525907,4.424777,2.644672,9.60225,7.918947,7.315446,2.2127523,4.7905126,6.956147,2.2127523,6.4455547,8.353588,5.8860497,3.872869,4.813191,2.8607523,10.537502,6.176122,6.8156714,4.330545,10.479147,7.1202455,2.2127523,11.470482,6.9303904,3.5035357,9.876394,7.4116745,5.7379246,4.8210554,9.864231,9.97206,8.542829,2.2127523,7.423048,2.2138164,8.079026,6.1013603,5.907676,3.7875104,8.77112,7.823235,4.6647387,8.811316,5.1845045,6.188684,3.4948225,2.2368913,5.229737,9.25822,7.755488,8.418658,8.190854,2.2127523,6.7467465,2.2313058,3.2009525,9.295823,6.056279,5.962819,4.886972,2.2127523,3.6482272,2.2127523,2.3492281,2.2127523,9.245028,2.5139081,6.7471952,12.071963,11.234724,6.758804,9.511135,2.2242558,9.315067,2.263954,10.245947,9.577704,6.277421,4.946807,8.717254,2.4767935,2.2127523,2.2150373,2.2127523,2.2183933,2.2127523,2.3779159,2.268739,2.2127523,2.5905693,9.63033,6.638057,5.296226,3.884127,10.056532,7.9620657,2.2127523,2.8281062,2.2514625,5.9051127,4.5913243,6.8534293,2.7797644,2.2127523,2.2253385,5.0408654,6.088571,5.968026,2.297801,10.441955,2.213497,4.5747924,5.6432786,2.2127523,10.5707445,2.2178147,6.211164,2.2127523,2.2783341,8.831272,2.2127523,2.2127523,2.2127523,2.2127523,5.4857492,8.399203,7.0212483,5.6648736,2.3218312,6.7455564,8.475009,7.862245,4.9479795,2.2127523,2.2146604,2.2127523,2.2127523,4.8596363,8.8982525,2.2127523,7.086911,5.0772295,2.2127523,9.38172,9.147938,9.392133,9.654494,9.518257,11.205265,12.115872,11.656393,2.7842302,5.4552627,5.6914945,6.370943,5.205599,2.2127523,6.916287,4.2367787,4.7199264,7.2256517,2.2133708,8.226276,2.2131917,2.2127523,2.2127523,3.3991687,12.908242,10.486559,9.951896,7.8300524,8.235137,6.549722,7.15565,9.629288,9.203406,7.8839707,5.6109543,2.3385468,2.2381606,5.224261,7.198221,4.552818,6.3488736,5.823649,2.2127523,5.463505,2.57089,5.8315983,2.2127523,8.223865,2.362699,6.800136,5.2809167,4.6765766,5.947744,9.84919,7.5043173,8.988657,3.9734533,5.649909,6.6367345,7.393802,8.151192,9.352129,6.30746,4.4566727,5.8417397,5.4020677,5.136983,4.4079194,3.7262397,7.5958376,6.2904115,2.66751,4.956039,3.706668,6.494105,8.388965,5.169945,6.901886,8.907555,9.207451,2.2127523,5.4057994,2.526283,2.2127523,5.068537,5.8672667,5.296543,6.3100967,5.6871424,7.2831717,9.006685,5.851125,7.2463965,3.9789677,2.2127523,2.2127523,3.1638858,6.190569,5.0660596,8.954738,10.074326,4.1293087,2.2127523,4.9930887,6.017206,10.395233,7.6135063,5.0596967,9.364053,13.243313,4.8577585,3.0608897,7.6926703,6.4586067,6.2159333,8.824208,3.5496352,2.8167694,4.5359535,8.437338,8.699361,2.2127523,6.9877963,5.0461016,4.1855626,2.2405996,10.336812,8.549668,9.795558,6.1745787,10.112103,3.7640877,7.2392774,2.2131789,2.2127523,7.424231,11.92465,8.117933,6.475873,10.465617,5.620805,4.2845974,2.2127523,2.2127523,4.280413,10.458075,8.960467,2.2127523,2.308474,2.2127523,6.026912,8.968449,3.3990247,2.8046894,4.579058,9.502137,10.771914,2.2616715,4.8904085,6.267177,2.3843837,2.2127523,9.058105,8.229281,3.404027,4.921815,3.0060363,12.318489,2.7848153,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,3.9463859,5.8487377,2.3739796,9.700296,5.070305,2.33281,3.0546062,6.536212,5.9970465,2.238931,5.653472,6.6834197,5.2442727,5.1965976,4.2581506,8.737972,7.002706,6.126671,10.06462,7.625523,4.7268815,7.0368958,8.251492,5.60284,7.2569623,7.7834725,7.133753,3.2434623,7.216109,6.718535,5.484994,7.6554585,9.761361,9.251405,9.724951,4.0977907,8.951105,8.615017,5.5710406,5.0998917,2.2127523,6.8560996,8.482907,4.483082,2.3163648,7.3034716,2.2127523,2.8016615,4.3481827,9.820343,3.758313,2.2127523,2.2127523,5.3589463,5.1830187,2.2127523,6.4436517,2.29129,3.994525,8.025638,3.5238822,3.2974505,2.2169766,7.158747,2.2127523,2.2134058,2.2127523,6.207893,2.2127523,4.9605827,2.2127523,2.4326532,6.9536567,4.9265623,8.298814,8.342047,7.1156096,10.866595,4.9417515,4.6786027,4.1275764,6.7338514,4.4748154,8.953055,4.103243,6.86803,7.6303363,8.392559,8.73487,2.2127523,7.625648,10.776261,4.6094527,2.858037,5.815081,2.2268264,2.2127523,9.169153,5.4405036,5.694863,3.2654057,4.3114405,4.2450438,5.4287343,7.7490315,9.222446,2.2127523,5.501999,5.3403215,2.5839016,2.6853108,6.726371,4.5978694,5.4693727,3.7060955,3.983515,8.285364,3.1339931,5.2192774,3.9106789,8.895906,5.006642,6.876979,4.6966515,2.2127523,4.213398,6.4985127,8.818541,4.8550034,8.458329,10.711712,7.3381076,8.342551,7.1940026,2.2364736,6.546547,4.474905,5.3743367,5.1574197,5.085167,5.3740153,6.3008637,5.133448,5.356565,7.0687566,2.4768825,2.2127523,2.2127523,2.2127523,2.8376198,4.07349,7.9152164,2.2335808,8.372377,5.529115,9.372018,2.2913241,7.462517,2.4789965,7.815627,11.0164175,5.5341897,9.412066,3.2484787,2.2225447,2.2127523,2.2127523,4.948484,4.4826107,5.6881514,7.857912,2.2932167,2.396615,8.553963,4.634785,8.313918,7.265284,2.2127523,3.8003387,2.2127523,5.1418004,2.2127523,2.4799528,2.2127523,5.551142,6.2756305,6.25569,2.2127523,2.2127523,2.3310974,2.2127523,2.2127523,2.2127523,5.9716926,7.6807394,3.395605,4.7265196,5.341208,2.2127523,4.1977563,7.5703807,7.372131,7.2581935,2.9622736,8.752784,6.2254987,4.0090456,2.9445755,2.2127523,4.2393594,8.720279,6.6984954,2.9973888,2.2127523,2.2127523,2.2127523,2.2852068,2.8278887,3.1305697,2.2137568,4.848199,2.4741127,2.328398,9.179557,3.0173664,2.3489811,2.2372837,2.6847878,2.2127523,2.2127523,4.2479687,2.2127523,5.5645704,5.401311,10.312321,6.0936785,4.089205,10.030644,10.709775,2.269425,2.314253,2.2127523,6.664188,4.649213,3.5927608,2.2127523,2.2127523,8.589732,4.0175157,9.82681,3.708086,7.856429,2.2127523,4.244798,9.2026,2.8558407,10.91269,3.1037476,2.2639031,9.538279,5.3797555,2.2127523,2.7092948,2.2127523,8.715395,2.3282375,2.2127523,10.638166,6.35085,2.2127523,2.59984,2.2127523,7.562476,4.2651267,2.2127523,8.037885,8.115726,6.514308,2.226149,5.324856,10.572287,5.426473,6.7954664,7.9457464,6.2919846,8.955002,9.56319,7.124571,7.627609,3.0804553,5.323113,9.098076,8.857249,4.397419,6.131849,9.185412,3.4215007,4.1241145,2.2127523,2.2477083,2.5430853,4.414322,4.1160765,14.142377,7.7837057,4.61929,9.491036,7.547926,8.760268,7.857896,8.8833685,2.2127523,6.870798,4.6925077,5.6557913,7.1777334,7.6355243,8.480799,4.3539057,7.2704535,6.4861155,9.413524,5.1839113,9.9555,8.143581,2.253065,6.2736793,6.0319324,6.9466643,8.468754,5.37469,6.4894166,7.2213297,2.4351184,10.6543255,6.8334475,5.8014216,6.490519,9.7876625,10.934375,9.320313,3.5089314,11.862612,10.08449,2.3850763,12.460167,4.6333704,12.676511,10.651024,7.475918,11.205276,2.3773286,6.3480463,6.5715113,2.2127523,5.5093064,2.2127523,2.6147122,5.682584,5.516666,5.6594296,2.854811,5.2179265,6.052311,7.8912606,4.024873,7.0434628,6.567779,3.5079424,4.45846,6.140555,5.229523,7.746727,2.5418968,2.2127523,5.6180844,3.608668,4.9610963,6.514373,6.350571,7.541276,2.694408,8.103666,3.7754085,4.0383687,3.7483406,6.0927925,5.9058266,2.2127523,5.801213,6.9910755,4.033719,7.3536663,5.1991773,9.190084,4.0467367,10.47293,3.410323,6.964135,7.0466456,3.3676054,2.6963708,2.2127523,4.4454174,4.6983347,4.2434254,8.243665,6.1595125,4.188153,4.582463,4.079881,9.0221,2.2127523,9.637763,4.269234,5.2746654,2.2127523,2.2127523,3.2928762,2.216782,4.348847,8.495493,6.3357334,6.123435,7.8382263,5.654163,3.6996815,5.863731,5.78639,8.973162,4.067148,6.263058,7.772207,5.931787,7.0636806,2.9258344,7.8256116,5.954813,2.2127523,2.2127523,4.2678843,7.5866947,3.1261702,5.92774,8.102847,3.712543,2.9327357,2.2127523,6.875628,9.298486,6.848354,9.058596,10.159136,3.38225,3.1408212,4.8724575,2.5414865,10.837248,4.8852205,4.3496933,2.7835479,2.3316097,9.410074,8.127361,3.237722,2.2484155,5.74195,7.0557356,6.2826166,2.3017993,4.2653666,4.127037,6.599648,5.5003185,8.533001,9.343394,9.14871,4.184709,2.4441457,7.9051976,6.2428455,5.2339797,3.8380442,9.850262,7.4522996,7.5773582,6.195262,3.0473418,6.5608826,9.048267,10.859497,4.2124443,5.7116303,3.3700514,5.1632566,2.9238272,2.2131584,3.5560365,2.3314214,2.440544,3.3010633,7.531099,5.186365,9.984314,6.005668,7.157256,3.8159902,3.2187853,2.2736459,2.2127523,9.284018,4.1039033,5.0557313,2.2268584,6.595531,5.945233,7.8871927,5.146511,5.1005154,4.1257696,4.8615303,8.36215,2.2128778,5.207159,3.0646033,2.2127523,4.853042,6.0307794,4.8371453,6.208611,7.4458966,7.835123,4.934337,3.3768144,2.632657,2.2127523,2.8689256,2.945598,6.711186,2.8756115,8.215899,3.2146592,5.8986692,3.7805784,2.2127523,5.502279,2.2127523,2.2127523,2.2127523,3.0743177,2.2127523,8.064667,3.9843357,2.801003,6.3772893,4.6908603,3.8244238,3.9675698,4.405822,9.705011,7.434104,2.3934462,6.656327,4.871959,2.965054,5.1403284,9.9874735,6.6003847,6.1708302,5.2732773,8.212809,6.577886,5.6665864,3.9742887,7.1444016,3.8303647,3.449695,7.408398,6.870782,4.1034684,4.6239614,6.550138,8.636756,4.126953,5.788547,2.337864,8.254548,10.331547,11.0877905,5.8321376,2.2188644,5.105561,6.2135096,2.3340452,7.5925,4.0943904,5.2144294,6.304295,9.174305,4.839985,5.752343,7.806491,9.981098,7.12475,2.2444055,3.8679504,2.2127523,5.2576327,7.2304955,4.100425,2.358698,2.9003952,4.2087913,4.188877,5.813103,6.3676224,4.480185,6.7211604,7.8167906,7.250931,8.985395,3.897263,8.034506,3.86231,3.8287442,6.4993386,6.30176,3.7703896,6.697237,5.575502,2.213486,8.95487,2.2248993,6.759673,7.612207,6.8379602,3.0738459,2.255531,3.0384681,9.940698,4.1547713,7.9392347,2.2127523,5.630855,3.1913385,6.132346,4.1753345,5.043691,6.0780787,2.9410603,7.287936,4.106552,7.408356,2.3587615,4.102758,3.2651505,5.494843,4.713519,6.698343,5.023483,6.807,3.5139802,5.6505637,5.8061423,2.2127523,6.5095615,6.6633644,3.1920943,2.2456892,4.2397056,6.0182633,9.080714,7.186908,9.287945,8.749401,5.3415008,5.3376284,7.238064,8.679902,6.5527787,5.0277095,6.7830753,2.2127523,2.2127523,2.881797,3.8965325,8.825282,5.1533947,4.7582984,4.782116,2.7173662,3.8361158,2.233517,6.4667277,2.2127523,2.2357693,2.2127523,3.7984595,5.9136543,4.0625153,2.4177055,7.454994,6.0280547,2.5834007,3.6758437,5.5439653,6.002518,7.5191135,7.961339,2.2127523,7.495012,2.2786558,10.054005,2.506683,2.3918097,4.6857862,5.8496037,7.050499,7.6567984,3.0139453,2.4950874,3.449815,7.4146924,4.828676,5.2208877,5.3519554,3.2218628,2.8648634,2.4285514,4.1623063,3.690603,5.195335,5.4612675,7.130393,3.083599,3.6881166,4.8371572,4.443836,2.6912584,2.684295,4.079805,4.771275,2.8517904,3.9109817,4.9160056,5.5017047,9.062501,2.2127523,2.220701,2.2127523,5.899642,4.1554193,2.9549856,4.6890497,7.095939,5.608786,3.3599513,6.2159567,3.0616834,2.343436,6.3311677,4.1139035,2.828307,2.5323014,6.1592474,6.4022384,5.290612,2.2127523,8.420028,3.8837583,2.2127523,9.021383,8.170962,8.7140255,7.5710716,4.913075,4.858461,5.0617146,3.8499353,3.7235956,3.7045276,4.6061783,3.7646441,6.346112,4.602305,3.7500215,3.842043,6.354191,6.391265,2.3163927,2.3819702,5.467084,3.8247552,8.297891,5.5819254,2.2517512,2.9443378,3.4781005,3.2508872,4.9087324,2.4598873,2.4246352,2.2127523,4.6017075,6.630353,2.2127523,5.078989,2.2127523,2.2127523,7.934685,3.1930285,2.2127523,4.219791,5.110463,5.0342245,4.097855,4.571194,2.4819274,5.3194017,4.6565623,5.4714475,8.573915,7.145452,7.2254243,5.5890512,8.5914,6.9068,2.2153792,5.9643216,4.054351,2.2395265,5.319434,3.9079823,3.0352068,3.4646292,6.459823,5.4583507,2.2205958,8.14866,2.2127523,3.0201445,2.2127523,3.6943245,2.2127523,5.588963,2.784237,2.4279337,5.032254,2.3014767,4.5438724,3.7253735,4.6011662,4.2443,3.6008978,2.2266767,3.4195938,7.0763235,8.931425,2.2127523,7.1479497,5.699296,3.8748913,6.6815414,5.942351,5.049507,5.975523,3.1765547,4.316457,5.8316097,4.4175243,4.0400915,2.2127523,2.4533703,5.850267,5.5346503,2.7198043,5.371167,3.4862204,2.2247283,6.194859,6.4797125,7.133361,3.3969886,2.792107,2.5810015,5.149076,4.8448505,4.188957,4.809706,8.673849,8.225373,4.8048277,5.8826056,7.201697,3.2200005,2.5620716,4.5303717,3.8247578,4.375953,2.654812,8.920831,4.3695073,5.2708983,4.7162595,4.074982,6.1498017,4.045623,6.358015,2.5179236,3.8032835,5.3516884,4.442547,6.955231,2.2127523,2.2145474,5.762971,9.791427,7.6523933,4.755257,5.1243773,4.8645353,5.6439414,3.1992793,5.950626,3.4236257,8.269304,6.1795273,4.93882,8.22689,2.2127523,2.8814785,4.4775677,9.646472,2.5411055,5.2221017,8.323354,5.660196,2.2127523,2.7531404,2.2127523,5.18937,10.067974,3.237372,2.280579,3.1953816,6.156266,3.267296,3.8125985,4.5203023,2.2127523,2.697211,6.2705255,2.9872503,5.9701014,8.415023,3.3204384,6.476695,5.163517,3.6416583,3.8785312,3.6201212,5.9406414,4.6730337,8.126959,4.362854,3.259635,4.0401874,2.2127523,8.711676,4.672585,3.5194972,4.868628,3.7194846,2.2127523,8.840961,6.223005,2.8875902,2.6780283,4.2849164,3.1643143,2.2127523,4.5081906,4.577076,9.313957,2.2127523,3.833143,6.5285573,3.3219242,4.3650956,4.604154,5.6384716,2.2156987,10.183243,8.770364,4.9305496,5.758241,5.209374,4.9638753,6.444068,8.425457,6.468846,3.9405122,6.01263,2.2127523,2.6600666,8.383927,3.1334395,2.8352938,3.6421459,2.6656427,2.4122524,5.7178097,5.9812703,5.2970543,6.1177106,5.7499957,3.902999,2.4695795,3.57354,4.1447544,6.471535,9.066102,5.06072,5.1093144,4.3161306,2.2142498,4.124821,2.388243,2.2127523,5.6114774,4.902688,2.2228694,6.0006323,4.681637,2.2127523,2.2127523,5.416818,2.5317554,7.43496,2.2341673,5.0425534,3.4060116,8.375596,7.5619473,2.5575256,2.29797,2.8945906,2.212858,4.9672217,8.224745,2.5919616,3.889743,4.7349114,3.7308118,2.2127523,3.8724024,2.234979,6.505345,2.7001672,2.2127523,5.0137405,4.220453,4.3457828,3.885105,2.2174764,5.981698,5.063427,4.3871875,2.2127523,9.3375025,9.569543,2.7084517,9.035259,6.2139573,4.2919407,2.49931,5.963439,5.3212748,2.2127523,8.072265,3.1460648,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,2.2127523,7.046812,6.7854495,10.693835,2.2127523,2.570894,7.202045,2.2127523,2.2127523,3.611079,2.2127523,6.5998116,3.2789419,5.642854,4.2705665,2.2436268,6.590305,5.028463,2.4214113,4.5953474,6.4895444,7.8383064,3.8715076,8.229744,4.217015,2.2127523,2.2127523,4.377183,3.809394,2.2127523,5.170274,2.2127523,2.6977625,9.426157,6.3461456,5.159025,6.255102,6.362769,6.7105746,8.48948,2.7373526,4.6452556,6.3553066,2.2127523,8.81963,8.111579,4.3730865,7.59391,13.764097,2.2127523,2.2183168,3.467357]

In [19]:
import os
import shelve
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client import models

# Initialize Qdrant client
client = QdrantClient("localhost", port=6333)

# Define the query filter
query_filter = models.Filter(
    must_not=[
        models.IsEmptyCondition(is_empty=models.PayloadField(key="ExperimentID")),
    ]
)

# Perform the search with the defined filter
response = client.search(
    collection_name='GPL570',
    query_vector=query_vector,
    search_params=models.SearchParams(hnsw_ef=128, exact=False),
    query_filter=query_filter,
    with_vectors=True,
    with_payload=True,
    limit=300
)

# Extract payload data, score, and vector from each ScoredPoint in the response
payload_data = [hit.payload for hit in response]
scores = [hit.score for hit in response]
vectors = [hit.vector for hit in response]

# Create DataFrame for metadata
lung_metadata_300_df = pd.DataFrame(payload_data)

# Create DataFrame for vectors
vectors_df = pd.DataFrame(vectors)

# Load gene names from the database
cache_path = "~/.local/share/biovdb"
cnames_path = os.path.join(os.path.expanduser(cache_path), "cnames")
with shelve.open(cnames_path) as db:
    gene_names = db.get("GPL570", [])

# Assign gene names to vector columns
vectors_df.columns = gene_names

# Extract 'GSM' values
gsm_values = [payload['GSM'] for payload in payload_data]  # Assuming 'GSM' key is present in the payload

# Add 'GSM' column to the data DataFrame
lung_data_300_df = vectors_df
lung_data_300_df['GSM'] = gsm_values

# Add 'Score' column to the metadata DataFrame
lung_metadata_300_df['Score'] = scores

# Reorder columns to move 'GSM' to the first column in lung_data_300_df
columns = ['GSM'] + [col for col in lung_data_300_df if col != 'GSM']
lung_data_300_df = lung_data_300_df[columns]


## Data

In [20]:
lung_data_300_df

,GSM,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,abParts,av27s1,hsa-let-7a-3,mir-223
0,GSM1001648,5.038562,2.415952,2.230390,7.193359,4.629608,2.220177,2.212752,2.592439,2.212752,...,6.355307,2.212752,8.819630,8.111579,4.373087,7.593910,13.764097,2.212752,2.218317,3.467357
1,GSM1001618,5.637071,3.320281,2.230390,6.635538,4.583446,2.212752,2.212752,2.592439,2.212752,...,6.078121,3.411802,9.678157,7.808173,4.238101,7.553570,10.300601,2.212752,2.213657,3.639912
2,GSM1001698,4.776393,2.415952,2.230390,7.716636,5.513504,2.212752,2.212752,2.592439,2.212752,...,5.573163,2.212752,8.529532,9.261359,4.405410,7.072839,11.280185,2.212752,2.266035,4.335000
3,GSM1001629,5.071888,3.469678,2.230390,7.221219,4.964468,2.212752,2.212752,2.592439,2.212752,...,5.757031,2.212752,8.646500,9.252953,4.277281,7.388263,12.584759,2.212752,3.301176,2.832930
4,GSM1001615,4.776393,2.415952,2.230390,7.883460,6.434856,2.212752,2.212752,2.560150,2.212752,...,6.769955,2.212752,8.463954,8.954547,4.563736,7.405613,12.800646,2.212752,2.282889,3.919336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,GSM703101,4.662750,2.400377,2.270522,6.762480,6.114451,2.272459,2.256923,3.251992,2.256923,...,5.137335,2.256923,7.622498,9.491187,4.295604,6.891183,12.235323,3.566876,2.983233,3.443233
296,GSM655716,5.757675,3.879776,2.480762,6.992297,3.770528,2.325346,2.223002,4.776174,2.134981,...,5.814338,2.670825,8.462850,9.020240,4.931974,7.106361,11.986553,2.614584,2.802089,2.334136
297,GSM655821,6.032594,3.831909,2.439870,6.695742,3.576077,2.281412,2.203537,4.750083,2.113939,...,5.294208,3.254327,7.978659,8.814510,4.664262,6.554324,13.463286,2.572962,2.327348,2.319774
298,GSM1589013,4.355996,3.720060,2.498229,6.746535,3.551318,2.396281,2.221412,4.694833,2.128402,...,5.391485,3.451036,8.297564,9.059227,4.618051,6.300982,14.415701,2.303686,3.924021,2.538993


## Metadata

In [21]:
lung_metadata_300_df

,Age,ExperimentID,GSM,PlatformID,Sex,Species,TissueID,TissueName,Score
0,69.0,40791.0,GSM1001648,570.0,0.0,Homo sapiens,763.0,lung,0.000000
1,60.0,40791.0,GSM1001618,570.0,1.0,Homo sapiens,763.0,lung,108.778180
2,68.0,40791.0,GSM1001698,570.0,1.0,Homo sapiens,763.0,lung,111.550760
3,74.0,40791.0,GSM1001629,570.0,1.0,Homo sapiens,763.0,lung,118.128650
4,74.0,40791.0,GSM1001615,570.0,0.0,Homo sapiens,763.0,lung,119.447075
...,...,...,...,...,...,...,...,...,...
295,NaN,28442.0,GSM703101,570.0,0.0,Homo sapiens,784.0,lymph node,178.003300
296,NaN,26639.0,GSM655716,570.0,NaN,Homo sapiens,149.0,breast,178.071330
297,NaN,26639.0,GSM655821,570.0,NaN,Homo sapiens,149.0,breast,178.130130
298,NaN,65216.0,GSM1589013,570.0,NaN,Homo sapiens,149.0,breast,178.137000
